In [ ]:
#!pip install textattack[tensorflow,optional]
#!pip install -U datasets

In [1]:
import os
import datetime

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision
import tensorflow as tf
import transformers
from transformers import CLIPProcessor, CLIPModel

import textattack
from textattack.models.wrappers import HuggingFaceModelWrapper

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
# SVC class is too slow to train for large dataset


D:\anaconda3\envs\tf\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
D:\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._p

#### Load CLIP Vocab Embedding

In [2]:
cfvocab = pd.read_table('../FMEmbeddings/counter-fitted-vectors.txt', header=None, sep=' ')
print(cfvocab.shape)
cfvocab = cfvocab.values.tolist()

(65713, 301)


In [3]:
# read counter-fitted vectors
cf = pd.read_table('../FMEmbeddings/counter-fitted-vectors.txt', header=None, sep=' ')
cf.set_index(0, inplace=True)

# check any non-string index
[idx for idx, w in enumerate(cf.index) if type(w) != type('word')]

[6167, 55574]

In [4]:
# cleaned
new_index = list(cf.index)
new_index[6167] = 'null'
new_index[55574] = 'nan'
cf.index = new_index
[idx for idx, w in enumerate(cf.index) if type(w) != type('word')]

[]

In [5]:
# create custom embedding attack

from textattack.shared import WordEmbedding

word2index = dict(zip(new_index, range(65713)))
index2word = dict(zip(range(65713), new_index))
nn_cf = pd.read_csv('../FMEmbeddings/nn_matrix_cf.txt',header=None)
nn_cf = nn_cf.values.tolist()

cfemb = WordEmbedding(cfvocab, word2index, index2word, nn_cf)

In [19]:
from textattack import Attack
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.goal_functions import UntargetedClassification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.transformations import WordSwapEmbedding, WordSwapWordNet 

from textattack.transformations.composite_transformation import CompositeTransformation

from textattack.attack_recipes.attack_recipe import AttackRecipe

class TFLepsilon(AttackRecipe):

    @staticmethod
    def build(model_wrapper, sim=0.5, thre=0.840845057):
        #
        # Swap words with their 50 closest embedding nearest-neighbors.
        # Embedding: Counter-fitted PARAGRAM-SL999 vectors.
        
        ## customised wordswapembedding
        transformation = WordSwapEmbedding(50, cfemb)
        #
        # Don't modify the same word twice or the stopwords defined
        # in the TextFooler public implementation.
        #
        # fmt: off
        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", 
             "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", 
             "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren",
             "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", 
             "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", 
             "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", 
             "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", 
             "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly",
             "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", 
             "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", 
             "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", 
             "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", 
             "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", 
             "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no",
             "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on",
             "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", 
             "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", 
             "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", 
             "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", 
             "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those",
             "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless",
             "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't",
             "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", 
             "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", 
             "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn",
             "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself",
             "yourselves"]
        )
        # fmt: on
        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]
        #
        # During entailment, we should only edit the hypothesis - keep the premise
        # the same.
        #
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        # Minimum word embedding cosine similarity of 0.5.
        # (The paper claims 0.7, but analysis of the released code and some empirical
        # results show that it's 0.5.)
        
        ### constraints.append(WordEmbeddingDistance(min_cos_sim=0.5))
        constraints.append(WordEmbeddingDistance(min_cos_sim=sim))
        #
        # Only replace words with the same part of speech (or nouns with verbs)
        #
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        #
        # Universal Sentence Encoder with a minimum angular similarity of ε = 0.5.
        #
        # In the TextFooler code, they forget to divide the angle between the two
        # embeddings by pi. So if the original threshold was that 1 - sim >= 0.5, the
        # new threshold is 1 - (0.5) / pi = 0.840845057
        #
        use_constraint = UniversalSentenceEncoder(
            threshold = thre,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        #
        # Goal is untargeted classification
        #
        goal_function = UntargetedClassification(model_wrapper)
        #
        # Greedily swap words with "Word Importance Ranking".
        #
        search_method = GreedyWordSwapWIR(wir_method="delete")

        return Attack(goal_function, constraints, transformation, search_method)


In [7]:
def runAttack(model_wrapper, n, path, target_ds, sim=0.5, thre=0.840845057):
    # used TFLepsilon
    attack = TFLepsilon.build(model_wrapper, sim, thre)
    attack_args = textattack.AttackArgs(num_examples=n, log_to_csv=path, shuffle=True, #parallel=True,
                                        checkpoint_interval=50, checkpoint_dir='checkpoints', 
                                        disable_stdout=True)
    attacker = textattack.Attacker(attack, target_ds, attack_args)
    attacker.attack_dataset()

In [8]:
# classifiers
def adaptation(train_set, test_set):
    lr1 = LogisticRegression(solver='saga', multi_class='multinomial') # default
    lr01 = LogisticRegression(solver='saga', multi_class='multinomial', C=0.1) # strong regularisation
    lr10 = LogisticRegression(solver='saga', multi_class='multinomial', C=10)  # weak regularisation
    
    rf7 = RandomForestClassifier(max_depth=7)
    rf14 = RandomForestClassifier(max_depth=14)
    rf30 = RandomForestClassifier(max_depth=30)
    
    knn7 = KNeighborsClassifier(n_neighbors=7)
    knn35 = KNeighborsClassifier(n_neighbors=35)
    knn175 = KNeighborsClassifier(n_neighbors=175)
    
    trainX = train_set.iloc[:,:-1]
    trainY = train_set.iloc[:,-1]
    testX = test_set.iloc[:,:-1]
    testY = test_set.iloc[:,-1]

      
    for key, clf in {'LR':lr1, 'LR C=0.1':lr01, 'LR C=10':lr10,  
                     'RF D=7':rf7, 'RF D=14':rf14, 'RF D=30':rf30,
                     '7-NN':knn7, '35-NN':knn35, '175-NN':knn175}.items():

        clf.fit(trainX, trainY) # fit train data
        pred = clf.predict(testX) # predict test set

        # print accuracy
        print(key, (pred==np.array(testY)).sum()/len(pred))

    return lr1, lr01, lr10, rf7, rf14, rf30, knn7, knn35, knn175

### BERT-based-uncased

In [9]:
# bert-based-uncased
model_path1 = 'bert-base-uncased'

bert_model = transformers.AutoModel.from_pretrained(model_path1)
bert_tokenizer = transformers.AutoTokenizer.from_pretrained(model_path1)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\anaconda3\envs\tf\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [10]:
bertagtrain = pd.read_csv('../dataset_embeddings/bertagnewstrain.txt', header=None)
bertagtest = pd.read_csv('../dataset_embeddings/bertagnewstest.txt', header=None)
bert_lr1, bert_lr01, bert_lr10, bert_rf7, bert_rf14, bert_rf30, bert_knn7, bert_knn35, bert_knn175 = adaptation(bertagtrain, bertagtest)

D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR 0.8943421052631579


D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR C=0.1 0.8864473684210527


D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR C=10 0.8940789473684211
RF D=7 0.7314473684210526
RF D=14 0.8014473684210527
RF D=30 0.8060526315789474
7-NN 0.7902631578947369
35-NN 0.795
175-NN 0.763421052631579


In [11]:
# custom model wrapper for BERT
class BERTModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [12]:
bert_lr1_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_lr1)
bert_lr01_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_lr01)
bert_lr10_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_lr10)

bert_rf7_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_rf7)
bert_rf14_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_rf14)
bert_rf30_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_rf30)

bert_knn7_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_knn7)
bert_knn35_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_knn35)
bert_knn175_ag = BERTModelWrapper(bert_tokenizer, bert_model, bert_knn175)

### CLIP

In [31]:
# Load CLIP model and processor
model_name = "openai/clip-vit-base-patch32"
clip_processor = CLIPProcessor.from_pretrained(model_name)
clip_model = CLIPModel.from_pretrained(model_name)

# Access text encoder
text_encoder = clip_model.text_model

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [32]:
clipagtrain = pd.read_csv('../dataset_embeddings/clipagnewstrain.txt', header=None)
clipagtest = pd.read_csv('../dataset_embeddings/clipagnewstest.txt', header=None)
clip_lr1, clip_lr01, clip_lr10, clip_rf7, clip_rf14, clip_rf30, clip_knn7, clip_knn35, clip_knn175 = adaptation(clipagtrain, clipagtest)

LR 0.915921052631579


D:\anaconda3\envs\tf\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LR C=0.1 0.9163157894736842
LR C=10 0.915921052631579
RF D=7 0.8757894736842106
RF D=14 0.9044736842105263
RF D=30 0.9101315789473684
7-NN 0.9248684210526316
35-NN 0.9210526315789473
175-NN 0.9093421052631578


In [33]:
# custom model wrapper for CLIP
class CLIPModelWrapper(HuggingFaceModelWrapper):
    def __init__(self, processor, text_encoder, classifier):
        self.processor = processor
        self.model = text_encoder
        self.classifier = classifier

    def __call__(self, text_input_list):
        inputs = self.processor(text_input_list, return_tensors='pt', padding=True, truncation=True)
        outputs = self.model(**inputs).pooler_output
        with torch.no_grad():
            preds = self.classifier.predict_proba(outputs)
        return preds

In [34]:
clip_lr1_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_lr1)
clip_lr01_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_lr01)
clip_lr10_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_lr10)

clip_rf7_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_rf7)
clip_rf14_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_rf14)
clip_rf30_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_rf30)

clip_knn7_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_knn7)
clip_knn35_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_knn35)
clip_knn175_ag = CLIPModelWrapper(clip_processor, text_encoder, clip_knn175)

### Run Attacks

In [13]:
target_ds_ag = textattack.datasets.HuggingFaceDataset('ag_news', split='test')

textattack: Loading datasets dataset ag_news, split test.


In [14]:
n_samples = 300

In [15]:
folder = '../attack_results_new/agcf/'

### Default parameters, threshold = 1 - 0.5/$\pi$ = 0.841, minSim = 0.5

In [16]:
suffix = 'ep0841.csv'
thre = 1-0.5/np.pi

In [17]:
st = datetime.datetime.now()

#### BERT

In [22]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattacklr1ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 43 / 2 / 5 / 50:  17%|████▏                    | 50/300 [11:34<57:51, 13.89s/it]textattack: Saving checkpoint under "checkpoints\1723391466421.ta.chkpt" at 2024-08-11 23:51:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 2 / 10 / 100:  33%|███████▎              | 100/300 [20:00<40:00, 12.00s/it]textattack: Saving checkpoint under "checkpoints\1723391972490.ta.chkpt" at 2024-08-11 23:59:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 131 / 4 / 15 / 150:  50%|██████████▌          | 150/300 [31:21<31:21, 12.54s/it]textattack: Saving checkpoint under "checkpoints\1723392653207.ta.chkpt" at 2024-08-12 00:10:53 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 173 / 6 / 21 / 200:  67%|██████████████       | 200/300 [41:20<20:40, 12.40s/it]textattack: Saving checkpoint under "checkpoints\1723393252168.ta.chkpt" at 2024-08-12 00:20:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 218 / 7 / 25 / 250:  83%|█████████████████▌   | 250/300 [52:09<10:25, 12.52s/it]textattack: Saving checkpoint under "checkpoints\1723393901765.ta.chkpt" at 2024-08-12 00:31:41 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 263 / 7 / 30 / 300: 100%|███████████████████| 300/300 [1:03:02<00:00, 12.61s/it]textattack: Saving checkpoint under "checkpoints\1723394554953.ta.chkpt" at 2024-08-12 00:42:34 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 263 / 7 / 30 / 300: 100%|███████████████████| 300/300 [1:03:02<00:00, 12.61s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 263    |
| Number of failed attacks:     | 7      |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 2.33%  |
| Attack success rate:          | 97.41% |
| Average perturbed word %:     | 14.95% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 188.96 |
+-------------------------------+--------+


In [23]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattacklr01ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 44 / 1 / 5 / 50:  17%|████▏                    | 50/300 [08:39<43:19, 10.40s/it]textattack: Saving checkpoint under "checkpoints\1723395075311.ta.chkpt" at 2024-08-12 00:51:15 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 1 / 11 / 100:  33%|███████▎              | 100/300 [17:07<34:14, 10.27s/it]textattack: Saving checkpoint under "checkpoints\1723395582806.ta.chkpt" at 2024-08-12 00:59:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 130 / 4 / 16 / 150:  50%|██████████▌          | 150/300 [27:47<27:47, 11.12s/it]textattack: Saving checkpoint under "checkpoints\1723396222786.ta.chkpt" at 2024-08-12 01:10:22 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 173 / 4 / 23 / 200:  67%|██████████████       | 200/300 [38:00<19:00, 11.40s/it]textattack: Saving checkpoint under "checkpoints\1723396835740.ta.chkpt" at 2024-08-12 01:20:35 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 217 / 6 / 27 / 250:  83%|█████████████████▌   | 250/300 [48:47<09:45, 11.71s/it]textattack: Saving checkpoint under "checkpoints\1723397482461.ta.chkpt" at 2024-08-12 01:31:22 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 262 / 6 / 32 / 300: 100%|█████████████████████| 300/300 [59:12<00:00, 11.84s/it]textattack: Saving checkpoint under "checkpoints\1723398107437.ta.chkpt" at 2024-08-12 01:41:47 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 262 / 6 / 32 / 300: 100%|█████████████████████| 300/300 [59:12<00:00, 11.84s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 262    |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 2.0%   |
| Attack success rate:          | 97.76% |
| Average perturbed word %:     | 15.75% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 194.46 |
+-------------------------------+--------+


In [24]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattacklr10ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 43 / 2 / 5 / 50:  17%|████▏                    | 50/300 [07:46<38:54,  9.34s/it]textattack: Saving checkpoint under "checkpoints\1723398574449.ta.chkpt" at 2024-08-12 01:49:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 86 / 3 / 11 / 100:  33%|███████▎              | 100/300 [15:56<31:53,  9.57s/it]textattack: Saving checkpoint under "checkpoints\1723399064124.ta.chkpt" at 2024-08-12 01:57:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 127 / 7 / 16 / 150:  50%|██████████▌          | 150/300 [26:53<26:53, 10.75s/it]textattack: Saving checkpoint under "checkpoints\1723399720691.ta.chkpt" at 2024-08-12 02:08:40 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 169 / 9 / 22 / 200:  67%|██████████████       | 200/300 [36:09<18:04, 10.85s/it]textattack: Saving checkpoint under "checkpoints\1723400276895.ta.chkpt" at 2024-08-12 02:17:56 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 213 / 11 / 26 / 250:  83%|████████████████▋   | 250/300 [46:41<09:20, 11.21s/it]textattack: Saving checkpoint under "checkpoints\1723400909416.ta.chkpt" at 2024-08-12 02:28:29 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 258 / 11 / 31 / 300: 100%|████████████████████| 300/300 [56:46<00:00, 11.36s/it]textattack: Saving checkpoint under "checkpoints\1723401514099.ta.chkpt" at 2024-08-12 02:38:34 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 258 / 11 / 31 / 300: 100%|████████████████████| 300/300 [56:46<00:00, 11.36s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 258    |
| Number of failed attacks:     | 11     |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 3.67%  |
| Attack success rate:          | 95.91% |
| Average perturbed word %:     | 14.43% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 188.13 |
+-------------------------------+--------+


In [25]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattackrf7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 34 / 2 / 14 / 50:  17%|████                    | 50/300 [04:58<24:50,  5.96s/it]textattack: Saving checkpoint under "checkpoints\1723401812294.ta.chkpt" at 2024-08-12 02:43:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 2 / 29 / 100:  33%|███████▎              | 100/300 [09:36<19:13,  5.77s/it]textattack: Saving checkpoint under "checkpoints\1723402090909.ta.chkpt" at 2024-08-12 02:48:10 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 103 / 5 / 42 / 150:  50%|██████████▌          | 150/300 [15:16<15:16,  6.11s/it]textattack: Saving checkpoint under "checkpoints\1723402431180.ta.chkpt" at 2024-08-12 02:53:51 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 135 / 7 / 58 / 200:  67%|██████████████       | 200/300 [24:34<12:17,  7.37s/it]textattack: Saving checkpoint under "checkpoints\1723402988910.ta.chkpt" at 2024-08-12 03:03:08 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 175 / 8 / 67 / 250:  83%|█████████████████▌   | 250/300 [30:20<06:04,  7.28s/it]textattack: Saving checkpoint under "checkpoints\1723403334539.ta.chkpt" at 2024-08-12 03:08:54 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 216 / 8 / 76 / 300: 100%|█████████████████████| 300/300 [36:37<00:00,  7.32s/it]textattack: Saving checkpoint under "checkpoints\1723403711557.ta.chkpt" at 2024-08-12 03:15:11 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 216 / 8 / 76 / 300: 100%|█████████████████████| 300/300 [36:37<00:00,  7.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 216    |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 76     |
| Original accuracy:            | 74.67% |
| Accuracy under attack:        | 2.67%  |
| Attack success rate:          | 96.43% |
| Average perturbed word %:     | 8.28%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 135.38 |
+-------------------------------+--------+


In [26]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattackrf14ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 37 / 3 / 10 / 50:  17%|████                    | 50/300 [06:29<32:27,  7.79s/it]textattack: Saving checkpoint under "checkpoints\1723404101489.ta.chkpt" at 2024-08-12 03:21:41 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 6 / 21 / 100:  33%|███████▎              | 100/300 [14:04<28:08,  8.44s/it]textattack: Saving checkpoint under "checkpoints\1723404556052.ta.chkpt" at 2024-08-12 03:29:16 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 113 / 9 / 28 / 150:  50%|██████████▌          | 150/300 [21:45<21:45,  8.70s/it]textattack: Saving checkpoint under "checkpoints\1723405017398.ta.chkpt" at 2024-08-12 03:36:57 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 145 / 11 / 44 / 200:  67%|█████████████▎      | 200/300 [30:47<15:23,  9.24s/it]textattack: Saving checkpoint under "checkpoints\1723405559180.ta.chkpt" at 2024-08-12 03:45:59 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 187 / 13 / 50 / 250:  83%|████████████████▋   | 250/300 [39:20<07:52,  9.44s/it]textattack: Saving checkpoint under "checkpoints\1723406072104.ta.chkpt" at 2024-08-12 03:54:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 228 / 14 / 58 / 300: 100%|████████████████████| 300/300 [47:58<00:00,  9.59s/it]textattack: Saving checkpoint under "checkpoints\1723406590333.ta.chkpt" at 2024-08-12 04:03:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 228 / 14 / 58 / 300: 100%|████████████████████| 300/300 [47:58<00:00,  9.59s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 228    |
| Number of failed attacks:     | 14     |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 4.67%  |
| Attack success rate:          | 94.21% |
| Average perturbed word %:     | 9.83%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 172.27 |
+-------------------------------+--------+


In [27]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattackrf30ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 38 / 4 / 8 / 50:  17%|████▏                    | 50/300 [07:23<36:58,  8.87s/it]textattack: Saving checkpoint under "checkpoints\1723407034433.ta.chkpt" at 2024-08-12 04:10:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 76 / 6 / 18 / 100:  33%|███████▎              | 100/300 [14:28<28:57,  8.69s/it]textattack: Saving checkpoint under "checkpoints\1723407459645.ta.chkpt" at 2024-08-12 04:17:39 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 118 / 9 / 23 / 150:  50%|██████████▌          | 150/300 [21:49<21:49,  8.73s/it]textattack: Saving checkpoint under "checkpoints\1723407900581.ta.chkpt" at 2024-08-12 04:25:00 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 151 / 13 / 36 / 200:  67%|█████████████▎      | 200/300 [32:32<16:16,  9.76s/it]textattack: Saving checkpoint under "checkpoints\1723408543496.ta.chkpt" at 2024-08-12 04:35:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 192 / 17 / 41 / 250:  83%|████████████████▋   | 250/300 [43:49<08:45, 10.52s/it]textattack: Saving checkpoint under "checkpoints\1723409220110.ta.chkpt" at 2024-08-12 04:47:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 232 / 19 / 49 / 300: 100%|████████████████████| 300/300 [53:48<00:00, 10.76s/it]textattack: Saving checkpoint under "checkpoints\1723409819523.ta.chkpt" at 2024-08-12 04:56:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 232 / 19 / 49 / 300: 100%|████████████████████| 300/300 [53:48<00:00, 10.76s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 232    |
| Number of failed attacks:     | 19     |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 6.33%  |
| Attack success rate:          | 92.43% |
| Average perturbed word %:     | 9.24%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 179.48 |
+-------------------------------+--------+


In [28]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattackknn7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 32 / 7 / 11 / 50:  17%|████                    | 50/300 [09:33<47:49, 11.48s/it]textattack: Saving checkpoint under "checkpoints\1723410393607.ta.chkpt" at 2024-08-12 05:06:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 10 / 20 / 100:  33%|███████              | 100/300 [18:48<37:37, 11.29s/it]textattack: Saving checkpoint under "checkpoints\1723410948443.ta.chkpt" at 2024-08-12 05:15:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 100 / 18 / 32 / 150:  50%|██████████          | 150/300 [30:10<30:10, 12.07s/it]textattack: Saving checkpoint under "checkpoints\1723411629932.ta.chkpt" at 2024-08-12 05:27:09 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 129 / 26 / 45 / 200:  67%|█████████████▎      | 200/300 [42:47<21:23, 12.84s/it]textattack: Saving checkpoint under "checkpoints\1723412387399.ta.chkpt" at 2024-08-12 05:39:47 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 166 / 32 / 52 / 250:  83%|████████████████▋   | 250/300 [53:19<10:39, 12.80s/it]textattack: Saving checkpoint under "checkpoints\1723413018870.ta.chkpt" at 2024-08-12 05:50:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 205 / 37 / 58 / 300: 100%|██████████████████| 300/300 [1:04:06<00:00, 12.82s/it]textattack: Saving checkpoint under "checkpoints\1723413666518.ta.chkpt" at 2024-08-12 06:01:06 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 205 / 37 / 58 / 300: 100%|██████████████████| 300/300 [1:04:06<00:00, 12.82s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 205    |
| Number of failed attacks:     | 37     |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 12.33% |
| Attack success rate:          | 84.71% |
| Average perturbed word %:     | 5.04%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 192.45 |
+-------------------------------+--------+


In [29]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattackknn35ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 36 / 5 / 9 / 50:  17%|████▏                    | 50/300 [09:57<49:48, 11.95s/it]textattack: Saving checkpoint under "checkpoints\1723414264316.ta.chkpt" at 2024-08-12 06:11:04 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 8 / 19 / 100:  33%|███████▎              | 100/300 [18:55<37:50, 11.35s/it]textattack: Saving checkpoint under "checkpoints\1723414801952.ta.chkpt" at 2024-08-12 06:20:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 14 / 28 / 150:  50%|██████████          | 150/300 [28:45<28:45, 11.50s/it]textattack: Saving checkpoint under "checkpoints\1723415392374.ta.chkpt" at 2024-08-12 06:29:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 138 / 19 / 43 / 200:  67%|█████████████▎      | 200/300 [41:31<20:45, 12.46s/it]textattack: Saving checkpoint under "checkpoints\1723416157863.ta.chkpt" at 2024-08-12 06:42:37 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 175 / 24 / 51 / 250:  83%|████████████████▋   | 250/300 [52:57<10:35, 12.71s/it]textattack: Saving checkpoint under "checkpoints\1723416844158.ta.chkpt" at 2024-08-12 06:54:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 216 / 28 / 56 / 300: 100%|██████████████████| 300/300 [1:03:04<00:00, 12.61s/it]textattack: Saving checkpoint under "checkpoints\1723417451139.ta.chkpt" at 2024-08-12 07:04:11 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 216 / 28 / 56 / 300: 100%|██████████████████| 300/300 [1:03:04<00:00, 12.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 216    |
| Number of failed attacks:     | 28     |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 9.33%  |
| Attack success rate:          | 88.52% |
| Average perturbed word %:     | 7.56%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 182.27 |
+-------------------------------+--------+


In [30]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/bertagattackknn175ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 41 / 0 / 9 / 50:  17%|████▏                    | 50/300 [07:08<35:44,  8.58s/it]textattack: Saving checkpoint under "checkpoints\1723417880342.ta.chkpt" at 2024-08-12 07:11:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 79 / 1 / 20 / 100:  33%|███████▎              | 100/300 [14:29<28:58,  8.69s/it]textattack: Saving checkpoint under "checkpoints\1723418320502.ta.chkpt" at 2024-08-12 07:18:40 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 115 / 2 / 33 / 150:  50%|██████████▌          | 150/300 [22:18<22:18,  8.92s/it]textattack: Saving checkpoint under "checkpoints\1723418789484.ta.chkpt" at 2024-08-12 07:26:29 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 149 / 4 / 47 / 200:  67%|██████████████       | 200/300 [31:34<15:47,  9.47s/it]textattack: Saving checkpoint under "checkpoints\1723419345656.ta.chkpt" at 2024-08-12 07:35:45 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 191 / 5 / 54 / 250:  83%|█████████████████▌   | 250/300 [40:31<08:06,  9.73s/it]textattack: Saving checkpoint under "checkpoints\1723419883345.ta.chkpt" at 2024-08-12 07:44:43 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 232 / 8 / 60 / 300: 100%|█████████████████████| 300/300 [49:12<00:00,  9.84s/it]textattack: Saving checkpoint under "checkpoints\1723420403958.ta.chkpt" at 2024-08-12 07:53:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 232 / 8 / 60 / 300: 100%|█████████████████████| 300/300 [49:12<00:00,  9.84s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 232    |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 2.67%  |
| Attack success rate:          | 96.67% |
| Average perturbed word %:     | 9.21%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 147.33 |
+-------------------------------+--------+


#### CLIP

In [35]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattacklr1ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 44 / 2 / 4 / 50:  17%|███▊                   | 50/300 [23:52<1:59:23, 28.66s/it]textattack: Saving checkpoint under "checkpoints\1723423534181.ta.chkpt" at 2024-08-12 08:45:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 4 / 9 / 100:  33%|███████              | 100/300 [30:49<1:01:38, 18.49s/it]textattack: Saving checkpoint under "checkpoints\1723423950729.ta.chkpt" at 2024-08-12 08:52:30 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 131 / 5 / 14 / 150:  50%|██████████▌          | 150/300 [38:29<38:29, 15.40s/it]textattack: Saving checkpoint under "checkpoints\1723424410741.ta.chkpt" at 2024-08-12 09:00:10 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 173 / 7 / 20 / 200:  67%|██████████████       | 200/300 [45:31<22:45, 13.66s/it]textattack: Saving checkpoint under "checkpoints\1723424832735.ta.chkpt" at 2024-08-12 09:07:12 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 219 / 7 / 24 / 250:  83%|█████████████████▌   | 250/300 [53:07<10:37, 12.75s/it]textattack: Saving checkpoint under "checkpoints\1723425288759.ta.chkpt" at 2024-08-12 09:14:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 261 / 10 / 29 / 300: 100%|██████████████████| 300/300 [1:01:44<00:00, 12.35s/it]textattack: Saving checkpoint under "checkpoints\1723425805882.ta.chkpt" at 2024-08-12 09:23:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 261 / 10 / 29 / 300: 100%|██████████████████| 300/300 [1:01:44<00:00, 12.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 261    |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 3.33%  |
| Attack success rate:          | 96.31% |
| Average perturbed word %:     | 19.17% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 229.41 |
+-------------------------------+--------+


In [36]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattacklr01ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 44 / 2 / 4 / 50:  17%|████▏                    | 50/300 [08:40<43:21, 10.40s/it]textattack: Saving checkpoint under "checkpoints\1723426326256.ta.chkpt" at 2024-08-12 09:32:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 4 / 9 / 100:  33%|███████▋               | 100/300 [15:25<30:51,  9.26s/it]textattack: Saving checkpoint under "checkpoints\1723426731960.ta.chkpt" at 2024-08-12 09:38:51 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 131 / 5 / 14 / 150:  50%|██████████▌          | 150/300 [22:33<22:33,  9.03s/it]textattack: Saving checkpoint under "checkpoints\1723427159955.ta.chkpt" at 2024-08-12 09:45:59 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 173 / 7 / 20 / 200:  67%|██████████████       | 200/300 [29:20<14:40,  8.80s/it]textattack: Saving checkpoint under "checkpoints\1723427566363.ta.chkpt" at 2024-08-12 09:52:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 218 / 8 / 24 / 250:  83%|█████████████████▌   | 250/300 [36:54<07:22,  8.86s/it]textattack: Saving checkpoint under "checkpoints\1723428020289.ta.chkpt" at 2024-08-12 10:00:20 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 260 / 11 / 29 / 300: 100%|████████████████████| 300/300 [45:37<00:00,  9.13s/it]textattack: Saving checkpoint under "checkpoints\1723428543699.ta.chkpt" at 2024-08-12 10:09:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 260 / 11 / 29 / 300: 100%|████████████████████| 300/300 [45:37<00:00,  9.13s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 260    |
| Number of failed attacks:     | 11     |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 3.67%  |
| Attack success rate:          | 95.94% |
| Average perturbed word %:     | 19.2%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 230.97 |
+-------------------------------+--------+


In [37]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattacklr10ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 44 / 2 / 4 / 50:  17%|████▏                    | 50/300 [08:42<43:34, 10.46s/it]textattack: Saving checkpoint under "checkpoints\1723429067065.ta.chkpt" at 2024-08-12 10:17:47 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 4 / 9 / 100:  33%|███████▋               | 100/300 [15:15<30:30,  9.15s/it]textattack: Saving checkpoint under "checkpoints\1723429459603.ta.chkpt" at 2024-08-12 10:24:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 131 / 5 / 14 / 150:  50%|██████████▌          | 150/300 [22:26<22:26,  8.98s/it]textattack: Saving checkpoint under "checkpoints\1723429890459.ta.chkpt" at 2024-08-12 10:31:30 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 173 / 7 / 20 / 200:  67%|██████████████       | 200/300 [29:14<14:37,  8.77s/it]textattack: Saving checkpoint under "checkpoints\1723430299036.ta.chkpt" at 2024-08-12 10:38:19 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 219 / 7 / 24 / 250:  83%|█████████████████▌   | 250/300 [36:28<07:17,  8.75s/it]textattack: Saving checkpoint under "checkpoints\1723430732589.ta.chkpt" at 2024-08-12 10:45:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 261 / 10 / 29 / 300: 100%|████████████████████| 300/300 [44:56<00:00,  8.99s/it]textattack: Saving checkpoint under "checkpoints\1723431240928.ta.chkpt" at 2024-08-12 10:54:00 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 261 / 10 / 29 / 300: 100%|████████████████████| 300/300 [44:56<00:00,  8.99s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 261    |
| Number of failed attacks:     | 10     |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 3.33%  |
| Attack success rate:          | 96.31% |
| Average perturbed word %:     | 19.14% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 229.34 |
+-------------------------------+--------+


In [38]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattackrf7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 33 / 13 / 4 / 50:  17%|████                    | 50/300 [10:04<50:20, 12.08s/it]textattack: Saving checkpoint under "checkpoints\1723431845408.ta.chkpt" at 2024-08-12 11:04:05 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 20 / 10 / 100:  33%|███████              | 100/300 [19:33<39:07, 11.74s/it]textattack: Saving checkpoint under "checkpoints\1723432415168.ta.chkpt" at 2024-08-12 11:13:35 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 105 / 28 / 17 / 150:  50%|██████████          | 150/300 [29:55<29:55, 11.97s/it]textattack: Saving checkpoint under "checkpoints\1723433036715.ta.chkpt" at 2024-08-12 11:23:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 134 / 37 / 29 / 200:  67%|█████████████▎      | 200/300 [38:15<19:07, 11.48s/it]textattack: Saving checkpoint under "checkpoints\1723433536879.ta.chkpt" at 2024-08-12 11:32:16 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 166 / 49 / 35 / 250:  83%|████████████████▋   | 250/300 [47:56<09:35, 11.51s/it]textattack: Saving checkpoint under "checkpoints\1723434118035.ta.chkpt" at 2024-08-12 11:41:58 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 197 / 62 / 41 / 300: 100%|██████████████████| 300/300 [1:00:39<00:00, 12.13s/it]textattack: Saving checkpoint under "checkpoints\1723434881247.ta.chkpt" at 2024-08-12 11:54:41 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 197 / 62 / 41 / 300: 100%|██████████████████| 300/300 [1:00:39<00:00, 12.13s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 197    |
| Number of failed attacks:     | 62     |
| Number of skipped attacks:    | 41     |
| Original accuracy:            | 86.33% |
| Accuracy under attack:        | 20.67% |
| Attack success rate:          | 76.06% |
| Average perturbed word %:     | 19.12% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 314.35 |
+-------------------------------+--------+


In [39]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattackrf14ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 31 / 16 / 3 / 50:  17%|███▋                  | 50/300 [12:42<1:03:30, 15.24s/it]textattack: Saving checkpoint under "checkpoints\1723435643537.ta.chkpt" at 2024-08-12 12:07:23 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 62 / 29 / 9 / 100:  33%|███████▎              | 100/300 [23:15<46:31, 13.96s/it]textattack: Saving checkpoint under "checkpoints\1723436277077.ta.chkpt" at 2024-08-12 12:17:57 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 94 / 43 / 13 / 150:  50%|██████████▌          | 150/300 [35:21<35:21, 14.15s/it]textattack: Saving checkpoint under "checkpoints\1723437003327.ta.chkpt" at 2024-08-12 12:30:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 115 / 59 / 26 / 200:  67%|█████████████▎      | 200/300 [44:50<22:25, 13.45s/it]textattack: Saving checkpoint under "checkpoints\1723437571872.ta.chkpt" at 2024-08-12 12:39:31 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 145 / 74 / 31 / 250:  83%|████████████████▋   | 250/300 [56:33<11:18, 13.57s/it]textattack: Saving checkpoint under "checkpoints\1723438274564.ta.chkpt" at 2024-08-12 12:51:14 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 172 / 92 / 36 / 300: 100%|██████████████████| 300/300 [1:09:27<00:00, 13.89s/it]textattack: Saving checkpoint under "checkpoints\1723439048890.ta.chkpt" at 2024-08-12 13:04:08 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 172 / 92 / 36 / 300: 100%|██████████████████| 300/300 [1:09:27<00:00, 13.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 172    |
| Number of failed attacks:     | 92     |
| Number of skipped attacks:    | 36     |
| Original accuracy:            | 88.0%  |
| Accuracy under attack:        | 30.67% |
| Attack success rate:          | 65.15% |
| Average perturbed word %:     | 17.94% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 350.72 |
+-------------------------------+--------+


In [40]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattackrf30ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 27 / 21 / 2 / 50:  17%|███▋                  | 50/300 [13:28<1:07:20, 16.16s/it]textattack: Saving checkpoint under "checkpoints\1723439857186.ta.chkpt" at 2024-08-12 13:17:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 36 / 7 / 100:  33%|███████▎              | 100/300 [25:02<50:04, 15.02s/it]textattack: Saving checkpoint under "checkpoints\1723440551357.ta.chkpt" at 2024-08-12 13:29:11 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 86 / 52 / 12 / 150:  50%|██████████▌          | 150/300 [36:58<36:58, 14.79s/it]textattack: Saving checkpoint under "checkpoints\1723441267883.ta.chkpt" at 2024-08-12 13:41:07 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 114 / 65 / 21 / 200:  67%|█████████████▎      | 200/300 [46:34<23:17, 13.97s/it]textattack: Saving checkpoint under "checkpoints\1723441843261.ta.chkpt" at 2024-08-12 13:50:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 140 / 84 / 26 / 250:  83%|████████████████▋   | 250/300 [58:51<11:46, 14.13s/it]textattack: Saving checkpoint under "checkpoints\1723442580996.ta.chkpt" at 2024-08-12 14:03:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 169 / 100 / 31 / 300: 100%|█████████████████| 300/300 [1:12:07<00:00, 14.43s/it]textattack: Saving checkpoint under "checkpoints\1723443376927.ta.chkpt" at 2024-08-12 14:16:16 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 169 / 100 / 31 / 300: 100%|█████████████████| 300/300 [1:12:07<00:00, 14.43s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 169    |
| Number of failed attacks:     | 100    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 33.33% |
| Attack success rate:          | 62.83% |
| Average perturbed word %:     | 16.27% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 355.62 |
+-------------------------------+--------+


In [41]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattackknn7ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 44 / 1 / 50:  17%|███▊                   | 50/300 [20:21<1:41:46, 24.43s/it]textattack: Saving checkpoint under "checkpoints\1723444598355.ta.chkpt" at 2024-08-12 14:36:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 78 / 4 / 100:  33%|██████▋             | 100/300 [40:15<1:20:30, 24.15s/it]textattack: Saving checkpoint under "checkpoints\1723445792225.ta.chkpt" at 2024-08-12 14:56:32 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 117 / 7 / 150:  50%|████████▌        | 150/300 [1:01:00<1:01:00, 24.40s/it]textattack: Saving checkpoint under "checkpoints\1723447037566.ta.chkpt" at 2024-08-12 15:17:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 150 / 14 / 200:  67%|████████████      | 200/300 [1:18:16<39:08, 23.48s/it]textattack: Saving checkpoint under "checkpoints\1723448073768.ta.chkpt" at 2024-08-12 15:34:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 48 / 185 / 17 / 250:  83%|███████████████   | 250/300 [1:40:00<20:00, 24.00s/it]textattack: Saving checkpoint under "checkpoints\1723449377473.ta.chkpt" at 2024-08-12 15:56:17 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 224 / 22 / 300: 100%|██████████████████| 300/300 [2:01:44<00:00, 24.35s/it]textattack: Saving checkpoint under "checkpoints\1723450681570.ta.chkpt" at 2024-08-12 16:18:01 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 54 / 224 / 22 / 300: 100%|██████████████████| 300/300 [2:01:44<00:00, 24.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 54     |
| Number of failed attacks:     | 224    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 74.67% |
| Attack success rate:          | 19.42% |
| Average perturbed word %:     | 5.8%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 512.82 |
+-------------------------------+--------+


In [42]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattackknn35ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 14 / 36 / 0 / 50:  17%|███▋                  | 50/300 [19:12<1:36:00, 23.04s/it]textattack: Saving checkpoint under "checkpoints\1723451834115.ta.chkpt" at 2024-08-12 16:37:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 65 / 3 / 100:  33%|██████▋             | 100/300 [37:40<1:15:21, 22.61s/it]textattack: Saving checkpoint under "checkpoints\1723452942600.ta.chkpt" at 2024-08-12 16:55:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 49 / 95 / 6 / 150:  50%|███████████           | 150/300 [56:02<56:02, 22.41s/it]textattack: Saving checkpoint under "checkpoints\1723454044075.ta.chkpt" at 2024-08-12 17:14:04 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 124 / 13 / 200:  67%|████████████      | 200/300 [1:13:07<36:33, 21.94s/it]textattack: Saving checkpoint under "checkpoints\1723455069141.ta.chkpt" at 2024-08-12 17:31:09 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 79 / 155 / 16 / 250:  83%|███████████████   | 250/300 [1:32:43<18:32, 22.25s/it]textattack: Saving checkpoint under "checkpoints\1723456245104.ta.chkpt" at 2024-08-12 17:50:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 91 / 187 / 22 / 300: 100%|██████████████████| 300/300 [1:52:18<00:00, 22.46s/it]textattack: Saving checkpoint under "checkpoints\1723457420216.ta.chkpt" at 2024-08-12 18:10:20 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 91 / 187 / 22 / 300: 100%|██████████████████| 300/300 [1:52:18<00:00, 22.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 91     |
| Number of failed attacks:     | 187    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 62.33% |
| Attack success rate:          | 32.73% |
| Average perturbed word %:     | 11.46% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 466.88 |
+-------------------------------+--------+


In [43]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agcf/clipagattackknn175ep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 24 / 25 / 1 / 50:  17%|███▋                  | 50/300 [16:28<1:22:21, 19.77s/it]textattack: Saving checkpoint under "checkpoints\1723458408765.ta.chkpt" at 2024-08-12 18:26:48 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 45 / 5 / 100:  33%|██████▋             | 100/300 [32:51<1:05:43, 19.72s/it]textattack: Saving checkpoint under "checkpoints\1723459392195.ta.chkpt" at 2024-08-12 18:43:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 68 / 9 / 150:  50%|███████████           | 150/300 [49:11<49:11, 19.67s/it]textattack: Saving checkpoint under "checkpoints\1723460371498.ta.chkpt" at 2024-08-12 18:59:31 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 89 / 90 / 21 / 200:  67%|████████████▋      | 200/300 [1:03:08<31:34, 18.94s/it]textattack: Saving checkpoint under "checkpoints\1723461209352.ta.chkpt" at 2024-08-12 19:13:29 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 114 / 28 / 250:  83%|██████████████▏  | 250/300 [1:19:07<15:49, 18.99s/it]textattack: Saving checkpoint under "checkpoints\1723462168075.ta.chkpt" at 2024-08-12 19:29:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 125 / 141 / 34 / 300: 100%|█████████████████| 300/300 [1:37:43<00:00, 19.55s/it]textattack: Saving checkpoint under "checkpoints\1723463284315.ta.chkpt" at 2024-08-12 19:48:04 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 125 / 141 / 34 / 300: 100%|█████████████████| 300/300 [1:37:43<00:00, 19.55s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 125    |
| Number of failed attacks:     | 141    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 47.0%  |
| Attack success rate:          | 46.99% |
| Average perturbed word %:     | 20.93% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 426.46 |
+-------------------------------+--------+


In [44]:
et = datetime.datetime.now()
et - st

datetime.timedelta(seconds=73141, microseconds=279532)

In [ ]:
here

### Threshold = 1 - 0.4/$\pi$ = 0.873

In [41]:
suffix = 'ep0873.csv'
thre = 1-0.4/np.pi

#### BERT

In [42]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 11 / 34 / 5 / 50:  17%|████                    | 50/300 [04:29<22:29,  5.40s/it]textattack: Saving checkpoint under "checkpoints\1722689100719.ta.chkpt" at 2024-08-03 20:45:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 57 / 10 / 100:  33%|███████              | 100/300 [08:36<17:12,  5.16s/it]textattack: Saving checkpoint under "checkpoints\1722689347148.ta.chkpt" at 2024-08-03 20:49:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 81 / 15 / 150:  50%|██████████▌          | 150/300 [13:00<13:00,  5.20s/it]textattack: Saving checkpoint under "checkpoints\1722689611230.ta.chkpt" at 2024-08-03 20:53:31 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 74 / 105 / 21 / 200:  67%|█████████████▎      | 200/300 [17:36<08:48,  5.28s/it]textattack: Saving checkpoint under "checkpoints\1722689886998.ta.chkpt" at 2024-08-03 20:58:06 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 94 / 131 / 25 / 250:  83%|████████████████▋   | 250/300 [22:34<04:30,  5.42s/it]textattack: Saving checkpoint under "checkpoints\1722690185231.ta.chkpt" at 2024-08-03 21:03:05 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 107 / 163 / 30 / 300: 100%|███████████████████| 300/300 [27:35<00:00,  5.52s/it]textattack: Saving checkpoint under "checkpoints\1722690486132.ta.chkpt" at 2024-08-03 21:08:06 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 107 / 163 / 30 / 300: 100%|███████████████████| 300/300 [27:35<00:00,  5.52s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 107    |
| Number of failed attacks:     | 163    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 54.33% |
| Attack success rate:          | 39.63% |
| Average perturbed word %:     | 15.44% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 66.71  |
+-------------------------------+--------+


In [43]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 14 / 31 / 5 / 50:  17%|████                    | 50/300 [04:05<20:25,  4.90s/it]textattack: Saving checkpoint under "checkpoints\1722690731413.ta.chkpt" at 2024-08-03 21:12:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 55 / 11 / 100:  33%|███████              | 100/300 [08:00<16:01,  4.81s/it]textattack: Saving checkpoint under "checkpoints\1722690967258.ta.chkpt" at 2024-08-03 21:16:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 53 / 81 / 16 / 150:  50%|██████████▌          | 150/300 [12:09<12:09,  4.87s/it]textattack: Saving checkpoint under "checkpoints\1722691216179.ta.chkpt" at 2024-08-03 21:20:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 102 / 23 / 200:  67%|█████████████▎      | 200/300 [16:29<08:14,  4.95s/it]textattack: Saving checkpoint under "checkpoints\1722691476117.ta.chkpt" at 2024-08-03 21:24:36 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 130 / 27 / 250:  83%|████████████████▋   | 250/300 [21:17<04:15,  5.11s/it]textattack: Saving checkpoint under "checkpoints\1722691763953.ta.chkpt" at 2024-08-03 21:29:23 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 160 / 32 / 300: 100%|███████████████████| 300/300 [26:14<00:00,  5.25s/it]textattack: Saving checkpoint under "checkpoints\1722692060451.ta.chkpt" at 2024-08-03 21:34:20 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 108 / 160 / 32 / 300: 100%|███████████████████| 300/300 [26:14<00:00,  5.25s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 108    |
| Number of failed attacks:     | 160    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 53.33% |
| Attack success rate:          | 40.3%  |
| Average perturbed word %:     | 16.13% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 66.27  |
+-------------------------------+--------+


In [44]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 11 / 34 / 5 / 50:  17%|████                    | 50/300 [04:09<20:47,  4.99s/it]textattack: Saving checkpoint under "checkpoints\1722692310285.ta.chkpt" at 2024-08-03 21:38:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 57 / 11 / 100:  33%|███████              | 100/300 [08:15<16:31,  4.96s/it]textattack: Saving checkpoint under "checkpoints\1722692556699.ta.chkpt" at 2024-08-03 21:42:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 53 / 81 / 16 / 150:  50%|██████████▌          | 150/300 [12:27<12:27,  4.98s/it]textattack: Saving checkpoint under "checkpoints\1722692808322.ta.chkpt" at 2024-08-03 21:46:48 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 104 / 21 / 200:  67%|█████████████▎      | 200/300 [16:59<08:29,  5.10s/it]textattack: Saving checkpoint under "checkpoints\1722693080135.ta.chkpt" at 2024-08-03 21:51:20 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 94 / 131 / 25 / 250:  83%|████████████████▋   | 250/300 [21:48<04:21,  5.23s/it]textattack: Saving checkpoint under "checkpoints\1722693368924.ta.chkpt" at 2024-08-03 21:56:08 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 162 / 30 / 300: 100%|███████████████████| 300/300 [26:39<00:00,  5.33s/it]textattack: Saving checkpoint under "checkpoints\1722693660140.ta.chkpt" at 2024-08-03 22:01:00 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 108 / 162 / 30 / 300: 100%|███████████████████| 300/300 [26:39<00:00,  5.33s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 108    |
| Number of failed attacks:     | 162    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 54.0%  |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 14.85% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 66.51  |
+-------------------------------+--------+


In [45]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 22 / 12 / 16 / 50:  17%|███▊                   | 50/300 [02:25<12:06,  2.91s/it]textattack: Saving checkpoint under "checkpoints\1722693805636.ta.chkpt" at 2024-08-03 22:03:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 26 / 31 / 100:  33%|███████              | 100/300 [05:04<10:09,  3.05s/it]textattack: Saving checkpoint under "checkpoints\1722693965074.ta.chkpt" at 2024-08-03 22:06:05 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 43 / 26 / 31 / 100:  34%|███████              | 101/300 [05:04<10:00,  3.02s/it]

[Succeeded / Failed / Skipped / Total] 71 / 39 / 40 / 150:  50%|██████████▌          | 150/300 [08:13<08:13,  3.29s/it]textattack: Saving checkpoint under "checkpoints\1722694153931.ta.chkpt" at 2024-08-03 22:09:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 89 / 56 / 55 / 200:  67%|██████████████       | 200/300 [11:34<05:47,  3.47s/it]textattack: Saving checkpoint under "checkpoints\1722694354735.ta.chkpt" at 2024-08-03 22:12:34 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 116 / 68 / 66 / 250:  83%|████████████████▋   | 250/300 [14:25<02:53,  3.46s/it]textattack: Saving checkpoint under "checkpoints\1722694526115.ta.chkpt" at 2024-08-03 22:15:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 141 / 81 / 78 / 300: 100%|████████████████████| 300/300 [17:40<00:00,  3.53s/it]textattack: Saving checkpoint under "checkpoints\1722694720518.ta.chkpt" at 2024-08-03 22:18:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 141 / 81 / 78 / 300: 100%|████████████████████| 300/300 [17:40<00:00,  3.53s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 141    |
| Number of failed attacks:     | 81     |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 27.0%  |
| Attack success rate:          | 63.51% |
| Average perturbed word %:     | 10.06% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 54.95  |
+-------------------------------+--------+


In [46]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 25 / 16 / 9 / 50:  17%|████                    | 50/300 [03:13<16:09,  3.88s/it]textattack: Saving checkpoint under "checkpoints\1722694914591.ta.chkpt" at 2024-08-03 22:21:54 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 49 / 31 / 20 / 100:  33%|███████              | 100/300 [06:20<12:40,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1722695101015.ta.chkpt" at 2024-08-03 22:25:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 48 / 29 / 150:  50%|██████████▌          | 150/300 [09:44<09:44,  3.90s/it]textattack: Saving checkpoint under "checkpoints\1722695305411.ta.chkpt" at 2024-08-03 22:28:25 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 67 / 45 / 200:  67%|██████████████       | 200/300 [13:26<06:43,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1722695527091.ta.chkpt" at 2024-08-03 22:32:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 91 / 51 / 250:  83%|████████████████▋   | 250/300 [17:44<03:32,  4.26s/it]textattack: Saving checkpoint under "checkpoints\1722695785276.ta.chkpt" at 2024-08-03 22:36:25 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 128 / 114 / 58 / 300: 100%|███████████████████| 300/300 [22:06<00:00,  4.42s/it]textattack: Saving checkpoint under "checkpoints\1722696047648.ta.chkpt" at 2024-08-03 22:40:47 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 128 / 114 / 58 / 300: 100%|███████████████████| 300/300 [22:06<00:00,  4.42s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 128    |
| Number of failed attacks:     | 114    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 38.0%  |
| Attack success rate:          | 52.89% |
| Average perturbed word %:     | 10.94% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 60.54  |
+-------------------------------+--------+


In [47]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 20 / 21 / 9 / 50:  17%|████                    | 50/300 [04:05<20:29,  4.92s/it]textattack: Saving checkpoint under "checkpoints\1722696293716.ta.chkpt" at 2024-08-03 22:44:53 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 39 / 18 / 100:  33%|███████              | 100/300 [07:30<15:00,  4.50s/it]textattack: Saving checkpoint under "checkpoints\1722696498139.ta.chkpt" at 2024-08-03 22:48:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 60 / 27 / 150:  50%|██████████▌          | 150/300 [11:15<11:15,  4.50s/it]textattack: Saving checkpoint under "checkpoints\1722696723345.ta.chkpt" at 2024-08-03 22:52:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 76 / 82 / 42 / 200:  67%|██████████████       | 200/300 [15:05<07:32,  4.53s/it]textattack: Saving checkpoint under "checkpoints\1722696953619.ta.chkpt" at 2024-08-03 22:55:53 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 109 / 48 / 250:  83%|████████████████▋   | 250/300 [19:28<03:53,  4.67s/it]textattack: Saving checkpoint under "checkpoints\1722697215959.ta.chkpt" at 2024-08-03 23:00:15 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 137 / 55 / 300: 100%|███████████████████| 300/300 [23:51<00:00,  4.77s/it]textattack: Saving checkpoint under "checkpoints\1722697479377.ta.chkpt" at 2024-08-03 23:04:39 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 108 / 137 / 55 / 300: 100%|███████████████████| 300/300 [23:51<00:00,  4.77s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 108    |
| Number of failed attacks:     | 137    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 45.67% |
| Attack success rate:          | 44.08% |
| Average perturbed word %:     | 8.42%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 62.27  |
+-------------------------------+--------+


In [48]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 22 / 17 / 11 / 50:  17%|███▌                 | 50/300 [25:56<2:09:42, 31.13s/it]textattack: Saving checkpoint under "checkpoints\1722699036077.ta.chkpt" at 2024-08-03 23:30:36 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 35 / 20 / 100:  33%|██████▎            | 100/300 [30:44<1:01:28, 18.44s/it]textattack: Saving checkpoint under "checkpoints\1722699323667.ta.chkpt" at 2024-08-03 23:35:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 62 / 56 / 32 / 150:  50%|██████████▌          | 150/300 [35:46<35:46, 14.31s/it]textattack: Saving checkpoint under "checkpoints\1722699626100.ta.chkpt" at 2024-08-03 23:40:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 80 / 75 / 45 / 200:  67%|██████████████       | 200/300 [41:00<20:30, 12.30s/it]textattack: Saving checkpoint under "checkpoints\1722699940101.ta.chkpt" at 2024-08-03 23:45:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 101 / 97 / 52 / 250:  83%|████████████████▋   | 250/300 [47:01<09:24, 11.29s/it]textattack: Saving checkpoint under "checkpoints\1722700300820.ta.chkpt" at 2024-08-03 23:51:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 122 / 120 / 58 / 300: 100%|███████████████████| 300/300 [52:54<00:00, 10.58s/it]textattack: Saving checkpoint under "checkpoints\1722700653907.ta.chkpt" at 2024-08-03 23:57:33 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 122 / 120 / 58 / 300: 100%|███████████████████| 300/300 [52:54<00:00, 10.58s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 122    |
| Number of failed attacks:     | 120    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 50.41% |
| Average perturbed word %:     | 5.12%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 59.77  |
+-------------------------------+--------+


In [49]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 20 / 21 / 9 / 50:  17%|████                    | 50/300 [05:03<25:18,  6.08s/it]textattack: Saving checkpoint under "checkpoints\1722700958518.ta.chkpt" at 2024-08-04 00:02:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 40 / 19 / 100:  33%|███████              | 100/300 [09:51<19:42,  5.91s/it]textattack: Saving checkpoint under "checkpoints\1722701245779.ta.chkpt" at 2024-08-04 00:07:25 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 62 / 60 / 28 / 150:  50%|██████████▌          | 150/300 [14:57<14:57,  5.98s/it]textattack: Saving checkpoint under "checkpoints\1722701551806.ta.chkpt" at 2024-08-04 00:12:31 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 77 / 80 / 43 / 200:  67%|██████████████       | 200/300 [20:08<10:04,  6.04s/it]textattack: Saving checkpoint under "checkpoints\1722701863690.ta.chkpt" at 2024-08-04 00:17:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 106 / 51 / 250:  83%|████████████████▋   | 250/300 [26:54<05:22,  6.46s/it]textattack: Saving checkpoint under "checkpoints\1722702269307.ta.chkpt" at 2024-08-04 00:24:29 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 112 / 132 / 56 / 300: 100%|███████████████████| 300/300 [33:05<00:00,  6.62s/it]textattack: Saving checkpoint under "checkpoints\1722702640284.ta.chkpt" at 2024-08-04 00:30:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 112 / 132 / 56 / 300: 100%|███████████████████| 300/300 [33:05<00:00,  6.62s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 112    |
| Number of failed attacks:     | 132    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 44.0%  |
| Attack success rate:          | 45.9%  |
| Average perturbed word %:     | 8.71%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 61.54  |
+-------------------------------+--------+


In [50]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 27 / 14 / 9 / 50:  17%|████                    | 50/300 [04:24<22:01,  5.29s/it]textattack: Saving checkpoint under "checkpoints\1722702904740.ta.chkpt" at 2024-08-04 00:35:04 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 48 / 32 / 20 / 100:  33%|███████              | 100/300 [08:57<17:55,  5.38s/it]textattack: Saving checkpoint under "checkpoints\1722703178399.ta.chkpt" at 2024-08-04 00:39:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 71 / 46 / 33 / 150:  50%|██████████▌          | 150/300 [13:22<13:22,  5.35s/it]textattack: Saving checkpoint under "checkpoints\1722703443098.ta.chkpt" at 2024-08-04 00:44:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 60 / 47 / 200:  67%|██████████████       | 200/300 [18:23<09:11,  5.52s/it]textattack: Saving checkpoint under "checkpoints\1722703743803.ta.chkpt" at 2024-08-04 00:49:03 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 114 / 82 / 54 / 250:  83%|████████████████▋   | 250/300 [24:26<04:53,  5.87s/it]textattack: Saving checkpoint under "checkpoints\1722704107308.ta.chkpt" at 2024-08-04 00:55:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 138 / 102 / 60 / 300: 100%|███████████████████| 300/300 [29:47<00:00,  5.96s/it]textattack: Saving checkpoint under "checkpoints\1722704427506.ta.chkpt" at 2024-08-04 01:00:27 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 138 / 102 / 60 / 300: 100%|███████████████████| 300/300 [29:47<00:00,  5.96s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 138    |
| Number of failed attacks:     | 102    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 34.0%  |
| Attack success rate:          | 57.5%  |
| Average perturbed word %:     | 10.08% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.6   |
+-------------------------------+--------+


#### CLIP

In [51]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:57<19:49,  4.76s/it]textattack: Saving checkpoint under "checkpoints\1722704665621.ta.chkpt" at 2024-08-04 01:04:25 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 77 / 9 / 100:  33%|███████▎              | 100/300 [07:29<14:58,  4.49s/it]textattack: Saving checkpoint under "checkpoints\1722704876976.ta.chkpt" at 2024-08-04 01:07:56 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 112 / 14 / 150:  50%|██████████          | 150/300 [11:07<11:07,  4.45s/it]textattack: Saving checkpoint under "checkpoints\1722705095387.ta.chkpt" at 2024-08-04 01:11:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 143 / 20 / 200:  67%|█████████████▎      | 200/300 [14:28<07:14,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1722705296148.ta.chkpt" at 2024-08-04 01:14:56 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 176 / 24 / 250:  83%|████████████████▋   | 250/300 [18:13<03:38,  4.37s/it]textattack: Saving checkpoint under "checkpoints\1722705520906.ta.chkpt" at 2024-08-04 01:18:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 59 / 212 / 29 / 300: 100%|████████████████████| 300/300 [21:43<00:00,  4.35s/it]textattack: Saving checkpoint under "checkpoints\1722705731221.ta.chkpt" at 2024-08-04 01:22:11 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 59 / 212 / 29 / 300: 100%|████████████████████| 300/300 [21:43<00:00,  4.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 59     |
| Number of failed attacks:     | 212    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 70.67% |
| Attack success rate:          | 21.77% |
| Average perturbed word %:     | 14.93% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 69.98  |
+-------------------------------+--------+


In [52]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:21<16:47,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1722705932899.ta.chkpt" at 2024-08-04 01:25:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 78 / 9 / 100:  33%|███████▎              | 100/300 [06:44<13:28,  4.04s/it]textattack: Saving checkpoint under "checkpoints\1722706135562.ta.chkpt" at 2024-08-04 01:28:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 113 / 14 / 150:  50%|██████████          | 150/300 [10:13<10:13,  4.09s/it]textattack: Saving checkpoint under "checkpoints\1722706344536.ta.chkpt" at 2024-08-04 01:32:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 144 / 20 / 200:  67%|█████████████▎      | 200/300 [13:29<06:44,  4.05s/it]textattack: Saving checkpoint under "checkpoints\1722706540560.ta.chkpt" at 2024-08-04 01:35:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 49 / 177 / 24 / 250:  83%|████████████████▋   | 250/300 [17:09<03:25,  4.12s/it]textattack: Saving checkpoint under "checkpoints\1722706761202.ta.chkpt" at 2024-08-04 01:39:21 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 58 / 213 / 29 / 300: 100%|████████████████████| 300/300 [20:40<00:00,  4.13s/it]textattack: Saving checkpoint under "checkpoints\1722706971854.ta.chkpt" at 2024-08-04 01:42:51 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 58 / 213 / 29 / 300: 100%|████████████████████| 300/300 [20:40<00:00,  4.14s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 58     |
| Number of failed attacks:     | 213    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 71.0%  |
| Attack success rate:          | 21.4%  |
| Average perturbed word %:     | 14.75% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 70.01  |
+-------------------------------+--------+


In [53]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:21<16:46,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1722707173439.ta.chkpt" at 2024-08-04 01:46:13 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 77 / 9 / 100:  33%|███████▎              | 100/300 [06:44<13:28,  4.04s/it]textattack: Saving checkpoint under "checkpoints\1722707376446.ta.chkpt" at 2024-08-04 01:49:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 112 / 14 / 150:  50%|██████████          | 150/300 [10:14<10:14,  4.09s/it]textattack: Saving checkpoint under "checkpoints\1722707586157.ta.chkpt" at 2024-08-04 01:53:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 143 / 20 / 200:  67%|█████████████▎      | 200/300 [13:29<06:44,  4.05s/it]textattack: Saving checkpoint under "checkpoints\1722707781854.ta.chkpt" at 2024-08-04 01:56:21 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 176 / 24 / 250:  83%|████████████████▋   | 250/300 [17:10<03:26,  4.12s/it]textattack: Saving checkpoint under "checkpoints\1722708002437.ta.chkpt" at 2024-08-04 02:00:02 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 59 / 212 / 29 / 300: 100%|████████████████████| 300/300 [20:35<00:00,  4.12s/it]textattack: Saving checkpoint under "checkpoints\1722708207183.ta.chkpt" at 2024-08-04 02:03:27 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 59 / 212 / 29 / 300: 100%|████████████████████| 300/300 [20:35<00:00,  4.12s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 59     |
| Number of failed attacks:     | 212    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 70.67% |
| Attack success rate:          | 21.77% |
| Average perturbed word %:     | 14.89% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 69.96  |
+-------------------------------+--------+


In [54]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 37 / 5 / 50:  17%|████▏                    | 50/300 [03:10<15:53,  3.81s/it]textattack: Saving checkpoint under "checkpoints\1722708398079.ta.chkpt" at 2024-08-04 02:06:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 68 / 13 / 100:  33%|███████              | 100/300 [06:22<12:44,  3.82s/it]textattack: Saving checkpoint under "checkpoints\1722708589496.ta.chkpt" at 2024-08-04 02:09:49 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 103 / 19 / 150:  50%|██████████          | 150/300 [09:45<09:45,  3.90s/it]textattack: Saving checkpoint under "checkpoints\1722708792761.ta.chkpt" at 2024-08-04 02:13:12 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 126 / 35 / 200:  67%|█████████████▎      | 200/300 [12:05<06:02,  3.63s/it]textattack: Saving checkpoint under "checkpoints\1722708933023.ta.chkpt" at 2024-08-04 02:15:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 47 / 159 / 44 / 250:  83%|████████████████▋   | 250/300 [15:19<03:03,  3.68s/it]textattack: Saving checkpoint under "checkpoints\1722709126882.ta.chkpt" at 2024-08-04 02:18:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 200 / 49 / 300: 100%|████████████████████| 300/300 [19:01<00:00,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1722709348780.ta.chkpt" at 2024-08-04 02:22:28 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 51 / 200 / 49 / 300: 100%|████████████████████| 300/300 [19:01<00:00,  3.80s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 51     |
| Number of failed attacks:     | 200    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 66.67% |
| Attack success rate:          | 20.32% |
| Average perturbed word %:     | 11.27% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 68.9   |
+-------------------------------+--------+


In [55]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 42 / 2 / 50:  17%|████▏                    | 50/300 [03:39<18:16,  4.39s/it]textattack: Saving checkpoint under "checkpoints\1722709568297.ta.chkpt" at 2024-08-04 02:26:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 78 / 7 / 100:  33%|███████▎              | 100/300 [07:10<14:21,  4.31s/it]textattack: Saving checkpoint under "checkpoints\1722709779419.ta.chkpt" at 2024-08-04 02:29:39 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 116 / 13 / 150:  50%|██████████          | 150/300 [10:40<10:40,  4.27s/it]textattack: Saving checkpoint under "checkpoints\1722709989554.ta.chkpt" at 2024-08-04 02:33:09 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 144 / 23 / 200:  67%|█████████████▎      | 200/300 [13:36<06:48,  4.08s/it]textattack: Saving checkpoint under "checkpoints\1722710165740.ta.chkpt" at 2024-08-04 02:36:05 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 42 / 179 / 29 / 250:  83%|████████████████▋   | 250/300 [17:16<03:27,  4.15s/it]textattack: Saving checkpoint under "checkpoints\1722710385597.ta.chkpt" at 2024-08-04 02:39:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 220 / 35 / 300: 100%|████████████████████| 300/300 [21:00<00:00,  4.20s/it]textattack: Saving checkpoint under "checkpoints\1722710609648.ta.chkpt" at 2024-08-04 02:43:29 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 45 / 220 / 35 / 300: 100%|████████████████████| 300/300 [21:00<00:00,  4.20s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 45     |
| Number of failed attacks:     | 220    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 73.33% |
| Attack success rate:          | 16.98% |
| Average perturbed word %:     | 11.84% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 70.98  |
+-------------------------------+--------+


In [56]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 42 / 2 / 50:  17%|████▏                    | 50/300 [03:46<18:54,  4.54s/it]textattack: Saving checkpoint under "checkpoints\1722710836714.ta.chkpt" at 2024-08-04 02:47:16 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 77 / 7 / 100:  33%|███████▎              | 100/300 [07:20<14:40,  4.40s/it]textattack: Saving checkpoint under "checkpoints\1722711050174.ta.chkpt" at 2024-08-04 02:50:50 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 116 / 10 / 150:  50%|██████████          | 150/300 [10:55<10:55,  4.37s/it]textattack: Saving checkpoint under "checkpoints\1722711265653.ta.chkpt" at 2024-08-04 02:54:25 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 146 / 20 / 200:  67%|█████████████▎      | 200/300 [13:55<06:57,  4.18s/it]textattack: Saving checkpoint under "checkpoints\1722711445765.ta.chkpt" at 2024-08-04 02:57:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 42 / 183 / 25 / 250:  83%|████████████████▋   | 250/300 [17:42<03:32,  4.25s/it]textattack: Saving checkpoint under "checkpoints\1722711672770.ta.chkpt" at 2024-08-04 03:01:12 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 224 / 31 / 300: 100%|████████████████████| 300/300 [21:18<00:00,  4.26s/it]textattack: Saving checkpoint under "checkpoints\1722711888820.ta.chkpt" at 2024-08-04 03:04:48 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 45 / 224 / 31 / 300: 100%|████████████████████| 300/300 [21:19<00:00,  4.26s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 45     |
| Number of failed attacks:     | 224    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 74.67% |
| Attack success rate:          | 16.73% |
| Average perturbed word %:     | 8.43%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 70.84  |
+-------------------------------+--------+


In [57]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [42:48<3:34:01, 51.37s/it]textattack: Saving checkpoint under "checkpoints\1722714457325.ta.chkpt" at 2024-08-04 03:47:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 91 / 4 / 100:  33%|███████              | 100/300 [48:20<1:36:40, 29.00s/it]textattack: Saving checkpoint under "checkpoints\1722714788994.ta.chkpt" at 2024-08-04 03:53:08 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 136 / 7 / 150:  50%|███████████           | 150/300 [53:48<53:48, 21.53s/it]textattack: Saving checkpoint under "checkpoints\1722715117809.ta.chkpt" at 2024-08-04 03:58:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 177 / 14 / 200:  67%|██████████████       | 200/300 [58:56<29:28, 17.68s/it]textattack: Saving checkpoint under "checkpoints\1722715425279.ta.chkpt" at 2024-08-04 04:03:45 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 221 / 17 / 250:  83%|███████████████   | 250/300 [1:04:53<12:58, 15.58s/it]textattack: Saving checkpoint under "checkpoints\1722715782941.ta.chkpt" at 2024-08-04 04:09:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 265 / 22 / 300: 100%|██████████████████| 300/300 [1:10:10<00:00, 14.04s/it]textattack: Saving checkpoint under "checkpoints\1722716099678.ta.chkpt" at 2024-08-04 04:14:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 265 / 22 / 300: 100%|██████████████████| 300/300 [1:10:10<00:00, 14.04s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 265    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 88.33% |
| Attack success rate:          | 4.68%  |
| Average perturbed word %:     | 3.82%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 74.88  |
+-------------------------------+--------+


In [58]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 46 / 0 / 50:  17%|████▏                    | 50/300 [05:22<26:52,  6.45s/it]textattack: Saving checkpoint under "checkpoints\1722716422333.ta.chkpt" at 2024-08-04 04:20:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 87 / 3 / 100:  33%|███████▎              | 100/300 [10:48<21:36,  6.48s/it]textattack: Saving checkpoint under "checkpoints\1722716748356.ta.chkpt" at 2024-08-04 04:25:48 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 129 / 6 / 150:  50%|██████████▌          | 150/300 [16:12<16:12,  6.48s/it]textattack: Saving checkpoint under "checkpoints\1722717071938.ta.chkpt" at 2024-08-04 04:31:11 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 169 / 13 / 200:  67%|█████████████▎      | 200/300 [21:05<10:32,  6.33s/it]textattack: Saving checkpoint under "checkpoints\1722717365719.ta.chkpt" at 2024-08-04 04:36:05 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 211 / 16 / 250:  83%|████████████████▋   | 250/300 [26:48<05:21,  6.43s/it]textattack: Saving checkpoint under "checkpoints\1722717708397.ta.chkpt" at 2024-08-04 04:41:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 254 / 22 / 300: 100%|████████████████████| 300/300 [31:59<00:00,  6.40s/it]textattack: Saving checkpoint under "checkpoints\1722718019006.ta.chkpt" at 2024-08-04 04:46:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 24 / 254 / 22 / 300: 100%|████████████████████| 300/300 [31:59<00:00,  6.40s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 254    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 84.67% |
| Attack success rate:          | 8.63%  |
| Average perturbed word %:     | 5.96%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 73.59  |
+-------------------------------+--------+


In [59]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175ep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 43 / 1 / 50:  17%|████▏                    | 50/300 [05:22<26:53,  6.45s/it]textattack: Saving checkpoint under "checkpoints\1722718342084.ta.chkpt" at 2024-08-04 04:52:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 83 / 5 / 100:  33%|███████▎              | 100/300 [10:41<21:22,  6.41s/it]textattack: Saving checkpoint under "checkpoints\1722718660822.ta.chkpt" at 2024-08-04 04:57:40 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 124 / 9 / 150:  50%|██████████▌          | 150/300 [16:07<16:07,  6.45s/it]textattack: Saving checkpoint under "checkpoints\1722718986854.ta.chkpt" at 2024-08-04 05:03:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 159 / 21 / 200:  67%|█████████████▎      | 200/300 [20:35<10:17,  6.18s/it]textattack: Saving checkpoint under "checkpoints\1722719255060.ta.chkpt" at 2024-08-04 05:07:35 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 199 / 28 / 250:  83%|████████████████▋   | 250/300 [25:55<05:11,  6.22s/it]textattack: Saving checkpoint under "checkpoints\1722719574920.ta.chkpt" at 2024-08-04 05:12:54 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 242 / 34 / 300: 100%|████████████████████| 300/300 [31:17<00:00,  6.26s/it]textattack: Saving checkpoint under "checkpoints\1722719897258.ta.chkpt" at 2024-08-04 05:18:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 24 / 242 / 34 / 300: 100%|████████████████████| 300/300 [31:17<00:00,  6.26s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 242    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 80.67% |
| Attack success rate:          | 9.02%  |
| Average perturbed word %:     | 11.53% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 73.31  |
+-------------------------------+--------+


In [60]:
et2 = datetime.datetime.now()
et2 - et

datetime.timedelta(seconds=31067, microseconds=737395)

### Threshold = 0.9

In [61]:
suffix = 'ep09.csv'
thre = 0.9

#### BERT

In [62]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 11 / 34 / 5 / 50:  17%|████                    | 50/300 [04:11<20:59,  5.04s/it]textattack: Saving checkpoint under "checkpoints\1722720151860.ta.chkpt" at 2024-08-04 05:22:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 62 / 10 / 100:  33%|███████              | 100/300 [08:07<16:14,  4.87s/it]textattack: Saving checkpoint under "checkpoints\1722720387410.ta.chkpt" at 2024-08-04 05:26:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 91 / 15 / 150:  50%|██████████▌          | 150/300 [12:07<12:07,  4.85s/it]textattack: Saving checkpoint under "checkpoints\1722720627567.ta.chkpt" at 2024-08-04 05:30:27 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 59 / 120 / 21 / 200:  67%|█████████████▎      | 200/300 [16:28<08:14,  4.94s/it]textattack: Saving checkpoint under "checkpoints\1722720888825.ta.chkpt" at 2024-08-04 05:34:48 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 72 / 153 / 25 / 250:  83%|████████████████▋   | 250/300 [21:14<04:14,  5.10s/it]textattack: Saving checkpoint under "checkpoints\1722721174188.ta.chkpt" at 2024-08-04 05:39:34 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 78 / 192 / 30 / 300: 100%|████████████████████| 300/300 [25:59<00:00,  5.20s/it]textattack: Saving checkpoint under "checkpoints\1722721458998.ta.chkpt" at 2024-08-04 05:44:18 after 300 attacks.


[Succeeded / Failed / Skipped / Total] 78 / 192 / 30 / 300: 100%|████████████████████| 300/300 [25:59<00:00,  5.20s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 78     |
| Number of failed attacks:     | 192    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 64.0%  |
| Attack success rate:          | 28.89% |
| Average perturbed word %:     | 12.53% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 59.23  |
+-------------------------------+--------+


In [63]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 11 / 34 / 5 / 50:  17%|████                    | 50/300 [03:50<19:11,  4.61s/it]textattack: Saving checkpoint under "checkpoints\1722721689770.ta.chkpt" at 2024-08-04 05:48:09 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 66 / 11 / 100:  33%|███████              | 100/300 [07:48<15:36,  4.68s/it]textattack: Saving checkpoint under "checkpoints\1722721927840.ta.chkpt" at 2024-08-04 05:52:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 95 / 16 / 150:  50%|██████████▌          | 150/300 [11:45<11:45,  4.70s/it]textattack: Saving checkpoint under "checkpoints\1722722164766.ta.chkpt" at 2024-08-04 05:56:04 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 122 / 23 / 200:  67%|█████████████▎      | 200/300 [15:55<07:57,  4.78s/it]textattack: Saving checkpoint under "checkpoints\1722722414470.ta.chkpt" at 2024-08-04 06:00:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 66 / 157 / 27 / 250:  83%|████████████████▋   | 250/300 [20:33<04:06,  4.93s/it]textattack: Saving checkpoint under "checkpoints\1722722692694.ta.chkpt" at 2024-08-04 06:04:52 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 195 / 32 / 300: 100%|████████████████████| 300/300 [25:07<00:00,  5.03s/it]textattack: Saving checkpoint under "checkpoints\1722722967262.ta.chkpt" at 2024-08-04 06:09:27 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 195 / 32 / 300: 100%|████████████████████| 300/300 [25:07<00:00,  5.03s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 195    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 65.0%  |
| Attack success rate:          | 27.24% |
| Average perturbed word %:     | 13.33% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 59.3   |
+-------------------------------+--------+


In [64]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 11 / 34 / 5 / 50:  17%|████                    | 50/300 [03:42<18:31,  4.45s/it]textattack: Saving checkpoint under "checkpoints\1722723189812.ta.chkpt" at 2024-08-04 06:13:09 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 62 / 11 / 100:  33%|███████              | 100/300 [07:36<15:13,  4.57s/it]textattack: Saving checkpoint under "checkpoints\1722723424480.ta.chkpt" at 2024-08-04 06:17:04 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 91 / 16 / 150:  50%|██████████▌          | 150/300 [11:30<11:30,  4.61s/it]textattack: Saving checkpoint under "checkpoints\1722723658261.ta.chkpt" at 2024-08-04 06:20:58 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 119 / 21 / 200:  67%|█████████████▎      | 200/300 [15:48<07:54,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1722723916278.ta.chkpt" at 2024-08-04 06:25:16 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 152 / 25 / 250:  83%|████████████████▋   | 250/300 [20:22<04:04,  4.89s/it]textattack: Saving checkpoint under "checkpoints\1722724190357.ta.chkpt" at 2024-08-04 06:29:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 79 / 191 / 30 / 300: 100%|████████████████████| 300/300 [24:57<00:00,  4.99s/it]textattack: Saving checkpoint under "checkpoints\1722724464793.ta.chkpt" at 2024-08-04 06:34:24 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 79 / 191 / 30 / 300: 100%|████████████████████| 300/300 [24:57<00:00,  4.99s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 79     |
| Number of failed attacks:     | 191    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 63.67% |
| Attack success rate:          | 29.26% |
| Average perturbed word %:     | 12.13% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 58.98  |
+-------------------------------+--------+


In [65]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 17 / 17 / 16 / 50:  17%|███▊                   | 50/300 [02:20<11:42,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1722724605455.ta.chkpt" at 2024-08-04 06:36:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 34 / 31 / 100:  33%|███████              | 100/300 [05:01<10:03,  3.02s/it]textattack: Saving checkpoint under "checkpoints\1722724766651.ta.chkpt" at 2024-08-04 06:39:26 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 35 / 34 / 32 / 101:  34%|███████              | 101/300 [05:01<09:54,  2.99s/it]

[Succeeded / Failed / Skipped / Total] 62 / 48 / 40 / 150:  50%|██████████▌          | 150/300 [08:04<08:04,  3.23s/it]textattack: Saving checkpoint under "checkpoints\1722724949314.ta.chkpt" at 2024-08-04 06:42:29 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 80 / 65 / 55 / 200:  67%|██████████████       | 200/300 [11:21<05:40,  3.41s/it]textattack: Saving checkpoint under "checkpoints\1722725146247.ta.chkpt" at 2024-08-04 06:45:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 104 / 80 / 66 / 250:  83%|████████████████▋   | 250/300 [14:12<02:50,  3.41s/it]textattack: Saving checkpoint under "checkpoints\1722725317140.ta.chkpt" at 2024-08-04 06:48:37 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 125 / 97 / 78 / 300: 100%|████████████████████| 300/300 [17:32<00:00,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1722725517351.ta.chkpt" at 2024-08-04 06:51:57 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 125 / 97 / 78 / 300: 100%|████████████████████| 300/300 [17:32<00:00,  3.51s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 125    |
| Number of failed attacks:     | 97     |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 32.33% |
| Attack success rate:          | 56.31% |
| Average perturbed word %:     | 10.3%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 51.06  |
+-------------------------------+--------+


In [66]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 20 / 21 / 9 / 50:  17%|████                    | 50/300 [03:22<16:53,  4.06s/it]textattack: Saving checkpoint under "checkpoints\1722725720355.ta.chkpt" at 2024-08-04 06:55:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 42 / 20 / 100:  33%|███████              | 100/300 [06:31<13:02,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1722725908734.ta.chkpt" at 2024-08-04 06:58:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 66 / 29 / 150:  50%|██████████▌          | 150/300 [09:54<09:54,  3.96s/it]textattack: Saving checkpoint under "checkpoints\1722726112169.ta.chkpt" at 2024-08-04 07:01:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 66 / 89 / 45 / 200:  67%|██████████████       | 200/300 [13:20<06:40,  4.00s/it]textattack: Saving checkpoint under "checkpoints\1722726318020.ta.chkpt" at 2024-08-04 07:05:18 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 79 / 120 / 51 / 250:  83%|████████████████▋   | 250/300 [17:40<03:32,  4.24s/it]textattack: Saving checkpoint under "checkpoints\1722726577606.ta.chkpt" at 2024-08-04 07:09:37 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 149 / 58 / 300: 100%|████████████████████| 300/300 [21:39<00:00,  4.33s/it]textattack: Saving checkpoint under "checkpoints\1722726817393.ta.chkpt" at 2024-08-04 07:13:37 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 93 / 149 / 58 / 300: 100%|████████████████████| 300/300 [21:39<00:00,  4.33s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 93     |
| Number of failed attacks:     | 149    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 49.67% |
| Attack success rate:          | 38.43% |
| Average perturbed word %:     | 10.21% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 55.58  |
+-------------------------------+--------+


In [67]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 15 / 26 / 9 / 50:  17%|████                    | 50/300 [03:30<17:34,  4.22s/it]textattack: Saving checkpoint under "checkpoints\1722727028457.ta.chkpt" at 2024-08-04 07:17:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 46 / 18 / 100:  33%|███████              | 100/300 [06:41<13:23,  4.02s/it]textattack: Saving checkpoint under "checkpoints\1722727219285.ta.chkpt" at 2024-08-04 07:20:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 53 / 70 / 27 / 150:  50%|██████████▌          | 150/300 [10:04<10:04,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1722727422216.ta.chkpt" at 2024-08-04 07:23:42 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 94 / 42 / 200:  67%|██████████████       | 200/300 [13:39<06:49,  4.10s/it]textattack: Saving checkpoint under "checkpoints\1722727637246.ta.chkpt" at 2024-08-04 07:27:17 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 80 / 122 / 48 / 250:  83%|████████████████▋   | 250/300 [17:51<03:34,  4.29s/it]textattack: Saving checkpoint under "checkpoints\1722727888887.ta.chkpt" at 2024-08-04 07:31:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 152 / 55 / 300: 100%|████████████████████| 300/300 [21:57<00:00,  4.39s/it]textattack: Saving checkpoint under "checkpoints\1722728134625.ta.chkpt" at 2024-08-04 07:35:34 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 93 / 152 / 55 / 300: 100%|████████████████████| 300/300 [21:57<00:00,  4.39s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 93     |
| Number of failed attacks:     | 152    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 50.67% |
| Attack success rate:          | 37.96% |
| Average perturbed word %:     | 8.43%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 55.48  |
+-------------------------------+--------+


In [68]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 19 / 20 / 11 / 50:  17%|███▌                 | 50/300 [25:31<2:07:38, 30.64s/it]textattack: Saving checkpoint under "checkpoints\1722729666509.ta.chkpt" at 2024-08-04 08:01:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 40 / 20 / 100:  33%|███████              | 100/300 [29:48<59:36, 17.88s/it]textattack: Saving checkpoint under "checkpoints\1722729923200.ta.chkpt" at 2024-08-04 08:05:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 63 / 32 / 150:  50%|██████████▌          | 150/300 [34:21<34:21, 13.74s/it]textattack: Saving checkpoint under "checkpoints\1722730196307.ta.chkpt" at 2024-08-04 08:09:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 86 / 45 / 200:  67%|██████████████       | 200/300 [39:19<19:39, 11.80s/it]textattack: Saving checkpoint under "checkpoints\1722730494375.ta.chkpt" at 2024-08-04 08:14:54 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 110 / 52 / 250:  83%|████████████████▋   | 250/300 [44:40<08:56, 10.72s/it]textattack: Saving checkpoint under "checkpoints\1722730815567.ta.chkpt" at 2024-08-04 08:20:15 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 134 / 58 / 300: 100%|███████████████████| 300/300 [50:05<00:00, 10.02s/it]textattack: Saving checkpoint under "checkpoints\1722731140142.ta.chkpt" at 2024-08-04 08:25:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 108 / 134 / 58 / 300: 100%|███████████████████| 300/300 [50:05<00:00, 10.02s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 108    |
| Number of failed attacks:     | 134    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 44.67% |
| Attack success rate:          | 44.63% |
| Average perturbed word %:     | 4.85%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 53.58  |
+-------------------------------+--------+


In [69]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 17 / 24 / 9 / 50:  17%|████                    | 50/300 [04:37<23:08,  5.55s/it]textattack: Saving checkpoint under "checkpoints\1722731417986.ta.chkpt" at 2024-08-04 08:30:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 45 / 19 / 100:  33%|███████              | 100/300 [09:15<18:30,  5.55s/it]textattack: Saving checkpoint under "checkpoints\1722731695386.ta.chkpt" at 2024-08-04 08:34:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 56 / 66 / 28 / 150:  50%|██████████▌          | 150/300 [13:49<13:49,  5.53s/it]textattack: Saving checkpoint under "checkpoints\1722731970018.ta.chkpt" at 2024-08-04 08:39:30 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 68 / 89 / 43 / 200:  67%|██████████████       | 200/300 [18:59<09:29,  5.70s/it]textattack: Saving checkpoint under "checkpoints\1722732279446.ta.chkpt" at 2024-08-04 08:44:39 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 82 / 117 / 51 / 250:  83%|████████████████▋   | 250/300 [24:48<04:57,  5.95s/it]textattack: Saving checkpoint under "checkpoints\1722732628700.ta.chkpt" at 2024-08-04 08:50:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 97 / 147 / 56 / 300: 100%|████████████████████| 300/300 [30:28<00:00,  6.09s/it]textattack: Saving checkpoint under "checkpoints\1722732968761.ta.chkpt" at 2024-08-04 08:56:08 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 97 / 147 / 56 / 300: 100%|████████████████████| 300/300 [30:28<00:00,  6.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 97     |
| Number of failed attacks:     | 147    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 49.0%  |
| Attack success rate:          | 39.75% |
| Average perturbed word %:     | 8.07%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 54.95  |
+-------------------------------+--------+


In [70]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 25 / 16 / 9 / 50:  17%|████                    | 50/300 [04:12<21:01,  5.05s/it]textattack: Saving checkpoint under "checkpoints\1722733221335.ta.chkpt" at 2024-08-04 09:00:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 36 / 20 / 100:  33%|███████              | 100/300 [08:24<16:48,  5.04s/it]textattack: Saving checkpoint under "checkpoints\1722733473224.ta.chkpt" at 2024-08-04 09:04:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 53 / 33 / 150:  50%|██████████▌          | 150/300 [12:32<12:32,  5.01s/it]textattack: Saving checkpoint under "checkpoints\1722733721204.ta.chkpt" at 2024-08-04 09:08:41 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 85 / 68 / 47 / 200:  67%|██████████████       | 200/300 [17:11<08:35,  5.16s/it]textattack: Saving checkpoint under "checkpoints\1722734000098.ta.chkpt" at 2024-08-04 09:13:20 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 106 / 90 / 54 / 250:  83%|████████████████▋   | 250/300 [22:40<04:32,  5.44s/it]textattack: Saving checkpoint under "checkpoints\1722734329606.ta.chkpt" at 2024-08-04 09:18:49 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 129 / 111 / 60 / 300: 100%|███████████████████| 300/300 [27:35<00:00,  5.52s/it]textattack: Saving checkpoint under "checkpoints\1722734624152.ta.chkpt" at 2024-08-04 09:23:44 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 129 / 111 / 60 / 300: 100%|███████████████████| 300/300 [27:35<00:00,  5.52s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 129    |
| Number of failed attacks:     | 111    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 37.0%  |
| Attack success rate:          | 53.75% |
| Average perturbed word %:     | 10.56% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 52.12  |
+-------------------------------+--------+


#### CLIP

In [71]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:35<17:56,  4.31s/it]textattack: Saving checkpoint under "checkpoints\1722734839664.ta.chkpt" at 2024-08-04 09:27:19 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 82 / 9 / 100:  33%|███████▋               | 100/300 [06:52<13:45,  4.13s/it]textattack: Saving checkpoint under "checkpoints\1722735037273.ta.chkpt" at 2024-08-04 09:30:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 124 / 14 / 150:  50%|██████████          | 150/300 [10:21<10:21,  4.15s/it]textattack: Saving checkpoint under "checkpoints\1722735246169.ta.chkpt" at 2024-08-04 09:34:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 161 / 20 / 200:  67%|█████████████▎      | 200/300 [13:37<06:48,  4.09s/it]textattack: Saving checkpoint under "checkpoints\1722735441676.ta.chkpt" at 2024-08-04 09:37:21 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 198 / 24 / 250:  83%|████████████████▋   | 250/300 [17:17<03:27,  4.15s/it]textattack: Saving checkpoint under "checkpoints\1722735661767.ta.chkpt" at 2024-08-04 09:41:01 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 236 / 29 / 300: 100%|████████████████████| 300/300 [20:39<00:00,  4.13s/it]textattack: Saving checkpoint under "checkpoints\1722735863431.ta.chkpt" at 2024-08-04 09:44:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 35 / 236 / 29 / 300: 100%|████████████████████| 300/300 [20:39<00:00,  4.13s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 35     |
| Number of failed attacks:     | 236    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 78.67% |
| Attack success rate:          | 12.92% |
| Average perturbed word %:     | 12.99% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 61.77  |
+-------------------------------+--------+


In [72]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:04<15:24,  3.70s/it]textattack: Saving checkpoint under "checkpoints\1722736048540.ta.chkpt" at 2024-08-04 09:47:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 82 / 9 / 100:  33%|███████▋               | 100/300 [06:12<12:24,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1722736235654.ta.chkpt" at 2024-08-04 09:50:35 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 124 / 14 / 150:  50%|██████████          | 150/300 [09:30<09:30,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1722736434082.ta.chkpt" at 2024-08-04 09:53:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 161 / 20 / 200:  67%|█████████████▎      | 200/300 [12:39<06:19,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1722736622736.ta.chkpt" at 2024-08-04 09:57:02 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 198 / 24 / 250:  83%|████████████████▋   | 250/300 [16:13<03:14,  3.89s/it]textattack: Saving checkpoint under "checkpoints\1722736836687.ta.chkpt" at 2024-08-04 10:00:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 236 / 29 / 300: 100%|████████████████████| 300/300 [19:26<00:00,  3.89s/it]textattack: Saving checkpoint under "checkpoints\1722737029648.ta.chkpt" at 2024-08-04 10:03:49 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 35 / 236 / 29 / 300: 100%|████████████████████| 300/300 [19:26<00:00,  3.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 35     |
| Number of failed attacks:     | 236    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 78.67% |
| Attack success rate:          | 12.92% |
| Average perturbed word %:     | 12.64% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 61.72  |
+-------------------------------+--------+


In [73]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:03<15:18,  3.67s/it]textattack: Saving checkpoint under "checkpoints\1722737213589.ta.chkpt" at 2024-08-04 10:06:53 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 82 / 9 / 100:  33%|███████▋               | 100/300 [06:12<12:24,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1722737401876.ta.chkpt" at 2024-08-04 10:10:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 124 / 14 / 150:  50%|██████████          | 150/300 [09:27<09:27,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1722737597800.ta.chkpt" at 2024-08-04 10:13:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 161 / 20 / 200:  67%|█████████████▎      | 200/300 [12:37<06:18,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1722737787354.ta.chkpt" at 2024-08-04 10:16:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 198 / 24 / 250:  83%|████████████████▋   | 250/300 [16:14<03:14,  3.90s/it]textattack: Saving checkpoint under "checkpoints\1722738004689.ta.chkpt" at 2024-08-04 10:20:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 236 / 29 / 300: 100%|████████████████████| 300/300 [19:27<00:00,  3.89s/it]textattack: Saving checkpoint under "checkpoints\1722738197361.ta.chkpt" at 2024-08-04 10:23:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 35 / 236 / 29 / 300: 100%|████████████████████| 300/300 [19:27<00:00,  3.89s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 35     |
| Number of failed attacks:     | 236    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 78.67% |
| Attack success rate:          | 12.92% |
| Average perturbed word %:     | 12.99% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 61.78  |
+-------------------------------+--------+


In [74]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 5 / 40 / 5 / 50:  17%|████▏                    | 50/300 [02:57<14:48,  3.55s/it]textattack: Saving checkpoint under "checkpoints\1722738375253.ta.chkpt" at 2024-08-04 10:26:15 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 72 / 13 / 100:  33%|███████              | 100/300 [05:52<11:44,  3.52s/it]textattack: Saving checkpoint under "checkpoints\1722738550001.ta.chkpt" at 2024-08-04 10:29:10 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 110 / 19 / 150:  50%|██████████          | 150/300 [09:00<09:00,  3.60s/it]textattack: Saving checkpoint under "checkpoints\1722738738045.ta.chkpt" at 2024-08-04 10:32:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 135 / 35 / 200:  67%|█████████████▎      | 200/300 [11:10<05:35,  3.35s/it]textattack: Saving checkpoint under "checkpoints\1722738868526.ta.chkpt" at 2024-08-04 10:34:28 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 171 / 44 / 250:  83%|████████████████▋   | 250/300 [14:14<02:50,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1722739052201.ta.chkpt" at 2024-08-04 10:37:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 214 / 49 / 300: 100%|████████████████████| 300/300 [17:36<00:00,  3.52s/it]textattack: Saving checkpoint under "checkpoints\1722739253893.ta.chkpt" at 2024-08-04 10:40:53 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 37 / 214 / 49 / 300: 100%|████████████████████| 300/300 [17:36<00:00,  3.52s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 37     |
| Number of failed attacks:     | 214    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 71.33% |
| Attack success rate:          | 14.74% |
| Average perturbed word %:     | 9.24%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 59.57  |
+-------------------------------+--------+


In [75]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 5 / 43 / 2 / 50:  17%|████▏                    | 50/300 [03:25<17:09,  4.12s/it]textattack: Saving checkpoint under "checkpoints\1722739459905.ta.chkpt" at 2024-08-04 10:44:19 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 83 / 7 / 100:  33%|███████▎              | 100/300 [06:43<13:27,  4.04s/it]textattack: Saving checkpoint under "checkpoints\1722739657732.ta.chkpt" at 2024-08-04 10:47:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 123 / 13 / 150:  50%|██████████          | 150/300 [09:58<09:58,  3.99s/it]textattack: Saving checkpoint under "checkpoints\1722739852017.ta.chkpt" at 2024-08-04 10:50:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 154 / 23 / 200:  67%|█████████████▎      | 200/300 [12:40<06:20,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1722740014546.ta.chkpt" at 2024-08-04 10:53:34 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 193 / 29 / 250:  83%|████████████████▋   | 250/300 [16:15<03:15,  3.90s/it]textattack: Saving checkpoint under "checkpoints\1722740229105.ta.chkpt" at 2024-08-04 10:57:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 29 / 236 / 35 / 300: 100%|████████████████████| 300/300 [19:34<00:00,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1722740428267.ta.chkpt" at 2024-08-04 11:00:28 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 29 / 236 / 35 / 300: 100%|████████████████████| 300/300 [19:34<00:00,  3.91s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 29     |
| Number of failed attacks:     | 236    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 78.67% |
| Attack success rate:          | 10.94% |
| Average perturbed word %:     | 10.21% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 61.77  |
+-------------------------------+--------+


In [76]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 45 / 2 / 50:  17%|████▏                    | 50/300 [03:37<18:05,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1722740645432.ta.chkpt" at 2024-08-04 11:04:05 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 84 / 7 / 100:  33%|███████▋               | 100/300 [06:56<13:52,  4.16s/it]textattack: Saving checkpoint under "checkpoints\1722740844630.ta.chkpt" at 2024-08-04 11:07:24 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 124 / 10 / 150:  50%|██████████          | 150/300 [10:13<10:13,  4.09s/it]textattack: Saving checkpoint under "checkpoints\1722741041439.ta.chkpt" at 2024-08-04 11:10:41 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 155 / 20 / 200:  67%|█████████████▎      | 200/300 [12:58<06:29,  3.89s/it]textattack: Saving checkpoint under "checkpoints\1722741206749.ta.chkpt" at 2024-08-04 11:13:26 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 191 / 25 / 250:  83%|████████████████▋   | 250/300 [16:21<03:16,  3.93s/it]textattack: Saving checkpoint under "checkpoints\1722741409831.ta.chkpt" at 2024-08-04 11:16:49 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 233 / 31 / 300: 100%|████████████████████| 300/300 [19:36<00:00,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1722741604829.ta.chkpt" at 2024-08-04 11:20:04 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 36 / 233 / 31 / 300: 100%|████████████████████| 300/300 [19:36<00:00,  3.92s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 36     |
| Number of failed attacks:     | 233    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 77.67% |
| Attack success rate:          | 13.38% |
| Average perturbed word %:     | 8.74%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 60.66  |
+-------------------------------+--------+


In [77]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [39:52<3:19:23, 47.86s/it]textattack: Saving checkpoint under "checkpoints\1722743997761.ta.chkpt" at 2024-08-04 11:59:57 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 93 / 4 / 100:  33%|███████              | 100/300 [44:48<1:29:37, 26.89s/it]textattack: Saving checkpoint under "checkpoints\1722744293561.ta.chkpt" at 2024-08-04 12:04:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 138 / 7 / 150:  50%|███████████           | 150/300 [49:43<49:43, 19.89s/it]textattack: Saving checkpoint under "checkpoints\1722744588192.ta.chkpt" at 2024-08-04 12:09:48 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 179 / 14 / 200:  67%|██████████████       | 200/300 [54:18<27:09, 16.29s/it]textattack: Saving checkpoint under "checkpoints\1722744863948.ta.chkpt" at 2024-08-04 12:14:23 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 223 / 17 / 250:  83%|████████████████▋   | 250/300 [59:45<11:57, 14.34s/it]textattack: Saving checkpoint under "checkpoints\1722745190797.ta.chkpt" at 2024-08-04 12:19:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 267 / 22 / 300: 100%|██████████████████| 300/300 [1:04:35<00:00, 12.92s/it]textattack: Saving checkpoint under "checkpoints\1722745480125.ta.chkpt" at 2024-08-04 12:24:40 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 11 / 267 / 22 / 300: 100%|██████████████████| 300/300 [1:04:35<00:00, 12.92s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 3.96%  |
| Average perturbed word %:     | 3.37%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 63.24  |
+-------------------------------+--------+


In [78]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 47 / 0 / 50:  17%|████▏                    | 50/300 [04:56<24:42,  5.93s/it]textattack: Saving checkpoint under "checkpoints\1722745776847.ta.chkpt" at 2024-08-04 12:29:36 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 90 / 3 / 100:  33%|███████▋               | 100/300 [09:55<19:50,  5.95s/it]textattack: Saving checkpoint under "checkpoints\1722746075660.ta.chkpt" at 2024-08-04 12:34:35 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 132 / 6 / 150:  50%|██████████▌          | 150/300 [14:47<14:47,  5.92s/it]textattack: Saving checkpoint under "checkpoints\1722746368118.ta.chkpt" at 2024-08-04 12:39:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 172 / 13 / 200:  67%|█████████████▎      | 200/300 [19:17<09:38,  5.79s/it]textattack: Saving checkpoint under "checkpoints\1722746637839.ta.chkpt" at 2024-08-04 12:43:57 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 214 / 16 / 250:  83%|████████████████▋   | 250/300 [24:32<04:54,  5.89s/it]textattack: Saving checkpoint under "checkpoints\1722746952421.ta.chkpt" at 2024-08-04 12:49:12 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 257 / 22 / 300: 100%|████████████████████| 300/300 [29:12<00:00,  5.84s/it]textattack: Saving checkpoint under "checkpoints\1722747232331.ta.chkpt" at 2024-08-04 12:53:52 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 21 / 257 / 22 / 300: 100%|████████████████████| 300/300 [29:12<00:00,  5.84s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 21     |
| Number of failed attacks:     | 257    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 85.67% |
| Attack success rate:          | 7.55%  |
| Average perturbed word %:     | 6.29%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 62.62  |
+-------------------------------+--------+


In [79]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175ep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 3 / 46 / 1 / 50:  17%|████▏                    | 50/300 [04:51<24:15,  5.82s/it]textattack: Saving checkpoint under "checkpoints\1722747523538.ta.chkpt" at 2024-08-04 12:58:43 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 89 / 5 / 100:  33%|███████▋               | 100/300 [09:40<19:21,  5.81s/it]textattack: Saving checkpoint under "checkpoints\1722747813314.ta.chkpt" at 2024-08-04 13:03:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 131 / 9 / 150:  50%|██████████▌          | 150/300 [14:27<14:27,  5.79s/it]textattack: Saving checkpoint under "checkpoints\1722748100382.ta.chkpt" at 2024-08-04 13:08:20 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 166 / 21 / 200:  67%|█████████████▎      | 200/300 [18:26<09:13,  5.53s/it]textattack: Saving checkpoint under "checkpoints\1722748339437.ta.chkpt" at 2024-08-04 13:12:19 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 205 / 28 / 250:  83%|████████████████▋   | 250/300 [23:08<04:37,  5.55s/it]textattack: Saving checkpoint under "checkpoints\1722748620819.ta.chkpt" at 2024-08-04 13:17:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 249 / 34 / 300: 100%|████████████████████| 300/300 [27:52<00:00,  5.57s/it]textattack: Saving checkpoint under "checkpoints\1722748904979.ta.chkpt" at 2024-08-04 13:21:44 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 17 / 249 / 34 / 300: 100%|████████████████████| 300/300 [27:52<00:00,  5.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 249    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 83.0%  |
| Attack success rate:          | 6.39%  |
| Average perturbed word %:     | 10.18% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 62.21  |
+-------------------------------+--------+


In [80]:
et3 = datetime.datetime.now()
et3 - et2

datetime.timedelta(seconds=29007, microseconds=705516)

### Threshold = 0.95

In [81]:
suffix = 'ep095.csv'
thre = 0.95

#### BERT

In [82]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [02:47<13:56,  3.35s/it]textattack: Saving checkpoint under "checkpoints\1722749072854.ta.chkpt" at 2024-08-04 13:24:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 89 / 10 / 100:  33%|███████▎              | 100/300 [05:36<11:13,  3.37s/it]textattack: Saving checkpoint under "checkpoints\1722749242188.ta.chkpt" at 2024-08-04 13:27:22 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 134 / 15 / 150:  50%|██████████▌          | 150/300 [08:26<08:26,  3.38s/it]textattack: Saving checkpoint under "checkpoints\1722749412152.ta.chkpt" at 2024-08-04 13:30:12 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [11:35<05:47,  3.48s/it]textattack: Saving checkpoint under "checkpoints\1722749600769.ta.chkpt" at 2024-08-04 13:33:20 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 224 / 25 / 250:  83%|█████████████████▌   | 250/300 [14:42<02:56,  3.53s/it]textattack: Saving checkpoint under "checkpoints\1722749788195.ta.chkpt" at 2024-08-04 13:36:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [17:39<00:00,  3.53s/it]textattack: Saving checkpoint under "checkpoints\1722749965191.ta.chkpt" at 2024-08-04 13:39:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [17:39<00:00,  3.53s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 2.04%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.88  |
+-------------------------------+--------+


In [83]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [02:47<13:59,  3.36s/it]textattack: Saving checkpoint under "checkpoints\1722750133511.ta.chkpt" at 2024-08-04 13:42:13 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 89 / 11 / 100:  33%|███████▎              | 100/300 [05:28<10:57,  3.29s/it]textattack: Saving checkpoint under "checkpoints\1722750294230.ta.chkpt" at 2024-08-04 13:44:54 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 134 / 16 / 150:  50%|██████████▌          | 150/300 [08:15<08:15,  3.30s/it]textattack: Saving checkpoint under "checkpoints\1722750460700.ta.chkpt" at 2024-08-04 13:47:40 after 150 attacks.


...

[Succeeded / Failed / Skipped / Total] 0 / 223 / 27 / 250:  83%|█████████████████▌   | 250/300 [14:02<02:48,  3.37s/it]textattack: Saving checkpoint under "checkpoints\1722750807682.ta.chkpt" at 2024-08-04 13:53:27 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 268 / 32 / 300: 100%|█████████████████████| 300/300 [16:57<00:00,  3.39s/it]textattack: Saving checkpoint under "checkpoints\1722750982781.ta.chkpt" at 2024-08-04 13:56:22 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 268 / 32 / 300: 100%|█████████████████████| 300/300 [16:57<00:00,  3.39s/it]


D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 268    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 89.33% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.85  |
+-------------------------------+--------+


D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [84]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [02:38<13:14,  3.18s/it]textattack: Saving checkpoint under "checkpoints\1722751143013.ta.chkpt" at 2024-08-04 13:59:03 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 89 / 11 / 100:  33%|███████▎              | 100/300 [05:20<10:40,  3.20s/it]textattack: Saving checkpoint under "checkpoints\1722751304484.ta.chkpt" at 2024-08-04 14:01:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 133 / 16 / 150:  50%|██████████▌          | 150/300 [08:04<08:04,  3.23s/it]textattack: Saving checkpoint under "checkpoints\1722751468471.ta.chkpt" at 2024-08-04 14:04:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 178 / 21 / 200:  67%|██████████████       | 200/300 [11:03<05:31,  3.32s/it]textattack: Saving checkpoint under "checkpoints\1722751647934.ta.chkpt" at 2024-08-04 14:07:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 224 / 25 / 250:  83%|█████████████████▌   | 250/300 [14:08<02:49,  3.39s/it]textattack: Saving checkpoint under "checkpoints\1722751832323.ta.chkpt" at 2024-08-04 14:10:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [17:06<00:00,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1722752010388.ta.chkpt" at 2024-08-04 14:13:30 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 269 / 30 / 300: 100%|█████████████████████| 300/300 [17:06<00:00,  3.42s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 9.52%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.87  |
+-------------------------------+--------+


In [85]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 3 / 31 / 16 / 50:  17%|████                    | 50/300 [02:00<10:03,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1722752131288.ta.chkpt" at 2024-08-04 14:15:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 65 / 31 / 100:  33%|███████▎              | 100/300 [04:09<08:19,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1722752260305.ta.chkpt" at 2024-08-04 14:17:40 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 65 / 32 / 101:  34%|███████▍              | 101/300 [04:09<08:12,  2.47s/it]

[Succeeded / Failed / Skipped / Total] 10 / 100 / 40 / 150:  50%|██████████          | 150/300 [06:35<06:35,  2.64s/it]textattack: Saving checkpoint under "checkpoints\1722752406274.ta.chkpt" at 2024-08-04 14:20:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 134 / 55 / 200:  67%|█████████████▎      | 200/300 [09:05<04:32,  2.73s/it]textattack: Saving checkpoint under "checkpoints\1722752556398.ta.chkpt" at 2024-08-04 14:22:36 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 172 / 66 / 250:  83%|████████████████▋   | 250/300 [11:35<02:19,  2.78s/it]textattack: Saving checkpoint under "checkpoints\1722752706331.ta.chkpt" at 2024-08-04 14:25:06 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 209 / 78 / 300: 100%|████████████████████| 300/300 [14:08<00:00,  2.83s/it]textattack: Saving checkpoint under "checkpoints\1722752858902.ta.chkpt" at 2024-08-04 14:27:38 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 209 / 78 / 300: 100%|████████████████████| 300/300 [14:08<00:00,  2.83s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 209    |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 69.67% |
| Attack success rate:          | 5.86%  |
| Average perturbed word %:     | 3.03%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.81  |
+-------------------------------+--------+


In [86]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [02:42<13:31,  3.25s/it]textattack: Saving checkpoint under "checkpoints\1722753021498.ta.chkpt" at 2024-08-04 14:30:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 75 / 20 / 100:  33%|███████▎              | 100/300 [04:58<09:57,  2.99s/it]textattack: Saving checkpoint under "checkpoints\1722753158061.ta.chkpt" at 2024-08-04 14:32:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 116 / 29 / 150:  50%|██████████▌          | 150/300 [07:34<07:34,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722753313166.ta.chkpt" at 2024-08-04 14:35:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 150 / 45 / 200:  67%|██████████████       | 200/300 [10:03<05:01,  3.02s/it]textattack: Saving checkpoint under "checkpoints\1722753462275.ta.chkpt" at 2024-08-04 14:37:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 192 / 51 / 250:  83%|█████████████████▌   | 250/300 [12:52<02:34,  3.09s/it]textattack: Saving checkpoint under "checkpoints\1722753631983.ta.chkpt" at 2024-08-04 14:40:31 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 234 / 58 / 300: 100%|█████████████████████| 300/300 [15:46<00:00,  3.15s/it]textattack: Saving checkpoint under "checkpoints\1722753805202.ta.chkpt" at 2024-08-04 14:43:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 234 / 58 / 300: 100%|█████████████████████| 300/300 [15:46<00:00,  3.15s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 234    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 78.0%  |
| Attack success rate:          | 3.31%  |
| Average perturbed word %:     | 2.76%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 31.07  |
+-------------------------------+--------+


In [87]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 40 / 9 / 50:  17%|████▏                    | 50/300 [02:44<13:43,  3.29s/it]textattack: Saving checkpoint under "checkpoints\1722753970073.ta.chkpt" at 2024-08-04 14:46:10 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 79 / 18 / 100:  33%|███████▎              | 100/300 [05:15<10:31,  3.16s/it]textattack: Saving checkpoint under "checkpoints\1722754121064.ta.chkpt" at 2024-08-04 14:48:41 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 120 / 27 / 150:  50%|██████████▌          | 150/300 [07:46<07:46,  3.11s/it]textattack: Saving checkpoint under "checkpoints\1722754271619.ta.chkpt" at 2024-08-04 14:51:11 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 155 / 42 / 200:  67%|██████████████       | 200/300 [10:15<05:07,  3.08s/it]textattack: Saving checkpoint under "checkpoints\1722754420823.ta.chkpt" at 2024-08-04 14:53:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 199 / 48 / 250:  83%|█████████████████▌   | 250/300 [13:09<02:37,  3.16s/it]textattack: Saving checkpoint under "checkpoints\1722754595351.ta.chkpt" at 2024-08-04 14:56:35 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 242 / 55 / 300: 100%|█████████████████████| 300/300 [15:59<00:00,  3.20s/it]textattack: Saving checkpoint under "checkpoints\1722754765297.ta.chkpt" at 2024-08-04 14:59:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 242 / 55 / 300: 100%|█████████████████████| 300/300 [15:59<00:00,  3.20s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 242    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 80.67% |
| Attack success rate:          | 1.22%  |
| Average perturbed word %:     | 3.27%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.91  |
+-------------------------------+--------+


In [88]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 2 / 37 / 11 / 50:  17%|███▋                  | 50/300 [26:40<2:13:24, 32.02s/it]textattack: Saving checkpoint under "checkpoints\1722756366309.ta.chkpt" at 2024-08-04 15:26:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 73 / 20 / 100:  33%|███████▎              | 100/300 [29:31<59:03, 17.72s/it]textattack: Saving checkpoint under "checkpoints\1722756537184.ta.chkpt" at 2024-08-04 15:28:57 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 105 / 32 / 150:  50%|██████████          | 150/300 [32:18<32:18, 12.92s/it]textattack: Saving checkpoint under "checkpoints\1722756703474.ta.chkpt" at 2024-08-04 15:31:43 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 138 / 45 / 200:  67%|█████████████▎      | 200/300 [35:20<17:40, 10.60s/it]textattack: Saving checkpoint under "checkpoints\1722756886076.ta.chkpt" at 2024-08-04 15:34:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 180 / 52 / 250:  83%|████████████████▋   | 250/300 [38:36<07:43,  9.27s/it]textattack: Saving checkpoint under "checkpoints\1722757082164.ta.chkpt" at 2024-08-04 15:38:02 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 224 / 58 / 300: 100%|████████████████████| 300/300 [42:00<00:00,  8.40s/it]textattack: Saving checkpoint under "checkpoints\1722757285777.ta.chkpt" at 2024-08-04 15:41:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 18 / 224 / 58 / 300: 100%|████████████████████| 300/300 [42:00<00:00,  8.40s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 18     |
| Number of failed attacks:     | 224    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 74.67% |
| Attack success rate:          | 7.44%  |
| Average perturbed word %:     | 3.1%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.97  |
+-------------------------------+--------+


In [89]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 40 / 9 / 50:  17%|████▏                    | 50/300 [02:58<14:52,  3.57s/it]textattack: Saving checkpoint under "checkpoints\1722757464379.ta.chkpt" at 2024-08-04 15:44:24 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 76 / 19 / 100:  33%|███████▎              | 100/300 [05:49<11:39,  3.50s/it]textattack: Saving checkpoint under "checkpoints\1722757635796.ta.chkpt" at 2024-08-04 15:47:15 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 114 / 28 / 150:  50%|██████████▌          | 150/300 [08:47<08:47,  3.52s/it]textattack: Saving checkpoint under "checkpoints\1722757813882.ta.chkpt" at 2024-08-04 15:50:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 147 / 43 / 200:  67%|█████████████▎      | 200/300 [11:42<05:51,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1722757988778.ta.chkpt" at 2024-08-04 15:53:08 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 189 / 51 / 250:  83%|████████████████▋   | 250/300 [14:53<02:58,  3.57s/it]textattack: Saving checkpoint under "checkpoints\1722758178963.ta.chkpt" at 2024-08-04 15:56:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 233 / 56 / 300: 100%|████████████████████| 300/300 [18:21<00:00,  3.67s/it]textattack: Saving checkpoint under "checkpoints\1722758387721.ta.chkpt" at 2024-08-04 15:59:47 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 11 / 233 / 56 / 300: 100%|████████████████████| 300/300 [18:21<00:00,  3.67s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 233    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 77.67% |
| Attack success rate:          | 4.51%  |
| Average perturbed word %:     | 2.93%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.89  |
+-------------------------------+--------+


In [90]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 41 / 9 / 50:  17%|████▏                    | 50/300 [02:54<14:31,  3.49s/it]textattack: Saving checkpoint under "checkpoints\1722758562252.ta.chkpt" at 2024-08-04 16:02:42 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 78 / 20 / 100:  33%|███████▎              | 100/300 [05:39<11:18,  3.39s/it]textattack: Saving checkpoint under "checkpoints\1722758727178.ta.chkpt" at 2024-08-04 16:05:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 114 / 33 / 150:  50%|██████████▌          | 150/300 [08:26<08:26,  3.37s/it]textattack: Saving checkpoint under "checkpoints\1722758893953.ta.chkpt" at 2024-08-04 16:08:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 150 / 47 / 200:  67%|██████████████       | 200/300 [11:26<05:43,  3.43s/it]textattack: Saving checkpoint under "checkpoints\1722759074334.ta.chkpt" at 2024-08-04 16:11:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 192 / 54 / 250:  83%|█████████████████▌   | 250/300 [14:37<02:55,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1722759265089.ta.chkpt" at 2024-08-04 16:14:25 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 235 / 60 / 300: 100%|█████████████████████| 300/300 [17:58<00:00,  3.59s/it]textattack: Saving checkpoint under "checkpoints\1722759466375.ta.chkpt" at 2024-08-04 16:17:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 5 / 235 / 60 / 300: 100%|█████████████████████| 300/300 [17:58<00:00,  3.60s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 235    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 78.33% |
| Attack success rate:          | 2.08%  |
| Average perturbed word %:     | 2.98%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.84  |
+-------------------------------+--------+

#### CLIP

In [91]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [02:34<12:50,  3.08s/it]textattack: Saving checkpoint under "checkpoints\1722759620628.ta.chkpt" at 2024-08-04 16:20:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [04:51<09:42,  2.91s/it]textattack: Saving checkpoint under "checkpoints\1722759757874.ta.chkpt" at 2024-08-04 16:22:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 135 / 14 / 150:  50%|██████████▌          | 150/300 [07:12<07:12,  2.88s/it]textattack: Saving checkpoint under "checkpoints\1722759899167.ta.chkpt" at 2024-08-04 16:24:59 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [09:28<04:44,  2.84s/it]textattack: Saving checkpoint under "checkpoints\1722760035385.ta.chkpt" at 2024-08-04 16:27:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 225 / 24 / 250:  83%|█████████████████▌   | 250/300 [11:57<02:23,  2.87s/it]textattack: Saving checkpoint under "checkpoints\1722760183691.ta.chkpt" at 2024-08-04 16:29:43 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [14:18<00:00,  2.86s/it]textattack: Saving checkpoint under "checkpoints\1722760324739.ta.chkpt" at 2024-08-04 16:32:04 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [14:18<00:00,  2.86s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 270    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 7.41%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.91  |
+-------------------------------+--------+


In [92]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [02:09<10:48,  2.59s/it]textattack: Saving checkpoint under "checkpoints\1722760454583.ta.chkpt" at 2024-08-04 16:34:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [04:24<08:48,  2.64s/it]textattack: Saving checkpoint under "checkpoints\1722760588997.ta.chkpt" at 2024-08-04 16:36:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 135 / 14 / 150:  50%|██████████▌          | 150/300 [06:42<06:42,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1722760727104.ta.chkpt" at 2024-08-04 16:38:47 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [08:54<04:27,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1722760859483.ta.chkpt" at 2024-08-04 16:40:59 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 225 / 24 / 250:  83%|█████████████████▌   | 250/300 [11:24<02:16,  2.74s/it]textattack: Saving checkpoint under "checkpoints\1722761009702.ta.chkpt" at 2024-08-04 16:43:29 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [13:45<00:00,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1722761150109.ta.chkpt" at 2024-08-04 16:45:50 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [13:45<00:00,  2.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 270    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 7.41%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.91  |
+-------------------------------+--------+


In [93]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 46 / 4 / 50:  17%|████▏                    | 50/300 [02:10<10:50,  2.60s/it]textattack: Saving checkpoint under "checkpoints\1722761280467.ta.chkpt" at 2024-08-04 16:48:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 91 / 9 / 100:  33%|███████▋               | 100/300 [04:21<08:43,  2.62s/it]textattack: Saving checkpoint under "checkpoints\1722761412069.ta.chkpt" at 2024-08-04 16:50:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 135 / 14 / 150:  50%|██████████▌          | 150/300 [06:38<06:38,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1722761548534.ta.chkpt" at 2024-08-04 16:52:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 179 / 20 / 200:  67%|██████████████       | 200/300 [08:50<04:25,  2.65s/it]textattack: Saving checkpoint under "checkpoints\1722761680442.ta.chkpt" at 2024-08-04 16:54:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 225 / 24 / 250:  83%|█████████████████▌   | 250/300 [11:17<02:15,  2.71s/it]textattack: Saving checkpoint under "checkpoints\1722761827376.ta.chkpt" at 2024-08-04 16:57:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [13:37<00:00,  2.73s/it]textattack: Saving checkpoint under "checkpoints\1722761967954.ta.chkpt" at 2024-08-04 16:59:27 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 270 / 29 / 300: 100%|█████████████████████| 300/300 [13:37<00:00,  2.73s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 270    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 0.37%  |
| Average perturbed word %:     | 7.41%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.91  |
+-------------------------------+--------+


In [94]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 45 / 5 / 50:  17%|████▏                    | 50/300 [02:06<10:34,  2.54s/it]textattack: Saving checkpoint under "checkpoints\1722762094975.ta.chkpt" at 2024-08-04 17:01:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 87 / 13 / 100:  33%|███████▎              | 100/300 [04:13<08:27,  2.54s/it]textattack: Saving checkpoint under "checkpoints\1722762222003.ta.chkpt" at 2024-08-04 17:03:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 130 / 19 / 150:  50%|██████████▌          | 150/300 [06:24<06:24,  2.57s/it]textattack: Saving checkpoint under "checkpoints\1722762352966.ta.chkpt" at 2024-08-04 17:05:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 164 / 35 / 200:  67%|██████████████       | 200/300 [08:06<04:03,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1722762455140.ta.chkpt" at 2024-08-04 17:07:35 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 204 / 44 / 250:  83%|█████████████████▌   | 250/300 [10:20<02:04,  2.48s/it]textattack: Saving checkpoint under "checkpoints\1722762588621.ta.chkpt" at 2024-08-04 17:09:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 249 / 49 / 300: 100%|█████████████████████| 300/300 [12:44<00:00,  2.55s/it]textattack: Saving checkpoint under "checkpoints\1722762732462.ta.chkpt" at 2024-08-04 17:12:12 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 249 / 49 / 300: 100%|█████████████████████| 300/300 [12:44<00:00,  2.55s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 249    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 83.0%  |
| Attack success rate:          | 0.8%   |
| Average perturbed word %:     | 2.73%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.64  |
+-------------------------------+--------+


In [95]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 1 / 47 / 2 / 50:  17%|████▏                    | 50/300 [02:28<12:23,  2.97s/it]textattack: Saving checkpoint under "checkpoints\1722762881245.ta.chkpt" at 2024-08-04 17:14:41 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 92 / 7 / 100:  33%|███████▋               | 100/300 [04:47<09:35,  2.88s/it]textattack: Saving checkpoint under "checkpoints\1722763020415.ta.chkpt" at 2024-08-04 17:17:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 136 / 13 / 150:  50%|██████████▌          | 150/300 [07:00<07:00,  2.80s/it]textattack: Saving checkpoint under "checkpoints\1722763153136.ta.chkpt" at 2024-08-04 17:19:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 175 / 23 / 200:  67%|██████████████       | 200/300 [09:01<04:30,  2.71s/it]textattack: Saving checkpoint under "checkpoints\1722763274003.ta.chkpt" at 2024-08-04 17:21:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 218 / 29 / 250:  83%|█████████████████▌   | 250/300 [11:25<02:17,  2.74s/it]textattack: Saving checkpoint under "checkpoints\1722763417667.ta.chkpt" at 2024-08-04 17:23:37 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 262 / 35 / 300: 100%|█████████████████████| 300/300 [13:44<00:00,  2.75s/it]textattack: Saving checkpoint under "checkpoints\1722763557059.ta.chkpt" at 2024-08-04 17:25:57 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 3 / 262 / 35 / 300: 100%|█████████████████████| 300/300 [13:44<00:00,  2.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 3      |
| Number of failed attacks:     | 262    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 87.33% |
| Attack success rate:          | 1.13%  |
| Average perturbed word %:     | 2.8%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.7   |
+-------------------------------+--------+


In [96]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|████▏                    | 50/300 [03:25<17:05,  4.10s/it]textattack: Saving checkpoint under "checkpoints\1722763762378.ta.chkpt" at 2024-08-04 17:29:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 93 / 7 / 100:  33%|███████▋               | 100/300 [06:11<12:23,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1722763928913.ta.chkpt" at 2024-08-04 17:32:08 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 138 / 10 / 150:  50%|██████████▌          | 150/300 [08:57<08:57,  3.58s/it]textattack: Saving checkpoint under "checkpoints\1722764094948.ta.chkpt" at 2024-08-04 17:34:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 178 / 20 / 200:  67%|██████████████       | 200/300 [11:20<05:40,  3.40s/it]textattack: Saving checkpoint under "checkpoints\1722764237567.ta.chkpt" at 2024-08-04 17:37:17 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 223 / 25 / 250:  83%|█████████████████▌   | 250/300 [14:08<02:49,  3.39s/it]textattack: Saving checkpoint under "checkpoints\1722764405960.ta.chkpt" at 2024-08-04 17:40:05 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 267 / 31 / 300: 100%|█████████████████████| 300/300 [16:35<00:00,  3.32s/it]textattack: Saving checkpoint under "checkpoints\1722764552562.ta.chkpt" at 2024-08-04 17:42:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 2 / 267 / 31 / 300: 100%|█████████████████████| 300/300 [16:35<00:00,  3.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 2      |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 0.74%  |
| Average perturbed word %:     | 5.09%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.67  |
+-------------------------------+--------+


In [97]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [29:02<2:25:14, 34.86s/it]textattack: Saving checkpoint under "checkpoints\1722766295652.ta.chkpt" at 2024-08-04 18:11:35 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 95 / 4 / 100:  33%|███████              | 100/300 [31:46<1:03:32, 19.06s/it]textattack: Saving checkpoint under "checkpoints\1722766459062.ta.chkpt" at 2024-08-04 18:14:19 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 142 / 7 / 150:  50%|███████████           | 150/300 [34:31<34:31, 13.81s/it]textattack: Saving checkpoint under "checkpoints\1722766624393.ta.chkpt" at 2024-08-04 18:17:04 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 185 / 14 / 200:  67%|██████████████       | 200/300 [37:13<18:36, 11.17s/it]textattack: Saving checkpoint under "checkpoints\1722766786355.ta.chkpt" at 2024-08-04 18:19:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 232 / 17 / 250:  83%|█████████████████▌   | 250/300 [40:08<08:01,  9.63s/it]textattack: Saving checkpoint under "checkpoints\1722766961063.ta.chkpt" at 2024-08-04 18:22:41 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [42:50<00:00,  8.57s/it]textattack: Saving checkpoint under "checkpoints\1722767123282.ta.chkpt" at 2024-08-04 18:25:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [42:50<00:00,  8.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 277    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 2.78%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.59  |
+-------------------------------+--------+


In [98]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 50 / 0 / 50:  17%|████▏                    | 50/300 [02:45<13:47,  3.31s/it]textattack: Saving checkpoint under "checkpoints\1722767288967.ta.chkpt" at 2024-08-04 18:28:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 97 / 3 / 100:  33%|███████▋               | 100/300 [05:42<11:25,  3.43s/it]textattack: Saving checkpoint under "checkpoints\1722767466399.ta.chkpt" at 2024-08-04 18:31:06 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 144 / 6 / 150:  50%|███████████           | 150/300 [08:37<08:37,  3.45s/it]textattack: Saving checkpoint under "checkpoints\1722767641235.ta.chkpt" at 2024-08-04 18:34:01 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 187 / 13 / 200:  67%|██████████████       | 200/300 [11:20<05:40,  3.40s/it]textattack: Saving checkpoint under "checkpoints\1722767803600.ta.chkpt" at 2024-08-04 18:36:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 234 / 16 / 250:  83%|█████████████████▌   | 250/300 [14:19<02:51,  3.44s/it]textattack: Saving checkpoint under "checkpoints\1722767982474.ta.chkpt" at 2024-08-04 18:39:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [17:04<00:00,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1722768148099.ta.chkpt" at 2024-08-04 18:42:28 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 277 / 22 / 300: 100%|█████████████████████| 300/300 [17:04<00:00,  3.42s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 277    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 92.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 2.78%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.81  |
+-------------------------------+--------+


In [99]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175ep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|████▏                    | 50/300 [02:39<13:15,  3.18s/it]textattack: Saving checkpoint under "checkpoints\1722768307523.ta.chkpt" at 2024-08-04 18:45:07 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 95 / 5 / 100:  33%|███████▋               | 100/300 [05:21<10:43,  3.22s/it]textattack: Saving checkpoint under "checkpoints\1722768470299.ta.chkpt" at 2024-08-04 18:47:50 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 141 / 9 / 150:  50%|███████████           | 150/300 [08:00<08:00,  3.20s/it]textattack: Saving checkpoint under "checkpoints\1722768628858.ta.chkpt" at 2024-08-04 18:50:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 179 / 21 / 200:  67%|██████████████       | 200/300 [10:09<05:04,  3.05s/it]textattack: Saving checkpoint under "checkpoints\1722768757831.ta.chkpt" at 2024-08-04 18:52:37 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 222 / 28 / 250:  83%|█████████████████▌   | 250/300 [12:52<02:34,  3.09s/it]textattack: Saving checkpoint under "checkpoints\1722768921121.ta.chkpt" at 2024-08-04 18:55:21 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [15:29<00:00,  3.10s/it]textattack: Saving checkpoint under "checkpoints\1722769078143.ta.chkpt" at 2024-08-04 18:57:58 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 0 / 266 / 34 / 300: 100%|█████████████████████| 300/300 [15:29<00:00,  3.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 0      |
| Number of failed attacks:     | 266    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 88.67% |
| Attack success rate:          | 0.0%   |
| Average perturbed word %:     | nan%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.51  |
+-------------------------------+--------+



D:\anaconda3\envs\tf\lib\site-packages\textattack\metrics\attack_metrics\words_perturbed.py:83: RuntimeWarning: Mean of empty slice.
  average_perc_words_perturbed = self.perturbed_word_percentages.mean()
D:\anaconda3\envs\tf\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [100]:
et4 = datetime.datetime.now()
et4 - et3

datetime.timedelta(seconds=20173, microseconds=149390)

### Threshold = 1 - 0.6/$\pi$ = 0.809

In [101]:
suffix = 'ep0809.csv'
thre = 1-0.6/np.pi

#### BERT

In [102]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 15 / 30 / 5 / 50:  17%|████                    | 50/300 [04:51<24:19,  5.84s/it]textattack: Saving checkpoint under "checkpoints\1722769371496.ta.chkpt" at 2024-08-04 19:02:51 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 52 / 10 / 100:  33%|███████              | 100/300 [09:39<19:18,  5.79s/it]textattack: Saving checkpoint under "checkpoints\1722769658603.ta.chkpt" at 2024-08-04 19:07:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 75 / 15 / 150:  50%|██████████▌          | 150/300 [14:41<14:41,  5.88s/it]textattack: Saving checkpoint under "checkpoints\1722769960837.ta.chkpt" at 2024-08-04 19:12:40 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 83 / 96 / 21 / 200:  67%|██████████████       | 200/300 [20:03<10:01,  6.02s/it]textattack: Saving checkpoint under "checkpoints\1722770282688.ta.chkpt" at 2024-08-04 19:18:02 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 105 / 120 / 25 / 250:  83%|███████████████▊   | 250/300 [25:05<05:01,  6.02s/it]textattack: Saving checkpoint under "checkpoints\1722770584730.ta.chkpt" at 2024-08-04 19:23:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 122 / 148 / 30 / 300: 100%|███████████████████| 300/300 [30:43<00:00,  6.15s/it]textattack: Saving checkpoint under "checkpoints\1722770923186.ta.chkpt" at 2024-08-04 19:28:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 122 / 148 / 30 / 300: 100%|███████████████████| 300/300 [30:43<00:00,  6.15s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 122    |
| Number of failed attacks:     | 148    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 49.33% |
| Attack success rate:          | 45.19% |
| Average perturbed word %:     | 15.96% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 70.51  |
+-------------------------------+--------+


In [103]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 16 / 29 / 5 / 50:  17%|████                    | 50/300 [05:03<25:18,  6.07s/it]textattack: Saving checkpoint under "checkpoints\1722771227097.ta.chkpt" at 2024-08-04 19:33:47 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 51 / 11 / 100:  33%|███████              | 100/300 [09:26<18:52,  5.66s/it]textattack: Saving checkpoint under "checkpoints\1722771489941.ta.chkpt" at 2024-08-04 19:38:09 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 74 / 16 / 150:  50%|██████████▌          | 150/300 [13:56<13:56,  5.57s/it]textattack: Saving checkpoint under "checkpoints\1722771759725.ta.chkpt" at 2024-08-04 19:42:39 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 84 / 93 / 23 / 200:  67%|██████████████       | 200/300 [18:28<09:14,  5.54s/it]textattack: Saving checkpoint under "checkpoints\1722772032393.ta.chkpt" at 2024-08-04 19:47:12 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 105 / 118 / 27 / 250:  83%|███████████████▊   | 250/300 [23:23<04:40,  5.61s/it]textattack: Saving checkpoint under "checkpoints\1722772326648.ta.chkpt" at 2024-08-04 19:52:06 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 123 / 145 / 32 / 300: 100%|███████████████████| 300/300 [28:48<00:00,  5.76s/it]textattack: Saving checkpoint under "checkpoints\1722772651566.ta.chkpt" at 2024-08-04 19:57:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 123 / 145 / 32 / 300: 100%|███████████████████| 300/300 [28:48<00:00,  5.76s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 123    |
| Number of failed attacks:     | 145    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 48.33% |
| Attack success rate:          | 45.9%  |
| Average perturbed word %:     | 16.37% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 70.38  |
+-------------------------------+--------+


In [104]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 16 / 29 / 5 / 50:  17%|████                    | 50/300 [04:34<22:50,  5.48s/it]textattack: Saving checkpoint under "checkpoints\1722772926005.ta.chkpt" at 2024-08-04 20:02:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 50 / 11 / 100:  33%|███████              | 100/300 [08:41<17:23,  5.22s/it]textattack: Saving checkpoint under "checkpoints\1722773173516.ta.chkpt" at 2024-08-04 20:06:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 73 / 16 / 150:  50%|██████████▌          | 150/300 [13:00<13:00,  5.21s/it]textattack: Saving checkpoint under "checkpoints\1722773432745.ta.chkpt" at 2024-08-04 20:10:32 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 86 / 93 / 21 / 200:  67%|██████████████       | 200/300 [17:38<08:49,  5.29s/it]textattack: Saving checkpoint under "checkpoints\1722773710755.ta.chkpt" at 2024-08-04 20:15:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 107 / 118 / 25 / 250:  83%|███████████████▊   | 250/300 [22:28<04:29,  5.39s/it]textattack: Saving checkpoint under "checkpoints\1722774000309.ta.chkpt" at 2024-08-04 20:20:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 123 / 147 / 30 / 300: 100%|███████████████████| 300/300 [27:40<00:00,  5.54s/it]textattack: Saving checkpoint under "checkpoints\1722774312564.ta.chkpt" at 2024-08-04 20:25:12 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 123 / 147 / 30 / 300: 100%|███████████████████| 300/300 [27:40<00:00,  5.54s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 123    |
| Number of failed attacks:     | 147    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 49.0%  |
| Attack success rate:          | 45.56% |
| Average perturbed word %:     | 15.55% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 70.09  |
+-------------------------------+--------+


In [105]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 23 / 11 / 16 / 50:  17%|███▊                   | 50/300 [02:37<13:08,  3.15s/it]textattack: Saving checkpoint under "checkpoints\1722774470387.ta.chkpt" at 2024-08-04 20:27:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 25 / 31 / 100:  33%|███████              | 100/300 [05:24<10:48,  3.24s/it]textattack: Saving checkpoint under "checkpoints\1722774637110.ta.chkpt" at 2024-08-04 20:30:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 37 / 40 / 150:  50%|██████████▌          | 150/300 [08:37<08:37,  3.45s/it]textattack: Saving checkpoint under "checkpoints\1722774830413.ta.chkpt" at 2024-08-04 20:33:50 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 52 / 55 / 200:  67%|██████████████       | 200/300 [12:07<06:03,  3.64s/it]textattack: Saving checkpoint under "checkpoints\1722775039797.ta.chkpt" at 2024-08-04 20:37:19 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 119 / 65 / 66 / 250:  83%|████████████████▋   | 250/300 [15:05<03:01,  3.62s/it]textattack: Saving checkpoint under "checkpoints\1722775218502.ta.chkpt" at 2024-08-04 20:40:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 146 / 76 / 78 / 300: 100%|████████████████████| 300/300 [18:36<00:00,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1722775428903.ta.chkpt" at 2024-08-04 20:43:48 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 146 / 76 / 78 / 300: 100%|████████████████████| 300/300 [18:36<00:00,  3.72s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 146    |
| Number of failed attacks:     | 76     |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 25.33% |
| Attack success rate:          | 65.77% |
| Average perturbed word %:     | 10.31% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.75  |
+-------------------------------+--------+


In [106]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 26 / 15 / 9 / 50:  17%|████                    | 50/300 [03:33<17:48,  4.28s/it]textattack: Saving checkpoint under "checkpoints\1722775642949.ta.chkpt" at 2024-08-04 20:47:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 29 / 20 / 100:  33%|███████              | 100/300 [06:56<13:52,  4.16s/it]textattack: Saving checkpoint under "checkpoints\1722775845368.ta.chkpt" at 2024-08-04 20:50:45 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 48 / 29 / 150:  50%|██████████▌          | 150/300 [10:31<10:31,  4.21s/it]textattack: Saving checkpoint under "checkpoints\1722776061022.ta.chkpt" at 2024-08-04 20:54:21 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 89 / 66 / 45 / 200:  67%|██████████████       | 200/300 [14:23<07:11,  4.32s/it]textattack: Saving checkpoint under "checkpoints\1722776293019.ta.chkpt" at 2024-08-04 20:58:13 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 113 / 86 / 51 / 250:  83%|████████████████▋   | 250/300 [18:45<03:45,  4.50s/it]textattack: Saving checkpoint under "checkpoints\1722776554860.ta.chkpt" at 2024-08-04 21:02:34 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 135 / 107 / 58 / 300: 100%|███████████████████| 300/300 [23:10<00:00,  4.64s/it]textattack: Saving checkpoint under "checkpoints\1722776819912.ta.chkpt" at 2024-08-04 21:06:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 135 / 107 / 58 / 300: 100%|███████████████████| 300/300 [23:10<00:00,  4.64s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 135    |
| Number of failed attacks:     | 107    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 35.67% |
| Attack success rate:          | 55.79% |
| Average perturbed word %:     | 11.25% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 64.43  |
+-------------------------------+--------+


In [107]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 21 / 20 / 9 / 50:  17%|████                    | 50/300 [03:57<19:47,  4.75s/it]textattack: Saving checkpoint under "checkpoints\1722777057714.ta.chkpt" at 2024-08-04 21:10:57 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 37 / 18 / 100:  33%|███████              | 100/300 [07:26<14:53,  4.47s/it]textattack: Saving checkpoint under "checkpoints\1722777267009.ta.chkpt" at 2024-08-04 21:14:27 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 56 / 27 / 150:  50%|██████████▌          | 150/300 [11:08<11:08,  4.46s/it]textattack: Saving checkpoint under "checkpoints\1722777489176.ta.chkpt" at 2024-08-04 21:18:09 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 79 / 79 / 42 / 200:  67%|██████████████       | 200/300 [15:00<07:30,  4.50s/it]textattack: Saving checkpoint under "checkpoints\1722777720837.ta.chkpt" at 2024-08-04 21:22:00 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 98 / 104 / 48 / 250:  83%|████████████████▋   | 250/300 [19:30<03:54,  4.68s/it]textattack: Saving checkpoint under "checkpoints\1722777990356.ta.chkpt" at 2024-08-04 21:26:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 113 / 132 / 55 / 300: 100%|███████████████████| 300/300 [24:19<00:00,  4.87s/it]textattack: Saving checkpoint under "checkpoints\1722778280158.ta.chkpt" at 2024-08-04 21:31:20 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 113 / 132 / 55 / 300: 100%|███████████████████| 300/300 [24:19<00:00,  4.87s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 113    |
| Number of failed attacks:     | 132    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 44.0%  |
| Attack success rate:          | 46.12% |
| Average perturbed word %:     | 8.39%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 65.71  |
+-------------------------------+--------+


In [108]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 22 / 17 / 11 / 50:  17%|███▌                 | 50/300 [26:24<2:12:04, 31.70s/it]textattack: Saving checkpoint under "checkpoints\1722779865259.ta.chkpt" at 2024-08-04 21:57:45 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 48 / 32 / 20 / 100:  33%|██████▎            | 100/300 [31:34<1:03:09, 18.95s/it]textattack: Saving checkpoint under "checkpoints\1722780175248.ta.chkpt" at 2024-08-04 22:02:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 66 / 52 / 32 / 150:  50%|██████████▌          | 150/300 [36:48<36:48, 14.72s/it]textattack: Saving checkpoint under "checkpoints\1722780488441.ta.chkpt" at 2024-08-04 22:08:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 83 / 72 / 45 / 200:  67%|██████████████       | 200/300 [42:44<21:22, 12.82s/it]textattack: Saving checkpoint under "checkpoints\1722780844923.ta.chkpt" at 2024-08-04 22:14:04 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 105 / 93 / 52 / 250:  83%|████████████████▋   | 250/300 [49:18<09:51, 11.83s/it]textattack: Saving checkpoint under "checkpoints\1722781238757.ta.chkpt" at 2024-08-04 22:20:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 126 / 116 / 58 / 300: 100%|███████████████████| 300/300 [55:47<00:00, 11.16s/it]textattack: Saving checkpoint under "checkpoints\1722781627738.ta.chkpt" at 2024-08-04 22:27:07 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 126 / 116 / 58 / 300: 100%|███████████████████| 300/300 [55:47<00:00, 11.16s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 126    |
| Number of failed attacks:     | 116    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 38.67% |
| Attack success rate:          | 52.07% |
| Average perturbed word %:     | 5.03%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 62.13  |
+-------------------------------+--------+

In [109]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 23 / 18 / 9 / 50:  17%|████                    | 50/300 [06:19<31:37,  7.59s/it]textattack: Saving checkpoint under "checkpoints\1722782008087.ta.chkpt" at 2024-08-04 22:33:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 47 / 34 / 19 / 100:  33%|███████              | 100/300 [11:36<23:12,  6.96s/it]textattack: Saving checkpoint under "checkpoints\1722782324767.ta.chkpt" at 2024-08-04 22:38:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 52 / 28 / 150:  50%|██████████▌          | 150/300 [17:07<17:07,  6.85s/it]textattack: Saving checkpoint under "checkpoints\1722782655781.ta.chkpt" at 2024-08-04 22:44:15 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 84 / 73 / 43 / 200:  67%|██████████████       | 200/300 [22:54<11:27,  6.87s/it]textattack: Saving checkpoint under "checkpoints\1722783003384.ta.chkpt" at 2024-08-04 22:50:03 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 102 / 97 / 51 / 250:  83%|████████████████▋   | 250/300 [30:01<06:00,  7.21s/it]textattack: Saving checkpoint under "checkpoints\1722783429855.ta.chkpt" at 2024-08-04 22:57:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 124 / 120 / 56 / 300: 100%|███████████████████| 300/300 [36:42<00:00,  7.34s/it]textattack: Saving checkpoint under "checkpoints\1722783830652.ta.chkpt" at 2024-08-04 23:03:50 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 124 / 120 / 56 / 300: 100%|███████████████████| 300/300 [36:42<00:00,  7.34s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 124    |
| Number of failed attacks:     | 120    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 50.82% |
| Average perturbed word %:     | 8.81%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 64.13  |
+-------------------------------+--------+


In [110]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 30 / 11 / 9 / 50:  17%|████                    | 50/300 [05:19<26:36,  6.39s/it]textattack: Saving checkpoint under "checkpoints\1722784150326.ta.chkpt" at 2024-08-04 23:09:10 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 29 / 20 / 100:  33%|███████              | 100/300 [10:21<20:42,  6.21s/it]textattack: Saving checkpoint under "checkpoints\1722784452110.ta.chkpt" at 2024-08-04 23:14:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 74 / 43 / 33 / 150:  50%|██████████▌          | 150/300 [14:52<14:52,  5.95s/it]textattack: Saving checkpoint under "checkpoints\1722784723332.ta.chkpt" at 2024-08-04 23:18:43 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 96 / 57 / 47 / 200:  67%|██████████████       | 200/300 [20:01<10:00,  6.01s/it]textattack: Saving checkpoint under "checkpoints\1722785032274.ta.chkpt" at 2024-08-04 23:23:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 119 / 77 / 54 / 250:  83%|████████████████▋   | 250/300 [26:14<05:14,  6.30s/it]textattack: Saving checkpoint under "checkpoints\1722785405794.ta.chkpt" at 2024-08-04 23:30:05 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 144 / 96 / 60 / 300: 100%|████████████████████| 300/300 [31:45<00:00,  6.35s/it]textattack: Saving checkpoint under "checkpoints\1722785736348.ta.chkpt" at 2024-08-04 23:35:36 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 144 / 96 / 60 / 300: 100%|████████████████████| 300/300 [31:45<00:00,  6.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 144    |
| Number of failed attacks:     | 96     |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 32.0%  |
| Attack success rate:          | 60.0%  |
| Average perturbed word %:     | 10.52% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 60.56  |
+-------------------------------+--------+


#### CLIP

In [111]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [04:26<22:11,  5.32s/it]textattack: Saving checkpoint under "checkpoints\1722786002826.ta.chkpt" at 2024-08-04 23:40:02 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 73 / 9 / 100:  33%|███████▎              | 100/300 [08:10<16:21,  4.91s/it]textattack: Saving checkpoint under "checkpoints\1722786227433.ta.chkpt" at 2024-08-04 23:43:47 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 106 / 14 / 150:  50%|██████████          | 150/300 [11:59<11:59,  4.80s/it]textattack: Saving checkpoint under "checkpoints\1722786456572.ta.chkpt" at 2024-08-04 23:47:36 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 135 / 20 / 200:  67%|█████████████▎      | 200/300 [15:25<07:42,  4.63s/it]textattack: Saving checkpoint under "checkpoints\1722786661684.ta.chkpt" at 2024-08-04 23:51:01 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 165 / 24 / 250:  83%|████████████████▋   | 250/300 [19:13<03:50,  4.61s/it]textattack: Saving checkpoint under "checkpoints\1722786889800.ta.chkpt" at 2024-08-04 23:54:49 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 198 / 29 / 300: 100%|████████████████████| 300/300 [22:49<00:00,  4.57s/it]textattack: Saving checkpoint under "checkpoints\1722787106547.ta.chkpt" at 2024-08-04 23:58:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 198 / 29 / 300: 100%|████████████████████| 300/300 [22:49<00:00,  4.57s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 198    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 66.0%  |
| Attack success rate:          | 26.94% |
| Average perturbed word %:     | 17.39% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 75.08  |
+-------------------------------+--------+


In [112]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 41 / 4 / 50:  17%|████▏                    | 50/300 [03:36<18:02,  4.33s/it]textattack: Saving checkpoint under "checkpoints\1722787323129.ta.chkpt" at 2024-08-05 00:02:03 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 73 / 9 / 100:  33%|███████▎              | 100/300 [07:10<14:21,  4.31s/it]textattack: Saving checkpoint under "checkpoints\1722787537680.ta.chkpt" at 2024-08-05 00:05:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 106 / 14 / 150:  50%|██████████          | 150/300 [10:50<10:50,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1722787757620.ta.chkpt" at 2024-08-05 00:09:17 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 135 / 20 / 200:  67%|█████████████▎      | 200/300 [14:12<07:06,  4.26s/it]textattack: Saving checkpoint under "checkpoints\1722787959029.ta.chkpt" at 2024-08-05 00:12:39 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 165 / 24 / 250:  83%|████████████████▋   | 250/300 [18:01<03:36,  4.33s/it]textattack: Saving checkpoint under "checkpoints\1722788188274.ta.chkpt" at 2024-08-05 00:16:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 198 / 29 / 300: 100%|████████████████████| 300/300 [21:36<00:00,  4.32s/it]textattack: Saving checkpoint under "checkpoints\1722788403497.ta.chkpt" at 2024-08-05 00:20:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 198 / 29 / 300: 100%|████████████████████| 300/300 [21:36<00:00,  4.32s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 198    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 66.0%  |
| Attack success rate:          | 26.94% |
| Average perturbed word %:     | 17.49% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 75.23  |
+-------------------------------+--------+


In [113]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:33<17:47,  4.27s/it]textattack: Saving checkpoint under "checkpoints\1722788617268.ta.chkpt" at 2024-08-05 00:23:37 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 73 / 9 / 100:  33%|███████▎              | 100/300 [07:05<14:11,  4.26s/it]textattack: Saving checkpoint under "checkpoints\1722788829556.ta.chkpt" at 2024-08-05 00:27:09 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 106 / 14 / 150:  50%|██████████          | 150/300 [10:46<10:46,  4.31s/it]textattack: Saving checkpoint under "checkpoints\1722789050513.ta.chkpt" at 2024-08-05 00:30:50 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 135 / 20 / 200:  67%|█████████████▎      | 200/300 [14:06<07:03,  4.23s/it]textattack: Saving checkpoint under "checkpoints\1722789250388.ta.chkpt" at 2024-08-05 00:34:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 165 / 24 / 250:  83%|████████████████▋   | 250/300 [17:55<03:35,  4.30s/it]textattack: Saving checkpoint under "checkpoints\1722789479064.ta.chkpt" at 2024-08-05 00:37:59 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 198 / 29 / 300: 100%|████████████████████| 300/300 [21:28<00:00,  4.30s/it]textattack: Saving checkpoint under "checkpoints\1722789692656.ta.chkpt" at 2024-08-05 00:41:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 198 / 29 / 300: 100%|████████████████████| 300/300 [21:29<00:00,  4.30s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 198    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 66.0%  |
| Attack success rate:          | 26.94% |
| Average perturbed word %:     | 17.35% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 75.08  |
+-------------------------------+--------+


In [114]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 9 / 36 / 5 / 50:  17%|████▏                    | 50/300 [03:19<16:35,  3.98s/it]textattack: Saving checkpoint under "checkpoints\1722789891974.ta.chkpt" at 2024-08-05 00:44:51 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 66 / 13 / 100:  33%|███████              | 100/300 [06:44<13:28,  4.04s/it]textattack: Saving checkpoint under "checkpoints\1722790097199.ta.chkpt" at 2024-08-05 00:48:17 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 99 / 19 / 150:  50%|██████████▌          | 150/300 [10:13<10:13,  4.09s/it]textattack: Saving checkpoint under "checkpoints\1722790306553.ta.chkpt" at 2024-08-05 00:51:46 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 46 / 119 / 35 / 200:  67%|█████████████▎      | 200/300 [12:36<06:18,  3.78s/it]textattack: Saving checkpoint under "checkpoints\1722790449150.ta.chkpt" at 2024-08-05 00:54:09 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 152 / 44 / 250:  83%|████████████████▋   | 250/300 [15:57<03:11,  3.83s/it]textattack: Saving checkpoint under "checkpoints\1722790650220.ta.chkpt" at 2024-08-05 00:57:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 191 / 49 / 300: 100%|████████████████████| 300/300 [19:55<00:00,  3.98s/it]textattack: Saving checkpoint under "checkpoints\1722790888044.ta.chkpt" at 2024-08-05 01:01:28 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 60 / 191 / 49 / 300: 100%|████████████████████| 300/300 [19:55<00:00,  3.98s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 60     |
| Number of failed attacks:     | 191    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 63.67% |
| Attack success rate:          | 23.9%  |
| Average perturbed word %:     | 13.2%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 73.86  |
+-------------------------------+--------+


In [115]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 41 / 2 / 50:  17%|████▏                    | 50/300 [03:59<19:59,  4.80s/it]textattack: Saving checkpoint under "checkpoints\1722791128229.ta.chkpt" at 2024-08-05 01:05:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 76 / 7 / 100:  33%|███████▎              | 100/300 [07:47<15:34,  4.67s/it]textattack: Saving checkpoint under "checkpoints\1722791355378.ta.chkpt" at 2024-08-05 01:09:15 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 112 / 13 / 150:  50%|██████████          | 150/300 [11:22<11:22,  4.55s/it]textattack: Saving checkpoint under "checkpoints\1722791570490.ta.chkpt" at 2024-08-05 01:12:50 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 141 / 23 / 200:  67%|█████████████▎      | 200/300 [14:25<07:12,  4.33s/it]textattack: Saving checkpoint under "checkpoints\1722791754064.ta.chkpt" at 2024-08-05 01:15:54 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 176 / 29 / 250:  83%|████████████████▋   | 250/300 [18:15<03:39,  4.38s/it]textattack: Saving checkpoint under "checkpoints\1722791984135.ta.chkpt" at 2024-08-05 01:19:44 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 47 / 218 / 35 / 300: 100%|████████████████████| 300/300 [22:13<00:00,  4.44s/it]textattack: Saving checkpoint under "checkpoints\1722792221523.ta.chkpt" at 2024-08-05 01:23:41 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 47 / 218 / 35 / 300: 100%|████████████████████| 300/300 [22:13<00:00,  4.44s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 47     |
| Number of failed attacks:     | 218    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 72.67% |
| Attack success rate:          | 17.74% |
| Average perturbed word %:     | 10.9%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 76.4   |
+-------------------------------+--------+


In [116]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 41 / 2 / 50:  17%|████▏                    | 50/300 [04:07<20:38,  4.95s/it]textattack: Saving checkpoint under "checkpoints\1722792469308.ta.chkpt" at 2024-08-05 01:27:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 76 / 7 / 100:  33%|███████▎              | 100/300 [07:53<15:47,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1722792695465.ta.chkpt" at 2024-08-05 01:31:35 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 112 / 10 / 150:  50%|██████████          | 150/300 [11:40<11:40,  4.67s/it]textattack: Saving checkpoint under "checkpoints\1722792922059.ta.chkpt" at 2024-08-05 01:35:22 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 140 / 20 / 200:  67%|█████████████▎      | 200/300 [14:43<07:21,  4.42s/it]textattack: Saving checkpoint under "checkpoints\1722793104944.ta.chkpt" at 2024-08-05 01:38:24 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 52 / 173 / 25 / 250:  83%|████████████████▋   | 250/300 [18:42<03:44,  4.49s/it]textattack: Saving checkpoint under "checkpoints\1722793344554.ta.chkpt" at 2024-08-05 01:42:24 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 214 / 31 / 300: 100%|████████████████████| 300/300 [22:37<00:00,  4.53s/it]textattack: Saving checkpoint under "checkpoints\1722793579566.ta.chkpt" at 2024-08-05 01:46:19 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 55 / 214 / 31 / 300: 100%|████████████████████| 300/300 [22:37<00:00,  4.53s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 55     |
| Number of failed attacks:     | 214    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 71.33% |
| Attack success rate:          | 20.45% |
| Average perturbed word %:     | 9.75%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 75.32  |
+-------------------------------+--------+


In [117]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [44:01<3:40:09, 52.84s/it]textattack: Saving checkpoint under "checkpoints\1722796221574.ta.chkpt" at 2024-08-05 02:30:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 91 / 4 / 100:  33%|███████              | 100/300 [49:49<1:39:39, 29.90s/it]textattack: Saving checkpoint under "checkpoints\1722796569374.ta.chkpt" at 2024-08-05 02:36:09 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 136 / 7 / 150:  50%|███████████           | 150/300 [55:37<55:37, 22.25s/it]textattack: Saving checkpoint under "checkpoints\1722796916848.ta.chkpt" at 2024-08-05 02:41:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 177 / 14 / 200:  67%|████████████▋      | 200/300 [1:01:01<30:30, 18.31s/it]textattack: Saving checkpoint under "checkpoints\1722797240939.ta.chkpt" at 2024-08-05 02:47:20 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 220 / 17 / 250:  83%|███████████████   | 250/300 [1:07:17<13:27, 16.15s/it]textattack: Saving checkpoint under "checkpoints\1722797617760.ta.chkpt" at 2024-08-05 02:53:37 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 264 / 22 / 300: 100%|██████████████████| 300/300 [1:12:54<00:00, 14.58s/it]textattack: Saving checkpoint under "checkpoints\1722797954653.ta.chkpt" at 2024-08-05 02:59:14 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 14 / 264 / 22 / 300: 100%|██████████████████| 300/300 [1:12:54<00:00, 14.58s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 264    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 88.0%  |
| Attack success rate:          | 5.04%  |
| Average perturbed word %:     | 3.75%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 80.86  |
+-------------------------------+--------+


In [118]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 46 / 0 / 50:  17%|████▏                    | 50/300 [05:46<28:53,  6.93s/it]textattack: Saving checkpoint under "checkpoints\1722798301574.ta.chkpt" at 2024-08-05 03:05:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 86 / 3 / 100:  33%|███████▎              | 100/300 [11:31<23:02,  6.91s/it]textattack: Saving checkpoint under "checkpoints\1722798646127.ta.chkpt" at 2024-08-05 03:10:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 127 / 6 / 150:  50%|██████████▌          | 150/300 [17:07<17:07,  6.85s/it]textattack: Saving checkpoint under "checkpoints\1722798982691.ta.chkpt" at 2024-08-05 03:16:22 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 164 / 13 / 200:  67%|█████████████▎      | 200/300 [22:08<11:04,  6.64s/it]textattack: Saving checkpoint under "checkpoints\1722799283488.ta.chkpt" at 2024-08-05 03:21:23 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 206 / 16 / 250:  83%|████████████████▋   | 250/300 [28:38<05:43,  6.87s/it]textattack: Saving checkpoint under "checkpoints\1722799673286.ta.chkpt" at 2024-08-05 03:27:53 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 29 / 249 / 22 / 300: 100%|████████████████████| 300/300 [34:10<00:00,  6.84s/it]textattack: Saving checkpoint under "checkpoints\1722800005817.ta.chkpt" at 2024-08-05 03:33:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 29 / 249 / 22 / 300: 100%|████████████████████| 300/300 [34:10<00:00,  6.84s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 29     |
| Number of failed attacks:     | 249    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 83.0%  |
| Attack success rate:          | 10.43% |
| Average perturbed word %:     | 6.82%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 79.35  |
+-------------------------------+--------+


In [119]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, 0.5, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175ep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 43 / 1 / 50:  17%|████▏                    | 50/300 [05:39<28:16,  6.79s/it]textattack: Saving checkpoint under "checkpoints\1722800345272.ta.chkpt" at 2024-08-05 03:39:05 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 83 / 5 / 100:  33%|███████▎              | 100/300 [11:11<22:22,  6.71s/it]textattack: Saving checkpoint under "checkpoints\1722800677492.ta.chkpt" at 2024-08-05 03:44:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 121 / 9 / 150:  50%|██████████▌          | 150/300 [16:36<16:36,  6.64s/it]textattack: Saving checkpoint under "checkpoints\1722801002091.ta.chkpt" at 2024-08-05 03:50:02 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 155 / 21 / 200:  67%|█████████████▎      | 200/300 [21:06<10:33,  6.33s/it]textattack: Saving checkpoint under "checkpoints\1722801273001.ta.chkpt" at 2024-08-05 03:54:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 192 / 28 / 250:  83%|████████████████▋   | 250/300 [26:31<05:18,  6.37s/it]textattack: Saving checkpoint under "checkpoints\1722801597634.ta.chkpt" at 2024-08-05 03:59:57 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 233 / 34 / 300: 100%|████████████████████| 300/300 [32:03<00:00,  6.41s/it]textattack: Saving checkpoint under "checkpoints\1722801929993.ta.chkpt" at 2024-08-05 04:05:29 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 33 / 233 / 34 / 300: 100%|████████████████████| 300/300 [32:04<00:00,  6.41s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 33     |
| Number of failed attacks:     | 233    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 77.67% |
| Attack success rate:          | 12.41% |
| Average perturbed word %:     | 14.78% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 78.49  |
+-------------------------------+--------+


In [120]:
et5 = datetime.datetime.now()
et5 - et4

datetime.timedelta(seconds=32851, microseconds=878007)

### MinSim 0.6, threshold 0.5

In [121]:
suffix = 'sim06.csv'
thre = 1-0.5/np.pi
minsim = 0.6

In [122]:
st = datetime.datetime.now()

#### BERT

In [123]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 11 / 34 / 5 / 50:  17%|████                    | 50/300 [04:23<21:57,  5.27s/it]textattack: Saving checkpoint under "checkpoints\1722802194788.ta.chkpt" at 2024-08-05 04:09:54 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 58 / 10 / 100:  33%|███████              | 100/300 [08:22<16:45,  5.03s/it]textattack: Saving checkpoint under "checkpoints\1722802433943.ta.chkpt" at 2024-08-05 04:13:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 84 / 15 / 150:  50%|██████████▌          | 150/300 [12:36<12:36,  5.04s/it]textattack: Saving checkpoint under "checkpoints\1722802687661.ta.chkpt" at 2024-08-05 04:18:07 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 68 / 111 / 21 / 200:  67%|█████████████▎      | 200/300 [17:09<08:34,  5.15s/it]textattack: Saving checkpoint under "checkpoints\1722802960664.ta.chkpt" at 2024-08-05 04:22:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 85 / 140 / 25 / 250:  83%|████████████████▋   | 250/300 [22:14<04:26,  5.34s/it]textattack: Saving checkpoint under "checkpoints\1722803265554.ta.chkpt" at 2024-08-05 04:27:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 94 / 176 / 30 / 300: 100%|████████████████████| 300/300 [27:15<00:00,  5.45s/it]textattack: Saving checkpoint under "checkpoints\1722803567226.ta.chkpt" at 2024-08-05 04:32:47 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 94 / 176 / 30 / 300: 100%|████████████████████| 300/300 [27:15<00:00,  5.45s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 94     |
| Number of failed attacks:     | 176    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 58.67% |
| Attack success rate:          | 34.81% |
| Average perturbed word %:     | 14.61% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 62.29  |
+-------------------------------+--------+


In [124]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 12 / 33 / 5 / 50:  17%|████                    | 50/300 [04:29<22:25,  5.38s/it]textattack: Saving checkpoint under "checkpoints\1722803837325.ta.chkpt" at 2024-08-05 04:37:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 57 / 11 / 100:  33%|███████              | 100/300 [08:27<16:55,  5.08s/it]textattack: Saving checkpoint under "checkpoints\1722804075997.ta.chkpt" at 2024-08-05 04:41:15 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 83 / 16 / 150:  50%|██████████▌          | 150/300 [12:33<12:33,  5.02s/it]textattack: Saving checkpoint under "checkpoints\1722804321313.ta.chkpt" at 2024-08-05 04:45:21 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 68 / 109 / 23 / 200:  67%|█████████████▎      | 200/300 [16:54<08:27,  5.07s/it]textattack: Saving checkpoint under "checkpoints\1722804582368.ta.chkpt" at 2024-08-05 04:49:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 84 / 139 / 27 / 250:  83%|████████████████▋   | 250/300 [21:25<04:17,  5.14s/it]textattack: Saving checkpoint under "checkpoints\1722804854105.ta.chkpt" at 2024-08-05 04:54:14 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 95 / 173 / 32 / 300: 100%|████████████████████| 300/300 [26:09<00:00,  5.23s/it]textattack: Saving checkpoint under "checkpoints\1722805137905.ta.chkpt" at 2024-08-05 04:58:57 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 95 / 173 / 32 / 300: 100%|████████████████████| 300/300 [26:09<00:00,  5.23s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 95     |
| Number of failed attacks:     | 173    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 57.67% |
| Attack success rate:          | 35.45% |
| Average perturbed word %:     | 14.88% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 62.2   |
+-------------------------------+--------+


In [125]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 11 / 34 / 5 / 50:  17%|████                    | 50/300 [04:03<20:16,  4.87s/it]textattack: Saving checkpoint under "checkpoints\1722805381409.ta.chkpt" at 2024-08-05 05:03:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 57 / 11 / 100:  33%|███████              | 100/300 [07:56<15:53,  4.77s/it]textattack: Saving checkpoint under "checkpoints\1722805615033.ta.chkpt" at 2024-08-05 05:06:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 83 / 16 / 150:  50%|██████████▌          | 150/300 [11:59<11:59,  4.80s/it]textattack: Saving checkpoint under "checkpoints\1722805857389.ta.chkpt" at 2024-08-05 05:10:57 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 110 / 21 / 200:  67%|█████████████▎      | 200/300 [16:18<08:09,  4.89s/it]textattack: Saving checkpoint under "checkpoints\1722806116453.ta.chkpt" at 2024-08-05 05:15:16 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 138 / 25 / 250:  83%|████████████████▋   | 250/300 [20:48<04:09,  4.99s/it]textattack: Saving checkpoint under "checkpoints\1722806386647.ta.chkpt" at 2024-08-05 05:19:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 96 / 174 / 30 / 300: 100%|████████████████████| 300/300 [25:28<00:00,  5.10s/it]textattack: Saving checkpoint under "checkpoints\1722806666707.ta.chkpt" at 2024-08-05 05:24:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 96 / 174 / 30 / 300: 100%|████████████████████| 300/300 [25:28<00:00,  5.10s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 96     |
| Number of failed attacks:     | 174    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 58.0%  |
| Attack success rate:          | 35.56% |
| Average perturbed word %:     | 14.57% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 62.08  |
+-------------------------------+--------+


In [126]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 24 / 10 / 16 / 50:  17%|███▊                   | 50/300 [02:22<11:52,  2.85s/it]textattack: Saving checkpoint under "checkpoints\1722806809459.ta.chkpt" at 2024-08-05 05:26:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 24 / 31 / 100:  33%|███████              | 100/300 [05:03<10:06,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722806970250.ta.chkpt" at 2024-08-05 05:29:30 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 45 / 24 / 32 / 101:  34%|███████              | 101/300 [05:03<09:57,  3.00s/it]

[Succeeded / Failed / Skipped / Total] 73 / 37 / 40 / 150:  50%|██████████▌          | 150/300 [08:08<08:08,  3.26s/it]textattack: Saving checkpoint under "checkpoints\1722807155881.ta.chkpt" at 2024-08-05 05:32:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 91 / 54 / 55 / 200:  67%|██████████████       | 200/300 [11:18<05:39,  3.39s/it]textattack: Saving checkpoint under "checkpoints\1722807345917.ta.chkpt" at 2024-08-05 05:35:45 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 118 / 66 / 66 / 250:  83%|████████████████▋   | 250/300 [14:04<02:48,  3.38s/it]textattack: Saving checkpoint under "checkpoints\1722807511141.ta.chkpt" at 2024-08-05 05:38:31 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 142 / 80 / 78 / 300: 100%|████████████████████| 300/300 [17:15<00:00,  3.45s/it]textattack: Saving checkpoint under "checkpoints\1722807702582.ta.chkpt" at 2024-08-05 05:41:42 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 142 / 80 / 78 / 300: 100%|████████████████████| 300/300 [17:15<00:00,  3.45s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 142    |
| Number of failed attacks:     | 80     |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 26.67% |
| Attack success rate:          | 63.96% |
| Average perturbed word %:     | 10.82% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 52.28  |
+-------------------------------+--------+


In [127]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 22 / 19 / 9 / 50:  17%|████                    | 50/300 [03:29<17:29,  4.20s/it]textattack: Saving checkpoint under "checkpoints\1722807912839.ta.chkpt" at 2024-08-05 05:45:12 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 40 / 20 / 100:  33%|███████              | 100/300 [06:46<13:33,  4.07s/it]textattack: Saving checkpoint under "checkpoints\1722808109563.ta.chkpt" at 2024-08-05 05:48:29 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 61 / 29 / 150:  50%|██████████▌          | 150/300 [10:20<10:20,  4.13s/it]textattack: Saving checkpoint under "checkpoints\1722808323112.ta.chkpt" at 2024-08-05 05:52:03 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 70 / 85 / 45 / 200:  67%|██████████████       | 200/300 [13:56<06:58,  4.18s/it]textattack: Saving checkpoint under "checkpoints\1722808539130.ta.chkpt" at 2024-08-05 05:55:39 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 92 / 107 / 51 / 250:  83%|████████████████▋   | 250/300 [17:59<03:35,  4.32s/it]textattack: Saving checkpoint under "checkpoints\1722808782673.ta.chkpt" at 2024-08-05 05:59:42 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 108 / 134 / 58 / 300: 100%|███████████████████| 300/300 [22:02<00:00,  4.41s/it]textattack: Saving checkpoint under "checkpoints\1722809025602.ta.chkpt" at 2024-08-05 06:03:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 108 / 134 / 58 / 300: 100%|███████████████████| 300/300 [22:02<00:00,  4.41s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 108    |
| Number of failed attacks:     | 134    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 44.67% |
| Attack success rate:          | 44.63% |
| Average perturbed word %:     | 10.08% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.74  |
+-------------------------------+--------+

In [128]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 20 / 21 / 9 / 50:  17%|████                    | 50/300 [03:41<18:25,  4.42s/it]textattack: Saving checkpoint under "checkpoints\1722809247143.ta.chkpt" at 2024-08-05 06:07:27 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 41 / 18 / 100:  33%|███████              | 100/300 [06:56<13:52,  4.16s/it]textattack: Saving checkpoint under "checkpoints\1722809442038.ta.chkpt" at 2024-08-05 06:10:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 60 / 63 / 27 / 150:  50%|██████████▌          | 150/300 [10:27<10:27,  4.18s/it]textattack: Saving checkpoint under "checkpoints\1722809653352.ta.chkpt" at 2024-08-05 06:14:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 89 / 42 / 200:  67%|██████████████       | 200/300 [14:01<07:00,  4.21s/it]textattack: Saving checkpoint under "checkpoints\1722809867850.ta.chkpt" at 2024-08-05 06:17:47 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 90 / 112 / 48 / 250:  83%|████████████████▋   | 250/300 [18:03<03:36,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1722810109848.ta.chkpt" at 2024-08-05 06:21:49 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 103 / 142 / 55 / 300: 100%|███████████████████| 300/300 [22:17<00:00,  4.46s/it]textattack: Saving checkpoint under "checkpoints\1722810363928.ta.chkpt" at 2024-08-05 06:26:03 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 103 / 142 / 55 / 300: 100%|███████████████████| 300/300 [22:17<00:00,  4.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 103    |
| Number of failed attacks:     | 142    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 47.33% |
| Attack success rate:          | 42.04% |
| Average perturbed word %:     | 8.51%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.49  |
+-------------------------------+--------+


In [129]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 21 / 18 / 11 / 50:  17%|███▌                 | 50/300 [29:34<2:27:54, 35.50s/it]textattack: Saving checkpoint under "checkpoints\1722812138881.ta.chkpt" at 2024-08-05 06:55:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 35 / 20 / 100:  33%|██████▎            | 100/300 [34:04<1:08:08, 20.44s/it]textattack: Saving checkpoint under "checkpoints\1722812408149.ta.chkpt" at 2024-08-05 07:00:08 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 59 / 59 / 32 / 150:  50%|██████████▌          | 150/300 [38:41<38:41, 15.48s/it]textattack: Saving checkpoint under "checkpoints\1722812685462.ta.chkpt" at 2024-08-05 07:04:45 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 76 / 79 / 45 / 200:  67%|██████████████       | 200/300 [43:38<21:49, 13.09s/it]textattack: Saving checkpoint under "checkpoints\1722812982306.ta.chkpt" at 2024-08-05 07:09:42 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 105 / 52 / 250:  83%|████████████████▋   | 250/300 [49:06<09:49, 11.79s/it]textattack: Saving checkpoint under "checkpoints\1722813310811.ta.chkpt" at 2024-08-05 07:15:10 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 112 / 130 / 58 / 300: 100%|███████████████████| 300/300 [54:39<00:00, 10.93s/it]textattack: Saving checkpoint under "checkpoints\1722813643924.ta.chkpt" at 2024-08-05 07:20:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 112 / 130 / 58 / 300: 100%|███████████████████| 300/300 [54:39<00:00, 10.93s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 112    |
| Number of failed attacks:     | 130    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 43.33% |
| Attack success rate:          | 46.28% |
| Average perturbed word %:     | 4.58%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 55.39  |
+-------------------------------+--------+


In [130]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 19 / 22 / 9 / 50:  17%|████                    | 50/300 [04:47<23:58,  5.76s/it]textattack: Saving checkpoint under "checkpoints\1722813931938.ta.chkpt" at 2024-08-05 07:25:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 42 / 19 / 100:  33%|███████              | 100/300 [09:34<19:09,  5.75s/it]textattack: Saving checkpoint under "checkpoints\1722814218891.ta.chkpt" at 2024-08-05 07:30:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 61 / 28 / 150:  50%|██████████▌          | 150/300 [14:29<14:29,  5.80s/it]textattack: Saving checkpoint under "checkpoints\1722814513869.ta.chkpt" at 2024-08-05 07:35:13 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 82 / 43 / 200:  67%|██████████████       | 200/300 [19:48<09:54,  5.94s/it]textattack: Saving checkpoint under "checkpoints\1722814832280.ta.chkpt" at 2024-08-05 07:40:32 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 94 / 105 / 51 / 250:  83%|████████████████▋   | 250/300 [26:07<05:13,  6.27s/it]textattack: Saving checkpoint under "checkpoints\1722815211738.ta.chkpt" at 2024-08-05 07:46:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 114 / 130 / 56 / 300: 100%|███████████████████| 300/300 [31:55<00:00,  6.38s/it]textattack: Saving checkpoint under "checkpoints\1722815559288.ta.chkpt" at 2024-08-05 07:52:39 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 114 / 130 / 56 / 300: 100%|███████████████████| 300/300 [31:55<00:00,  6.38s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 114    |
| Number of failed attacks:     | 130    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 43.33% |
| Attack success rate:          | 46.72% |
| Average perturbed word %:     | 8.73%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.19  |
+-------------------------------+--------+


In [131]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 26 / 15 / 9 / 50:  17%|████                    | 50/300 [04:18<21:32,  5.17s/it]textattack: Saving checkpoint under "checkpoints\1722815818451.ta.chkpt" at 2024-08-05 07:56:58 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 47 / 33 / 20 / 100:  33%|███████              | 100/300 [08:49<17:38,  5.29s/it]textattack: Saving checkpoint under "checkpoints\1722816089094.ta.chkpt" at 2024-08-05 08:01:29 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 50 / 33 / 150:  50%|██████████▌          | 150/300 [12:57<12:57,  5.18s/it]textattack: Saving checkpoint under "checkpoints\1722816337059.ta.chkpt" at 2024-08-05 08:05:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 91 / 62 / 47 / 200:  67%|██████████████       | 200/300 [17:43<08:51,  5.32s/it]textattack: Saving checkpoint under "checkpoints\1722816623414.ta.chkpt" at 2024-08-05 08:10:23 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 115 / 81 / 54 / 250:  83%|████████████████▋   | 250/300 [23:24<04:40,  5.62s/it]textattack: Saving checkpoint under "checkpoints\1722816964134.ta.chkpt" at 2024-08-05 08:16:04 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 139 / 101 / 60 / 300: 100%|███████████████████| 300/300 [28:19<00:00,  5.67s/it]textattack: Saving checkpoint under "checkpoints\1722817259643.ta.chkpt" at 2024-08-05 08:20:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 139 / 101 / 60 / 300: 100%|███████████████████| 300/300 [28:19<00:00,  5.67s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 139    |
| Number of failed attacks:     | 101    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 33.67% |
| Attack success rate:          | 57.92% |
| Average perturbed word %:     | 10.29% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 53.79  |
+-------------------------------+--------+


#### CLIP

In [132]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:48<19:02,  4.57s/it]textattack: Saving checkpoint under "checkpoints\1722817488382.ta.chkpt" at 2024-08-05 08:24:48 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 80 / 9 / 100:  33%|███████▎              | 100/300 [07:13<14:27,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1722817693740.ta.chkpt" at 2024-08-05 08:28:13 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 115 / 14 / 150:  50%|██████████          | 150/300 [10:37<10:37,  4.25s/it]textattack: Saving checkpoint under "checkpoints\1722817897416.ta.chkpt" at 2024-08-05 08:31:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 149 / 20 / 200:  67%|█████████████▎      | 200/300 [13:45<06:52,  4.13s/it]textattack: Saving checkpoint under "checkpoints\1722818084988.ta.chkpt" at 2024-08-05 08:34:44 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 185 / 24 / 250:  83%|████████████████▋   | 250/300 [17:17<03:27,  4.15s/it]textattack: Saving checkpoint under "checkpoints\1722818297428.ta.chkpt" at 2024-08-05 08:38:17 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 221 / 29 / 300: 100%|████████████████████| 300/300 [20:34<00:00,  4.12s/it]textattack: Saving checkpoint under "checkpoints\1722818494403.ta.chkpt" at 2024-08-05 08:41:34 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 50 / 221 / 29 / 300: 100%|████████████████████| 300/300 [20:34<00:00,  4.12s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 50     |
| Number of failed attacks:     | 221    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 73.67% |
| Attack success rate:          | 18.45% |
| Average perturbed word %:     | 14.24% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 64.68  |
+-------------------------------+--------+


In [133]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:11<15:59,  3.84s/it]textattack: Saving checkpoint under "checkpoints\1722818686472.ta.chkpt" at 2024-08-05 08:44:46 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 80 / 9 / 100:  33%|███████▎              | 100/300 [06:24<12:48,  3.84s/it]textattack: Saving checkpoint under "checkpoints\1722818878631.ta.chkpt" at 2024-08-05 08:47:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 116 / 14 / 150:  50%|██████████          | 150/300 [09:39<09:39,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1722819074253.ta.chkpt" at 2024-08-05 08:51:14 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 150 / 20 / 200:  67%|█████████████▎      | 200/300 [12:44<06:22,  3.82s/it]textattack: Saving checkpoint under "checkpoints\1722819258869.ta.chkpt" at 2024-08-05 08:54:18 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 186 / 24 / 250:  83%|████████████████▋   | 250/300 [16:10<03:14,  3.88s/it]textattack: Saving checkpoint under "checkpoints\1722819464881.ta.chkpt" at 2024-08-05 08:57:44 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 49 / 222 / 29 / 300: 100%|████████████████████| 300/300 [19:20<00:00,  3.87s/it]textattack: Saving checkpoint under "checkpoints\1722819655593.ta.chkpt" at 2024-08-05 09:00:55 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 49 / 222 / 29 / 300: 100%|████████████████████| 300/300 [19:21<00:00,  3.87s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 49     |
| Number of failed attacks:     | 222    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 74.0%  |
| Attack success rate:          | 18.08% |
| Average perturbed word %:     | 13.91% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 64.65  |
+-------------------------------+--------+


In [134]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 42 / 4 / 50:  17%|████▏                    | 50/300 [03:09<15:48,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1722819845478.ta.chkpt" at 2024-08-05 09:04:05 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 80 / 9 / 100:  33%|███████▎              | 100/300 [06:22<12:45,  3.83s/it]textattack: Saving checkpoint under "checkpoints\1722820038348.ta.chkpt" at 2024-08-05 09:07:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 115 / 14 / 150:  50%|██████████          | 150/300 [09:39<09:39,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1722820235362.ta.chkpt" at 2024-08-05 09:10:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 149 / 20 / 200:  67%|█████████████▎      | 200/300 [12:41<06:20,  3.81s/it]textattack: Saving checkpoint under "checkpoints\1722820417537.ta.chkpt" at 2024-08-05 09:13:37 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 185 / 24 / 250:  83%|████████████████▋   | 250/300 [16:06<03:13,  3.87s/it]textattack: Saving checkpoint under "checkpoints\1722820622742.ta.chkpt" at 2024-08-05 09:17:02 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 221 / 29 / 300: 100%|████████████████████| 300/300 [19:17<00:00,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1722820813264.ta.chkpt" at 2024-08-05 09:20:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 50 / 221 / 29 / 300: 100%|████████████████████| 300/300 [19:17<00:00,  3.86s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 50     |
| Number of failed attacks:     | 221    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 73.67% |
| Attack success rate:          | 18.45% |
| Average perturbed word %:     | 14.24% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 64.68  |
+-------------------------------+--------+


In [135]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 38 / 5 / 50:  17%|████▏                    | 50/300 [02:58<14:54,  3.58s/it]textattack: Saving checkpoint under "checkpoints\1722820992635.ta.chkpt" at 2024-08-05 09:23:12 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 71 / 13 / 100:  33%|███████              | 100/300 [05:59<11:58,  3.59s/it]textattack: Saving checkpoint under "checkpoints\1722821172998.ta.chkpt" at 2024-08-05 09:26:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 107 / 19 / 150:  50%|██████████          | 150/300 [09:10<09:10,  3.67s/it]textattack: Saving checkpoint under "checkpoints\1722821364807.ta.chkpt" at 2024-08-05 09:29:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 134 / 35 / 200:  67%|█████████████▎      | 200/300 [11:27<05:43,  3.44s/it]textattack: Saving checkpoint under "checkpoints\1722821501693.ta.chkpt" at 2024-08-05 09:31:41 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 166 / 44 / 250:  83%|████████████████▋   | 250/300 [14:23<02:52,  3.45s/it]textattack: Saving checkpoint under "checkpoints\1722821677125.ta.chkpt" at 2024-08-05 09:34:37 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 207 / 49 / 300: 100%|████████████████████| 300/300 [17:53<00:00,  3.58s/it]textattack: Saving checkpoint under "checkpoints\1722821887528.ta.chkpt" at 2024-08-05 09:38:07 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 44 / 207 / 49 / 300: 100%|████████████████████| 300/300 [17:53<00:00,  3.58s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 44     |
| Number of failed attacks:     | 207    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 69.0%  |
| Attack success rate:          | 17.53% |
| Average perturbed word %:     | 10.89% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 63.53  |
+-------------------------------+--------+


In [136]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 41 / 2 / 50:  17%|████▏                    | 50/300 [03:26<17:11,  4.13s/it]textattack: Saving checkpoint under "checkpoints\1722822094056.ta.chkpt" at 2024-08-05 09:41:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 80 / 7 / 100:  33%|███████▎              | 100/300 [06:53<13:47,  4.14s/it]textattack: Saving checkpoint under "checkpoints\1722822301389.ta.chkpt" at 2024-08-05 09:45:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 119 / 13 / 150:  50%|██████████          | 150/300 [10:09<10:09,  4.06s/it]textattack: Saving checkpoint under "checkpoints\1722822496965.ta.chkpt" at 2024-08-05 09:48:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 150 / 23 / 200:  67%|█████████████▎      | 200/300 [12:52<06:26,  3.86s/it]textattack: Saving checkpoint under "checkpoints\1722822660365.ta.chkpt" at 2024-08-05 09:51:00 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 185 / 29 / 250:  83%|████████████████▋   | 250/300 [16:20<03:16,  3.92s/it]textattack: Saving checkpoint under "checkpoints\1722822868608.ta.chkpt" at 2024-08-05 09:54:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 226 / 35 / 300: 100%|████████████████████| 300/300 [19:47<00:00,  3.96s/it]textattack: Saving checkpoint under "checkpoints\1722823075175.ta.chkpt" at 2024-08-05 09:57:55 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 39 / 226 / 35 / 300: 100%|████████████████████| 300/300 [19:47<00:00,  3.96s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 39     |
| Number of failed attacks:     | 226    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 75.33% |
| Attack success rate:          | 14.72% |
| Average perturbed word %:     | 11.0%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 65.53  |
+-------------------------------+--------+


In [137]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 41 / 2 / 50:  17%|████▏                    | 50/300 [04:06<20:31,  4.92s/it]textattack: Saving checkpoint under "checkpoints\1722823321561.ta.chkpt" at 2024-08-05 10:02:01 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 76 / 7 / 100:  33%|███████▎              | 100/300 [07:27<14:55,  4.48s/it]textattack: Saving checkpoint under "checkpoints\1722823523053.ta.chkpt" at 2024-08-05 10:05:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 115 / 10 / 150:  50%|██████████          | 150/300 [11:00<11:00,  4.40s/it]textattack: Saving checkpoint under "checkpoints\1722823735904.ta.chkpt" at 2024-08-05 10:08:55 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 147 / 20 / 200:  67%|█████████████▎      | 200/300 [13:53<06:56,  4.17s/it]textattack: Saving checkpoint under "checkpoints\1722823909326.ta.chkpt" at 2024-08-05 10:11:49 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 181 / 25 / 250:  83%|████████████████▋   | 250/300 [17:28<03:29,  4.19s/it]textattack: Saving checkpoint under "checkpoints\1722824123624.ta.chkpt" at 2024-08-05 10:15:23 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 48 / 221 / 31 / 300: 100%|████████████████████| 300/300 [20:51<00:00,  4.17s/it]textattack: Saving checkpoint under "checkpoints\1722824326691.ta.chkpt" at 2024-08-05 10:18:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 48 / 221 / 31 / 300: 100%|████████████████████| 300/300 [20:51<00:00,  4.17s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 48     |
| Number of failed attacks:     | 221    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 73.67% |
| Attack success rate:          | 17.84% |
| Average perturbed word %:     | 9.27%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 64.57  |
+-------------------------------+--------+


In [138]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 48 / 1 / 50:  17%|███▊                   | 50/300 [30:45<2:33:48, 36.92s/it]textattack: Saving checkpoint under "checkpoints\1722826172557.ta.chkpt" at 2024-08-05 10:49:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 90 / 4 / 100:  33%|███████              | 100/300 [35:59<1:11:58, 21.59s/it]textattack: Saving checkpoint under "checkpoints\1722826485867.ta.chkpt" at 2024-08-05 10:54:45 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 135 / 7 / 150:  50%|███████████           | 150/300 [41:06<41:06, 16.44s/it]textattack: Saving checkpoint under "checkpoints\1722826792836.ta.chkpt" at 2024-08-05 10:59:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 177 / 14 / 200:  67%|██████████████       | 200/300 [45:56<22:58, 13.78s/it]textattack: Saving checkpoint under "checkpoints\1722827083004.ta.chkpt" at 2024-08-05 11:04:43 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 221 / 17 / 250:  83%|████████████████▋   | 250/300 [51:30<10:18, 12.36s/it]textattack: Saving checkpoint under "checkpoints\1722827417736.ta.chkpt" at 2024-08-05 11:10:17 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 265 / 22 / 300: 100%|████████████████████| 300/300 [56:26<00:00, 11.29s/it]textattack: Saving checkpoint under "checkpoints\1722827713566.ta.chkpt" at 2024-08-05 11:15:13 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 265 / 22 / 300: 100%|████████████████████| 300/300 [56:26<00:00, 11.29s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 265    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 88.33% |
| Attack success rate:          | 4.68%  |
| Average perturbed word %:     | 3.81%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 68.0   |
+-------------------------------+--------+


In [139]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 46 / 0 / 50:  17%|████▏                    | 50/300 [05:05<25:28,  6.11s/it]textattack: Saving checkpoint under "checkpoints\1722828019829.ta.chkpt" at 2024-08-05 11:20:19 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 87 / 3 / 100:  33%|███████▎              | 100/300 [10:19<20:38,  6.19s/it]textattack: Saving checkpoint under "checkpoints\1722828333545.ta.chkpt" at 2024-08-05 11:25:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 129 / 6 / 150:  50%|██████████▌          | 150/300 [15:19<15:19,  6.13s/it]textattack: Saving checkpoint under "checkpoints\1722828633842.ta.chkpt" at 2024-08-05 11:30:33 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 167 / 13 / 200:  67%|█████████████▎      | 200/300 [19:49<09:54,  5.95s/it]textattack: Saving checkpoint under "checkpoints\1722828903444.ta.chkpt" at 2024-08-05 11:35:03 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 209 / 16 / 250:  83%|████████████████▋   | 250/300 [25:12<05:02,  6.05s/it]textattack: Saving checkpoint under "checkpoints\1722829226249.ta.chkpt" at 2024-08-05 11:40:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 252 / 22 / 300: 100%|████████████████████| 300/300 [30:04<00:00,  6.02s/it]textattack: Saving checkpoint under "checkpoints\1722829518907.ta.chkpt" at 2024-08-05 11:45:18 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 26 / 252 / 22 / 300: 100%|████████████████████| 300/300 [30:04<00:00,  6.02s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 26     |
| Number of failed attacks:     | 252    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 84.0%  |
| Attack success rate:          | 9.35%  |
| Average perturbed word %:     | 6.41%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 66.77  |
+-------------------------------+--------+


In [140]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175sim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 43 / 1 / 50:  17%|████▏                    | 50/300 [05:01<25:06,  6.03s/it]textattack: Saving checkpoint under "checkpoints\1722829820592.ta.chkpt" at 2024-08-05 11:50:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 82 / 5 / 100:  33%|███████▎              | 100/300 [10:04<20:09,  6.05s/it]textattack: Saving checkpoint under "checkpoints\1722830123742.ta.chkpt" at 2024-08-05 11:55:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 124 / 9 / 150:  50%|██████████▌          | 150/300 [14:59<14:59,  6.00s/it]textattack: Saving checkpoint under "checkpoints\1722830418709.ta.chkpt" at 2024-08-05 12:00:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 160 / 21 / 200:  67%|█████████████▎      | 200/300 [19:05<09:32,  5.73s/it]textattack: Saving checkpoint under "checkpoints\1722830664318.ta.chkpt" at 2024-08-05 12:04:24 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 199 / 28 / 250:  83%|████████████████▋   | 250/300 [24:00<04:48,  5.76s/it]textattack: Saving checkpoint under "checkpoints\1722830959629.ta.chkpt" at 2024-08-05 12:09:19 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 242 / 34 / 300: 100%|████████████████████| 300/300 [28:57<00:00,  5.79s/it]textattack: Saving checkpoint under "checkpoints\1722831256890.ta.chkpt" at 2024-08-05 12:14:16 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 24 / 242 / 34 / 300: 100%|████████████████████| 300/300 [28:57<00:00,  5.79s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 24     |
| Number of failed attacks:     | 242    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 80.67% |
| Attack success rate:          | 9.02%  |
| Average perturbed word %:     | 13.41% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 66.78  |
+-------------------------------+--------+


In [141]:
et6 = datetime.datetime.now()
et6 - st

datetime.timedelta(seconds=29325, microseconds=918829)

### MinSim 0.7, threshold 0.5

In [142]:
suffix = 'sim07.csv'
thre = 1-0.5/np.pi
minsim = 0.7

#### BERT

In [143]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 12 / 33 / 5 / 50:  17%|████                    | 50/300 [03:39<18:18,  4.40s/it]textattack: Saving checkpoint under "checkpoints\1722831478195.ta.chkpt" at 2024-08-05 12:17:58 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 60 / 10 / 100:  33%|███████              | 100/300 [07:10<14:21,  4.31s/it]textattack: Saving checkpoint under "checkpoints\1722831689173.ta.chkpt" at 2024-08-05 12:21:29 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 90 / 15 / 150:  50%|██████████▌          | 150/300 [11:06<11:06,  4.44s/it]textattack: Saving checkpoint under "checkpoints\1722831924457.ta.chkpt" at 2024-08-05 12:25:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 122 / 21 / 200:  67%|█████████████▎      | 200/300 [15:32<07:46,  4.66s/it]textattack: Saving checkpoint under "checkpoints\1722832191139.ta.chkpt" at 2024-08-05 12:29:51 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 68 / 157 / 25 / 250:  83%|████████████████▋   | 250/300 [19:49<03:57,  4.76s/it]textattack: Saving checkpoint under "checkpoints\1722832447943.ta.chkpt" at 2024-08-05 12:34:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 74 / 196 / 30 / 300: 100%|████████████████████| 300/300 [24:08<00:00,  4.83s/it]textattack: Saving checkpoint under "checkpoints\1722832706934.ta.chkpt" at 2024-08-05 12:38:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 74 / 196 / 30 / 300: 100%|████████████████████| 300/300 [24:08<00:00,  4.83s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 74     |
| Number of failed attacks:     | 196    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 65.33% |
| Attack success rate:          | 27.41% |
| Average perturbed word %:     | 12.39% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 55.01  |
+-------------------------------+--------+


In [144]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 10 / 35 / 5 / 50:  17%|████                    | 50/300 [03:47<18:55,  4.54s/it]textattack: Saving checkpoint under "checkpoints\1722832934980.ta.chkpt" at 2024-08-05 12:42:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 62 / 11 / 100:  33%|███████              | 100/300 [07:14<14:28,  4.34s/it]textattack: Saving checkpoint under "checkpoints\1722833142112.ta.chkpt" at 2024-08-05 12:45:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 93 / 16 / 150:  50%|██████████▌          | 150/300 [11:00<11:00,  4.40s/it]textattack: Saving checkpoint under "checkpoints\1722833368533.ta.chkpt" at 2024-08-05 12:49:28 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 122 / 23 / 200:  67%|█████████████▎      | 200/300 [14:47<07:23,  4.44s/it]textattack: Saving checkpoint under "checkpoints\1722833595262.ta.chkpt" at 2024-08-05 12:53:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 66 / 157 / 27 / 250:  83%|████████████████▋   | 250/300 [18:51<03:46,  4.52s/it]textattack: Saving checkpoint under "checkpoints\1722833838959.ta.chkpt" at 2024-08-05 12:57:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 72 / 196 / 32 / 300: 100%|████████████████████| 300/300 [23:08<00:00,  4.63s/it]textattack: Saving checkpoint under "checkpoints\1722834096304.ta.chkpt" at 2024-08-05 13:01:36 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 72 / 196 / 32 / 300: 100%|████████████████████| 300/300 [23:08<00:00,  4.63s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 72     |
| Number of failed attacks:     | 196    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 65.33% |
| Attack success rate:          | 26.87% |
| Average perturbed word %:     | 12.13% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 54.65  |
+-------------------------------+--------+


In [145]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 10 / 35 / 5 / 50:  17%|████                    | 50/300 [03:40<18:24,  4.42s/it]textattack: Saving checkpoint under "checkpoints\1722834317318.ta.chkpt" at 2024-08-05 13:05:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 62 / 11 / 100:  33%|███████              | 100/300 [07:04<14:08,  4.24s/it]textattack: Saving checkpoint under "checkpoints\1722834520599.ta.chkpt" at 2024-08-05 13:08:40 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 42 / 92 / 16 / 150:  50%|██████████▌          | 150/300 [11:04<11:04,  4.43s/it]textattack: Saving checkpoint under "checkpoints\1722834760849.ta.chkpt" at 2024-08-05 13:12:40 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 57 / 122 / 21 / 200:  67%|█████████████▎      | 200/300 [15:04<07:32,  4.52s/it]textattack: Saving checkpoint under "checkpoints\1722835000986.ta.chkpt" at 2024-08-05 13:16:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 158 / 25 / 250:  83%|████████████████▋   | 250/300 [19:15<03:51,  4.62s/it]textattack: Saving checkpoint under "checkpoints\1722835251521.ta.chkpt" at 2024-08-05 13:20:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 197 / 30 / 300: 100%|████████████████████| 300/300 [23:28<00:00,  4.69s/it]textattack: Saving checkpoint under "checkpoints\1722835504934.ta.chkpt" at 2024-08-05 13:25:04 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 73 / 197 / 30 / 300: 100%|████████████████████| 300/300 [23:28<00:00,  4.70s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 73     |
| Number of failed attacks:     | 197    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 65.67% |
| Attack success rate:          | 27.04% |
| Average perturbed word %:     | 12.38% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 55.12  |
+-------------------------------+--------+

In [146]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 20 / 14 / 16 / 50:  17%|███▊                   | 50/300 [02:15<11:17,  2.71s/it]textattack: Saving checkpoint under "checkpoints\1722835640643.ta.chkpt" at 2024-08-05 13:27:20 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 33 / 31 / 100:  33%|███████              | 100/300 [04:51<09:43,  2.92s/it]textattack: Saving checkpoint under "checkpoints\1722835797140.ta.chkpt" at 2024-08-05 13:29:57 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 36 / 33 / 32 / 101:  34%|███████              | 101/300 [04:52<09:35,  2.89s/it]

[Succeeded / Failed / Skipped / Total] 60 / 50 / 40 / 150:  50%|██████████▌          | 150/300 [07:47<07:47,  3.11s/it]textattack: Saving checkpoint under "checkpoints\1722835972356.ta.chkpt" at 2024-08-05 13:32:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 76 / 69 / 55 / 200:  67%|██████████████       | 200/300 [10:49<05:24,  3.25s/it]textattack: Saving checkpoint under "checkpoints\1722836155182.ta.chkpt" at 2024-08-05 13:35:55 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 100 / 84 / 66 / 250:  83%|████████████████▋   | 250/300 [13:33<02:42,  3.25s/it]textattack: Saving checkpoint under "checkpoints\1722836318419.ta.chkpt" at 2024-08-05 13:38:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 120 / 102 / 78 / 300: 100%|███████████████████| 300/300 [16:26<00:00,  3.29s/it]textattack: Saving checkpoint under "checkpoints\1722836491769.ta.chkpt" at 2024-08-05 13:41:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 120 / 102 / 78 / 300: 100%|███████████████████| 300/300 [16:26<00:00,  3.29s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 120    |
| Number of failed attacks:     | 102    |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 34.0%  |
| Attack success rate:          | 54.05% |
| Average perturbed word %:     | 9.46%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 48.55  |
+-------------------------------+--------+


In [147]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 21 / 20 / 9 / 50:  17%|████                    | 50/300 [02:57<14:49,  3.56s/it]textattack: Saving checkpoint under "checkpoints\1722836669932.ta.chkpt" at 2024-08-05 13:44:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 43 / 20 / 100:  33%|███████              | 100/300 [05:56<11:53,  3.57s/it]textattack: Saving checkpoint under "checkpoints\1722836848630.ta.chkpt" at 2024-08-05 13:47:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 56 / 65 / 29 / 150:  50%|██████████▌          | 150/300 [09:06<09:06,  3.64s/it]textattack: Saving checkpoint under "checkpoints\1722837038141.ta.chkpt" at 2024-08-05 13:50:38 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 88 / 45 / 200:  67%|██████████████       | 200/300 [12:18<06:09,  3.69s/it]textattack: Saving checkpoint under "checkpoints\1722837230092.ta.chkpt" at 2024-08-05 13:53:50 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 85 / 114 / 51 / 250:  83%|████████████████▋   | 250/300 [15:57<03:11,  3.83s/it]textattack: Saving checkpoint under "checkpoints\1722837449434.ta.chkpt" at 2024-08-05 13:57:29 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 95 / 147 / 58 / 300: 100%|████████████████████| 300/300 [19:46<00:00,  3.95s/it]textattack: Saving checkpoint under "checkpoints\1722837678150.ta.chkpt" at 2024-08-05 14:01:18 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 95 / 147 / 58 / 300: 100%|████████████████████| 300/300 [19:46<00:00,  3.95s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 95     |
| Number of failed attacks:     | 147    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 49.0%  |
| Attack success rate:          | 39.26% |
| Average perturbed word %:     | 9.65%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 51.69  |
+-------------------------------+--------+


In [148]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 18 / 23 / 9 / 50:  17%|████                    | 50/300 [03:15<16:17,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1722837874098.ta.chkpt" at 2024-08-05 14:04:34 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 47 / 18 / 100:  33%|███████              | 100/300 [06:19<12:38,  3.79s/it]textattack: Saving checkpoint under "checkpoints\1722838057791.ta.chkpt" at 2024-08-05 14:07:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 48 / 75 / 27 / 150:  50%|██████████▌          | 150/300 [09:29<09:29,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1722838248304.ta.chkpt" at 2024-08-05 14:10:48 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 58 / 100 / 42 / 200:  67%|█████████████▎      | 200/300 [12:48<06:24,  3.84s/it]textattack: Saving checkpoint under "checkpoints\1722838447395.ta.chkpt" at 2024-08-05 14:14:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 77 / 125 / 48 / 250:  83%|████████████████▋   | 250/300 [16:27<03:17,  3.95s/it]textattack: Saving checkpoint under "checkpoints\1722838666240.ta.chkpt" at 2024-08-05 14:17:46 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 90 / 155 / 55 / 300: 100%|████████████████████| 300/300 [20:14<00:00,  4.05s/it]textattack: Saving checkpoint under "checkpoints\1722838892873.ta.chkpt" at 2024-08-05 14:21:32 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 90 / 155 / 55 / 300: 100%|████████████████████| 300/300 [20:14<00:00,  4.05s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 90     |
| Number of failed attacks:     | 155    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 51.67% |
| Attack success rate:          | 36.73% |
| Average perturbed word %:     | 7.8%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 51.08  |
+-------------------------------+--------+


In [149]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 18 / 21 / 11 / 50:  17%|███▌                 | 50/300 [30:35<2:32:59, 36.72s/it]textattack: Saving checkpoint under "checkpoints\1722840728885.ta.chkpt" at 2024-08-05 14:52:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 42 / 20 / 100:  33%|██████▎            | 100/300 [35:01<1:10:02, 21.01s/it]textattack: Saving checkpoint under "checkpoints\1722840994004.ta.chkpt" at 2024-08-05 14:56:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 52 / 66 / 32 / 150:  50%|██████████▌          | 150/300 [39:28<39:28, 15.79s/it]textattack: Saving checkpoint under "checkpoints\1722841261105.ta.chkpt" at 2024-08-05 15:01:01 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 92 / 45 / 200:  67%|██████████████       | 200/300 [44:21<22:10, 13.31s/it]textattack: Saving checkpoint under "checkpoints\1722841554246.ta.chkpt" at 2024-08-05 15:05:54 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 77 / 121 / 52 / 250:  83%|████████████████▋   | 250/300 [49:17<09:51, 11.83s/it]textattack: Saving checkpoint under "checkpoints\1722841850248.ta.chkpt" at 2024-08-05 15:10:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 97 / 145 / 58 / 300: 100%|████████████████████| 300/300 [54:04<00:00, 10.82s/it]textattack: Saving checkpoint under "checkpoints\1722842137574.ta.chkpt" at 2024-08-05 15:15:37 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 97 / 145 / 58 / 300: 100%|████████████████████| 300/300 [54:04<00:00, 10.82s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 97     |
| Number of failed attacks:     | 145    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 48.33% |
| Attack success rate:          | 40.08% |
| Average perturbed word %:     | 4.37%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 50.41  |
+-------------------------------+--------+

In [150]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 19 / 22 / 9 / 50:  17%|████                    | 50/300 [04:30<22:30,  5.40s/it]textattack: Saving checkpoint under "checkpoints\1722842408027.ta.chkpt" at 2024-08-05 15:20:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 42 / 19 / 100:  33%|███████              | 100/300 [08:50<17:41,  5.31s/it]textattack: Saving checkpoint under "checkpoints\1722842668749.ta.chkpt" at 2024-08-05 15:24:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 68 / 28 / 150:  50%|██████████▌          | 150/300 [13:28<13:28,  5.39s/it]textattack: Saving checkpoint under "checkpoints\1722842946424.ta.chkpt" at 2024-08-05 15:29:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 94 / 43 / 200:  67%|██████████████       | 200/300 [18:09<09:04,  5.45s/it]textattack: Saving checkpoint under "checkpoints\1722843227480.ta.chkpt" at 2024-08-05 15:33:47 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 124 / 51 / 250:  83%|████████████████▋   | 250/300 [23:25<04:41,  5.62s/it]textattack: Saving checkpoint under "checkpoints\1722843543458.ta.chkpt" at 2024-08-05 15:39:03 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 93 / 151 / 56 / 300: 100%|████████████████████| 300/300 [28:28<00:00,  5.70s/it]textattack: Saving checkpoint under "checkpoints\1722843846825.ta.chkpt" at 2024-08-05 15:44:06 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 93 / 151 / 56 / 300: 100%|████████████████████| 300/300 [28:28<00:00,  5.70s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 93     |
| Number of failed attacks:     | 151    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 50.33% |
| Attack success rate:          | 38.11% |
| Average perturbed word %:     | 7.56%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 51.47  |
+-------------------------------+--------+

In [151]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 25 / 16 / 9 / 50:  17%|████                    | 50/300 [03:46<18:51,  4.53s/it]textattack: Saving checkpoint under "checkpoints\1722844073466.ta.chkpt" at 2024-08-05 15:47:53 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 37 / 20 / 100:  33%|███████              | 100/300 [07:48<15:36,  4.68s/it]textattack: Saving checkpoint under "checkpoints\1722844315634.ta.chkpt" at 2024-08-05 15:51:55 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 59 / 58 / 33 / 150:  50%|██████████▌          | 150/300 [11:39<11:39,  4.67s/it]textattack: Saving checkpoint under "checkpoints\1722844547095.ta.chkpt" at 2024-08-05 15:55:47 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 75 / 78 / 47 / 200:  67%|██████████████       | 200/300 [16:08<08:04,  4.84s/it]textattack: Saving checkpoint under "checkpoints\1722844815947.ta.chkpt" at 2024-08-05 16:00:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 95 / 101 / 54 / 250:  83%|████████████████▋   | 250/300 [21:11<04:14,  5.09s/it]textattack: Saving checkpoint under "checkpoints\1722845119029.ta.chkpt" at 2024-08-05 16:05:19 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 115 / 125 / 60 / 300: 100%|███████████████████| 300/300 [25:46<00:00,  5.16s/it]textattack: Saving checkpoint under "checkpoints\1722845394109.ta.chkpt" at 2024-08-05 16:09:54 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 115 / 125 / 60 / 300: 100%|███████████████████| 300/300 [25:46<00:00,  5.16s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 115    |
| Number of failed attacks:     | 125    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 41.67% |
| Attack success rate:          | 47.92% |
| Average perturbed word %:     | 8.87%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 48.85  |
+-------------------------------+--------+


#### CLIP

In [152]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 43 / 4 / 50:  17%|████▏                    | 50/300 [03:24<17:00,  4.08s/it]textattack: Saving checkpoint under "checkpoints\1722845598338.ta.chkpt" at 2024-08-05 16:13:18 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 82 / 9 / 100:  33%|███████▋               | 100/300 [06:27<12:54,  3.87s/it]textattack: Saving checkpoint under "checkpoints\1722845781688.ta.chkpt" at 2024-08-05 16:16:21 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 122 / 14 / 150:  50%|██████████          | 150/300 [09:32<09:32,  3.82s/it]textattack: Saving checkpoint under "checkpoints\1722845966737.ta.chkpt" at 2024-08-05 16:19:26 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 158 / 20 / 200:  67%|█████████████▎      | 200/300 [12:24<06:12,  3.72s/it]textattack: Saving checkpoint under "checkpoints\1722846139259.ta.chkpt" at 2024-08-05 16:22:19 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 196 / 24 / 250:  83%|████████████████▋   | 250/300 [15:32<03:06,  3.73s/it]textattack: Saving checkpoint under "checkpoints\1722846326672.ta.chkpt" at 2024-08-05 16:25:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 234 / 29 / 300: 100%|████████████████████| 300/300 [18:32<00:00,  3.71s/it]textattack: Saving checkpoint under "checkpoints\1722846506814.ta.chkpt" at 2024-08-05 16:28:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 37 / 234 / 29 / 300: 100%|████████████████████| 300/300 [18:32<00:00,  3.71s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 37     |
| Number of failed attacks:     | 234    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 78.0%  |
| Attack success rate:          | 13.65% |
| Average perturbed word %:     | 12.71% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 56.58  |
+-------------------------------+--------+


In [153]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 43 / 4 / 50:  17%|████▏                    | 50/300 [02:51<14:15,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1722846678192.ta.chkpt" at 2024-08-05 16:31:18 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 82 / 9 / 100:  33%|███████▋               | 100/300 [05:40<11:21,  3.41s/it]textattack: Saving checkpoint under "checkpoints\1722846847783.ta.chkpt" at 2024-08-05 16:34:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 122 / 14 / 150:  50%|██████████          | 150/300 [08:39<08:39,  3.46s/it]textattack: Saving checkpoint under "checkpoints\1722847026123.ta.chkpt" at 2024-08-05 16:37:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 158 / 20 / 200:  67%|█████████████▎      | 200/300 [11:27<05:43,  3.44s/it]textattack: Saving checkpoint under "checkpoints\1722847194283.ta.chkpt" at 2024-08-05 16:39:54 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 196 / 24 / 250:  83%|████████████████▋   | 250/300 [14:36<02:55,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1722847383271.ta.chkpt" at 2024-08-05 16:43:03 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 234 / 29 / 300: 100%|████████████████████| 300/300 [17:32<00:00,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1722847559411.ta.chkpt" at 2024-08-05 16:45:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 37 / 234 / 29 / 300: 100%|████████████████████| 300/300 [17:32<00:00,  3.51s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 37     |
| Number of failed attacks:     | 234    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 78.0%  |
| Attack success rate:          | 13.65% |
| Average perturbed word %:     | 12.91% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 56.63  |
+-------------------------------+--------+


In [154]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 43 / 4 / 50:  17%|████▏                    | 50/300 [02:52<14:22,  3.45s/it]textattack: Saving checkpoint under "checkpoints\1722847732156.ta.chkpt" at 2024-08-05 16:48:52 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 82 / 9 / 100:  33%|███████▋               | 100/300 [05:41<11:22,  3.41s/it]textattack: Saving checkpoint under "checkpoints\1722847901011.ta.chkpt" at 2024-08-05 16:51:41 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 122 / 14 / 150:  50%|██████████          | 150/300 [08:40<08:40,  3.47s/it]textattack: Saving checkpoint under "checkpoints\1722848079829.ta.chkpt" at 2024-08-05 16:54:39 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 158 / 20 / 200:  67%|█████████████▎      | 200/300 [11:26<05:43,  3.43s/it]textattack: Saving checkpoint under "checkpoints\1722848246284.ta.chkpt" at 2024-08-05 16:57:26 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 196 / 24 / 250:  83%|████████████████▋   | 250/300 [14:32<02:54,  3.49s/it]textattack: Saving checkpoint under "checkpoints\1722848432241.ta.chkpt" at 2024-08-05 17:00:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 37 / 234 / 29 / 300: 100%|████████████████████| 300/300 [17:26<00:00,  3.49s/it]textattack: Saving checkpoint under "checkpoints\1722848606461.ta.chkpt" at 2024-08-05 17:03:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 37 / 234 / 29 / 300: 100%|████████████████████| 300/300 [17:26<00:00,  3.49s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 37     |
| Number of failed attacks:     | 234    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 78.0%  |
| Attack success rate:          | 13.65% |
| Average perturbed word %:     | 12.71% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 56.58  |
+-------------------------------+--------+


In [155]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 40 / 5 / 50:  17%|████▏                    | 50/300 [02:42<13:30,  3.24s/it]textattack: Saving checkpoint under "checkpoints\1722848768839.ta.chkpt" at 2024-08-05 17:06:08 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 75 / 13 / 100:  33%|███████              | 100/300 [05:26<10:53,  3.27s/it]textattack: Saving checkpoint under "checkpoints\1722848933520.ta.chkpt" at 2024-08-05 17:08:53 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 112 / 19 / 150:  50%|██████████          | 150/300 [08:16<08:16,  3.31s/it]textattack: Saving checkpoint under "checkpoints\1722849103360.ta.chkpt" at 2024-08-05 17:11:43 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 140 / 35 / 200:  67%|█████████████▎      | 200/300 [10:21<05:10,  3.11s/it]textattack: Saving checkpoint under "checkpoints\1722849228401.ta.chkpt" at 2024-08-05 17:13:48 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 176 / 44 / 250:  83%|████████████████▋   | 250/300 [13:08<02:37,  3.16s/it]textattack: Saving checkpoint under "checkpoints\1722849395589.ta.chkpt" at 2024-08-05 17:16:35 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 218 / 49 / 300: 100%|████████████████████| 300/300 [16:12<00:00,  3.24s/it]textattack: Saving checkpoint under "checkpoints\1722849578759.ta.chkpt" at 2024-08-05 17:19:38 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 33 / 218 / 49 / 300: 100%|████████████████████| 300/300 [16:12<00:00,  3.24s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 33     |
| Number of failed attacks:     | 218    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 72.67% |
| Attack success rate:          | 13.15% |
| Average perturbed word %:     | 8.93%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 55.64  |
+-------------------------------+--------+


In [156]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 44 / 2 / 50:  17%|████▏                    | 50/300 [03:09<15:49,  3.80s/it]textattack: Saving checkpoint under "checkpoints\1722849768833.ta.chkpt" at 2024-08-05 17:22:48 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 83 / 7 / 100:  33%|███████▎              | 100/300 [06:10<12:20,  3.70s/it]textattack: Saving checkpoint under "checkpoints\1722849949442.ta.chkpt" at 2024-08-05 17:25:49 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 124 / 13 / 150:  50%|██████████          | 150/300 [09:10<09:10,  3.67s/it]textattack: Saving checkpoint under "checkpoints\1722850129259.ta.chkpt" at 2024-08-05 17:28:49 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 157 / 23 / 200:  67%|█████████████▎      | 200/300 [11:41<05:50,  3.51s/it]textattack: Saving checkpoint under "checkpoints\1722850280626.ta.chkpt" at 2024-08-05 17:31:20 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 195 / 29 / 250:  83%|████████████████▋   | 250/300 [14:46<02:57,  3.55s/it]textattack: Saving checkpoint under "checkpoints\1722850465925.ta.chkpt" at 2024-08-05 17:34:25 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 238 / 35 / 300: 100%|████████████████████| 300/300 [17:47<00:00,  3.56s/it]textattack: Saving checkpoint under "checkpoints\1722850646915.ta.chkpt" at 2024-08-05 17:37:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 27 / 238 / 35 / 300: 100%|████████████████████| 300/300 [17:47<00:00,  3.56s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 27     |
| Number of failed attacks:     | 238    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 79.33% |
| Attack success rate:          | 10.19% |
| Average perturbed word %:     | 8.47%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 56.63  |
+-------------------------------+--------+


In [157]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 44 / 2 / 50:  17%|████▏                    | 50/300 [03:55<19:39,  4.72s/it]textattack: Saving checkpoint under "checkpoints\1722850882926.ta.chkpt" at 2024-08-05 17:41:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 81 / 7 / 100:  33%|███████▎              | 100/300 [06:59<13:58,  4.19s/it]textattack: Saving checkpoint under "checkpoints\1722851066080.ta.chkpt" at 2024-08-05 17:44:26 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 122 / 10 / 150:  50%|██████████          | 150/300 [10:04<10:04,  4.03s/it]textattack: Saving checkpoint under "checkpoints\1722851251332.ta.chkpt" at 2024-08-05 17:47:31 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 152 / 20 / 200:  67%|█████████████▎      | 200/300 [12:34<06:17,  3.77s/it]textattack: Saving checkpoint under "checkpoints\1722851401461.ta.chkpt" at 2024-08-05 17:50:01 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 189 / 25 / 250:  83%|████████████████▋   | 250/300 [15:42<03:08,  3.77s/it]textattack: Saving checkpoint under "checkpoints\1722851589255.ta.chkpt" at 2024-08-05 17:53:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 230 / 31 / 300: 100%|████████████████████| 300/300 [18:43<00:00,  3.74s/it]textattack: Saving checkpoint under "checkpoints\1722851770289.ta.chkpt" at 2024-08-05 17:56:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 39 / 230 / 31 / 300: 100%|████████████████████| 300/300 [18:43<00:00,  3.74s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 39     |
| Number of failed attacks:     | 230    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 76.67% |
| Attack success rate:          | 14.5%  |
| Average perturbed word %:     | 9.44%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 56.12  |
+-------------------------------+--------+


In [158]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [32:14<2:41:10, 38.68s/it]textattack: Saving checkpoint under "checkpoints\1722853704399.ta.chkpt" at 2024-08-05 18:28:24 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 92 / 4 / 100:  33%|███████              | 100/300 [36:51<1:13:42, 22.11s/it]textattack: Saving checkpoint under "checkpoints\1722853981396.ta.chkpt" at 2024-08-05 18:33:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 138 / 7 / 150:  50%|███████████           | 150/300 [41:24<41:24, 16.56s/it]textattack: Saving checkpoint under "checkpoints\1722854254431.ta.chkpt" at 2024-08-05 18:37:34 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 181 / 14 / 200:  67%|██████████████       | 200/300 [45:50<22:55, 13.75s/it]textattack: Saving checkpoint under "checkpoints\1722854520879.ta.chkpt" at 2024-08-05 18:42:00 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 225 / 17 / 250:  83%|█████████████████▌   | 250/300 [50:39<10:07, 12.16s/it]textattack: Saving checkpoint under "checkpoints\1722854810299.ta.chkpt" at 2024-08-05 18:46:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 269 / 22 / 300: 100%|█████████████████████| 300/300 [55:15<00:00, 11.05s/it]textattack: Saving checkpoint under "checkpoints\1722855085389.ta.chkpt" at 2024-08-05 18:51:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 9 / 269 / 22 / 300: 100%|█████████████████████| 300/300 [55:15<00:00, 11.05s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 9      |
| Number of failed attacks:     | 269    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 89.67% |
| Attack success rate:          | 3.24%  |
| Average perturbed word %:     | 3.42%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.96  |
+-------------------------------+--------+


In [159]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 46 / 0 / 50:  17%|████▏                    | 50/300 [04:52<24:20,  5.84s/it]textattack: Saving checkpoint under "checkpoints\1722855377606.ta.chkpt" at 2024-08-05 18:56:17 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 88 / 3 / 100:  33%|███████▋               | 100/300 [09:34<19:09,  5.75s/it]textattack: Saving checkpoint under "checkpoints\1722855660472.ta.chkpt" at 2024-08-05 19:01:00 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 133 / 6 / 150:  50%|██████████▌          | 150/300 [14:11<14:11,  5.68s/it]textattack: Saving checkpoint under "checkpoints\1722855937269.ta.chkpt" at 2024-08-05 19:05:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 171 / 13 / 200:  67%|█████████████▎      | 200/300 [18:18<09:09,  5.49s/it]textattack: Saving checkpoint under "checkpoints\1722856184467.ta.chkpt" at 2024-08-05 19:09:44 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 21 / 213 / 16 / 250:  83%|████████████████▋   | 250/300 [23:01<04:36,  5.52s/it]textattack: Saving checkpoint under "checkpoints\1722856466577.ta.chkpt" at 2024-08-05 19:14:26 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 256 / 22 / 300: 100%|████████████████████| 300/300 [27:18<00:00,  5.46s/it]textattack: Saving checkpoint under "checkpoints\1722856723889.ta.chkpt" at 2024-08-05 19:18:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 22 / 256 / 22 / 300: 100%|████████████████████| 300/300 [27:18<00:00,  5.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 22     |
| Number of failed attacks:     | 256    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 85.33% |
| Attack success rate:          | 7.91%  |
| Average perturbed word %:     | 6.34%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.31  |
+-------------------------------+--------+


In [160]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175sim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 46 / 1 / 50:  17%|████▏                    | 50/300 [04:27<22:15,  5.34s/it]textattack: Saving checkpoint under "checkpoints\1722856991062.ta.chkpt" at 2024-08-05 19:23:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 85 / 5 / 100:  33%|███████▎              | 100/300 [08:55<17:50,  5.35s/it]textattack: Saving checkpoint under "checkpoints\1722857259502.ta.chkpt" at 2024-08-05 19:27:39 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 128 / 9 / 150:  50%|██████████▌          | 150/300 [13:21<13:21,  5.35s/it]textattack: Saving checkpoint under "checkpoints\1722857525809.ta.chkpt" at 2024-08-05 19:32:05 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 164 / 21 / 200:  67%|█████████████▎      | 200/300 [17:02<08:31,  5.11s/it]textattack: Saving checkpoint under "checkpoints\1722857746557.ta.chkpt" at 2024-08-05 19:35:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 204 / 28 / 250:  83%|████████████████▋   | 250/300 [21:23<04:16,  5.13s/it]textattack: Saving checkpoint under "checkpoints\1722858007158.ta.chkpt" at 2024-08-05 19:40:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 248 / 34 / 300: 100%|████████████████████| 300/300 [25:39<00:00,  5.13s/it]textattack: Saving checkpoint under "checkpoints\1722858263412.ta.chkpt" at 2024-08-05 19:44:23 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 18 / 248 / 34 / 300: 100%|████████████████████| 300/300 [25:39<00:00,  5.13s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 18     |
| Number of failed attacks:     | 248    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 82.67% |
| Attack success rate:          | 6.77%  |
| Average perturbed word %:     | 10.54% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.26  |
+-------------------------------+--------+


In [161]:
et7 = datetime.datetime.now()
et7 - et6

datetime.timedelta(seconds=27006, microseconds=640144)

### MinSim 0.8, threshold 0.5

In [162]:
suffix = 'sim08.csv'
thre = 1-0.5/np.pi
minsim = 0.8

#### BERT

In [163]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 40 / 5 / 50:  17%|████▏                    | 50/300 [03:34<17:50,  4.28s/it]textattack: Saving checkpoint under "checkpoints\1722858478349.ta.chkpt" at 2024-08-05 19:47:58 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 73 / 10 / 100:  33%|███████              | 100/300 [06:48<13:36,  4.08s/it]textattack: Saving checkpoint under "checkpoints\1722858672624.ta.chkpt" at 2024-08-05 19:51:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 111 / 15 / 150:  50%|██████████          | 150/300 [10:12<10:12,  4.08s/it]textattack: Saving checkpoint under "checkpoints\1722858876667.ta.chkpt" at 2024-08-05 19:54:36 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 147 / 21 / 200:  67%|█████████████▎      | 200/300 [13:45<06:52,  4.13s/it]textattack: Saving checkpoint under "checkpoints\1722859089539.ta.chkpt" at 2024-08-05 19:58:09 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 187 / 25 / 250:  83%|████████████████▋   | 250/300 [17:28<03:29,  4.19s/it]textattack: Saving checkpoint under "checkpoints\1722859312845.ta.chkpt" at 2024-08-05 20:01:52 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 230 / 30 / 300: 100%|████████████████████| 300/300 [21:21<00:00,  4.27s/it]textattack: Saving checkpoint under "checkpoints\1722859545381.ta.chkpt" at 2024-08-05 20:05:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 40 / 230 / 30 / 300: 100%|████████████████████| 300/300 [21:21<00:00,  4.27s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 40     |
| Number of failed attacks:     | 230    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 76.67% |
| Attack success rate:          | 14.81% |
| Average perturbed word %:     | 9.59%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 46.51  |
+-------------------------------+--------+


In [164]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 40 / 5 / 50:  17%|████▏                    | 50/300 [04:09<20:49,  5.00s/it]textattack: Saving checkpoint under "checkpoints\1722859795332.ta.chkpt" at 2024-08-05 20:09:55 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 72 / 11 / 100:  33%|███████              | 100/300 [07:46<15:32,  4.66s/it]textattack: Saving checkpoint under "checkpoints\1722860011931.ta.chkpt" at 2024-08-05 20:13:31 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 109 / 16 / 150:  50%|██████████          | 150/300 [11:46<11:46,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1722860252038.ta.chkpt" at 2024-08-05 20:17:32 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 144 / 23 / 200:  67%|█████████████▎      | 200/300 [15:43<07:51,  4.72s/it]textattack: Saving checkpoint under "checkpoints\1722860489306.ta.chkpt" at 2024-08-05 20:21:29 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 185 / 27 / 250:  83%|████████████████▋   | 250/300 [19:45<03:57,  4.74s/it]textattack: Saving checkpoint under "checkpoints\1722860730542.ta.chkpt" at 2024-08-05 20:25:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 225 / 32 / 300: 100%|████████████████████| 300/300 [23:44<00:00,  4.75s/it]textattack: Saving checkpoint under "checkpoints\1722860969729.ta.chkpt" at 2024-08-05 20:29:29 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 43 / 225 / 32 / 300: 100%|████████████████████| 300/300 [23:44<00:00,  4.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 43     |
| Number of failed attacks:     | 225    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 75.0%  |
| Attack success rate:          | 16.04% |
| Average perturbed word %:     | 9.84%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 46.39  |
+-------------------------------+--------+


In [165]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 5 / 40 / 5 / 50:  17%|████▏                    | 50/300 [03:52<19:24,  4.66s/it]textattack: Saving checkpoint under "checkpoints\1722861202803.ta.chkpt" at 2024-08-05 20:33:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 72 / 11 / 100:  33%|███████              | 100/300 [07:17<14:34,  4.37s/it]textattack: Saving checkpoint under "checkpoints\1722861407111.ta.chkpt" at 2024-08-05 20:36:47 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 109 / 16 / 150:  50%|██████████          | 150/300 [11:04<11:04,  4.43s/it]textattack: Saving checkpoint under "checkpoints\1722861634183.ta.chkpt" at 2024-08-05 20:40:34 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 145 / 21 / 200:  67%|█████████████▎      | 200/300 [14:57<07:28,  4.49s/it]textattack: Saving checkpoint under "checkpoints\1722861867278.ta.chkpt" at 2024-08-05 20:44:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 185 / 25 / 250:  83%|████████████████▋   | 250/300 [18:36<03:43,  4.47s/it]textattack: Saving checkpoint under "checkpoints\1722862086415.ta.chkpt" at 2024-08-05 20:48:06 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 227 / 30 / 300: 100%|████████████████████| 300/300 [22:14<00:00,  4.45s/it]textattack: Saving checkpoint under "checkpoints\1722862304429.ta.chkpt" at 2024-08-05 20:51:44 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 43 / 227 / 30 / 300: 100%|████████████████████| 300/300 [22:14<00:00,  4.45s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 43     |
| Number of failed attacks:     | 227    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 75.67% |
| Attack success rate:          | 15.93% |
| Average perturbed word %:     | 9.92%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 46.49  |
+-------------------------------+--------+


In [166]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 13 / 21 / 16 / 50:  17%|███▊                   | 50/300 [02:19<11:35,  2.78s/it]textattack: Saving checkpoint under "checkpoints\1722862443664.ta.chkpt" at 2024-08-05 20:54:03 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 44 / 31 / 100:  33%|███████              | 100/300 [04:43<09:26,  2.83s/it]textattack: Saving checkpoint under "checkpoints\1722862587835.ta.chkpt" at 2024-08-05 20:56:27 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 25 / 44 / 31 / 100:  34%|███████              | 101/300 [04:43<09:18,  2.81s/it]

[Succeeded / Failed / Skipped / Total] 40 / 70 / 40 / 150:  50%|██████████▌          | 150/300 [07:30<07:30,  3.00s/it]textattack: Saving checkpoint under "checkpoints\1722862755401.ta.chkpt" at 2024-08-05 20:59:15 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 51 / 94 / 55 / 200:  67%|██████████████       | 200/300 [10:17<05:08,  3.09s/it]textattack: Saving checkpoint under "checkpoints\1722862921914.ta.chkpt" at 2024-08-05 21:02:01 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 69 / 115 / 66 / 250:  83%|████████████████▋   | 250/300 [12:53<02:34,  3.09s/it]textattack: Saving checkpoint under "checkpoints\1722863078293.ta.chkpt" at 2024-08-05 21:04:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 82 / 140 / 78 / 300: 100%|████████████████████| 300/300 [15:40<00:00,  3.13s/it]textattack: Saving checkpoint under "checkpoints\1722863244789.ta.chkpt" at 2024-08-05 21:07:24 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 82 / 140 / 78 / 300: 100%|████████████████████| 300/300 [15:40<00:00,  3.13s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 82     |
| Number of failed attacks:     | 140    |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 46.67% |
| Attack success rate:          | 36.94% |
| Average perturbed word %:     | 7.09%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 43.14  |
+-------------------------------+--------+


In [167]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 16 / 25 / 9 / 50:  17%|████                    | 50/300 [02:49<14:07,  3.39s/it]textattack: Saving checkpoint under "checkpoints\1722863414418.ta.chkpt" at 2024-08-05 21:10:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 49 / 20 / 100:  33%|███████              | 100/300 [05:24<10:49,  3.25s/it]textattack: Saving checkpoint under "checkpoints\1722863569642.ta.chkpt" at 2024-08-05 21:12:49 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 42 / 79 / 29 / 150:  50%|██████████▌          | 150/300 [08:23<08:23,  3.35s/it]textattack: Saving checkpoint under "checkpoints\1722863748231.ta.chkpt" at 2024-08-05 21:15:48 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 105 / 45 / 200:  67%|█████████████▎      | 200/300 [11:13<05:36,  3.37s/it]textattack: Saving checkpoint under "checkpoints\1722863918135.ta.chkpt" at 2024-08-05 21:18:38 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 136 / 51 / 250:  83%|████████████████▋   | 250/300 [14:27<02:53,  3.47s/it]textattack: Saving checkpoint under "checkpoints\1722864112258.ta.chkpt" at 2024-08-05 21:21:52 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 71 / 171 / 58 / 300: 100%|████████████████████| 300/300 [17:35<00:00,  3.52s/it]textattack: Saving checkpoint under "checkpoints\1722864300345.ta.chkpt" at 2024-08-05 21:25:00 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 71 / 171 / 58 / 300: 100%|████████████████████| 300/300 [17:35<00:00,  3.52s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 71     |
| Number of failed attacks:     | 171    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 57.0%  |
| Attack success rate:          | 29.34% |
| Average perturbed word %:     | 7.13%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 44.53  |
+-------------------------------+--------+


In [168]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 12 / 29 / 9 / 50:  17%|████                    | 50/300 [03:11<15:56,  3.82s/it]textattack: Saving checkpoint under "checkpoints\1722864491899.ta.chkpt" at 2024-08-05 21:28:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 56 / 18 / 100:  33%|███████              | 100/300 [05:55<11:51,  3.56s/it]textattack: Saving checkpoint under "checkpoints\1722864656493.ta.chkpt" at 2024-08-05 21:30:56 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 84 / 27 / 150:  50%|██████████▌          | 150/300 [08:42<08:42,  3.48s/it]textattack: Saving checkpoint under "checkpoints\1722864823369.ta.chkpt" at 2024-08-05 21:33:43 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 48 / 110 / 42 / 200:  67%|█████████████▎      | 200/300 [11:39<05:49,  3.50s/it]textattack: Saving checkpoint under "checkpoints\1722864999771.ta.chkpt" at 2024-08-05 21:36:39 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 61 / 141 / 48 / 250:  83%|████████████████▋   | 250/300 [14:53<02:58,  3.57s/it]textattack: Saving checkpoint under "checkpoints\1722865194410.ta.chkpt" at 2024-08-05 21:39:54 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 67 / 178 / 55 / 300: 100%|████████████████████| 300/300 [18:16<00:00,  3.65s/it]textattack: Saving checkpoint under "checkpoints\1722865396709.ta.chkpt" at 2024-08-05 21:43:16 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 67 / 178 / 55 / 300: 100%|████████████████████| 300/300 [18:16<00:00,  3.65s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 67     |
| Number of failed attacks:     | 178    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 59.33% |
| Attack success rate:          | 27.35% |
| Average perturbed word %:     | 6.26%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 44.05  |
+-------------------------------+--------+


In [169]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 16 / 23 / 11 / 50:  17%|███▌                 | 50/300 [28:58<2:24:53, 34.77s/it]textattack: Saving checkpoint under "checkpoints\1722867135607.ta.chkpt" at 2024-08-05 22:12:15 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 45 / 20 / 100:  33%|██████▎            | 100/300 [32:44<1:05:29, 19.65s/it]textattack: Saving checkpoint under "checkpoints\1722867361768.ta.chkpt" at 2024-08-05 22:16:01 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 48 / 70 / 32 / 150:  50%|██████████▌          | 150/300 [36:38<36:38, 14.65s/it]textattack: Saving checkpoint under "checkpoints\1722867594924.ta.chkpt" at 2024-08-05 22:19:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 58 / 97 / 45 / 200:  67%|██████████████       | 200/300 [40:59<20:29, 12.30s/it]textattack: Saving checkpoint under "checkpoints\1722867855991.ta.chkpt" at 2024-08-05 22:24:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 72 / 126 / 52 / 250:  83%|████████████████▋   | 250/300 [45:22<09:04, 10.89s/it]textattack: Saving checkpoint under "checkpoints\1722868119710.ta.chkpt" at 2024-08-05 22:28:39 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 88 / 154 / 58 / 300: 100%|████████████████████| 300/300 [49:29<00:00,  9.90s/it]textattack: Saving checkpoint under "checkpoints\1722868366569.ta.chkpt" at 2024-08-05 22:32:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 88 / 154 / 58 / 300: 100%|████████████████████| 300/300 [49:29<00:00,  9.90s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 88     |
| Number of failed attacks:     | 154    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 51.33% |
| Attack success rate:          | 36.36% |
| Average perturbed word %:     | 3.93%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 43.21  |
+-------------------------------+--------+


In [170]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 14 / 27 / 9 / 50:  17%|████                    | 50/300 [04:02<20:14,  4.86s/it]textattack: Saving checkpoint under "checkpoints\1722868609805.ta.chkpt" at 2024-08-05 22:36:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 51 / 19 / 100:  33%|███████              | 100/300 [07:57<15:55,  4.78s/it]textattack: Saving checkpoint under "checkpoints\1722868844402.ta.chkpt" at 2024-08-05 22:40:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 78 / 28 / 150:  50%|██████████▌          | 150/300 [11:58<11:58,  4.79s/it]textattack: Saving checkpoint under "checkpoints\1722869085396.ta.chkpt" at 2024-08-05 22:44:45 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 53 / 104 / 43 / 200:  67%|█████████████▎      | 200/300 [16:09<08:04,  4.85s/it]textattack: Saving checkpoint under "checkpoints\1722869336055.ta.chkpt" at 2024-08-05 22:48:56 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 63 / 136 / 51 / 250:  83%|████████████████▋   | 250/300 [20:41<04:08,  4.97s/it]textattack: Saving checkpoint under "checkpoints\1722869608702.ta.chkpt" at 2024-08-05 22:53:28 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 77 / 167 / 56 / 300: 100%|████████████████████| 300/300 [25:12<00:00,  5.04s/it]textattack: Saving checkpoint under "checkpoints\1722869879410.ta.chkpt" at 2024-08-05 22:57:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 77 / 167 / 56 / 300: 100%|████████████████████| 300/300 [25:12<00:00,  5.04s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 77     |
| Number of failed attacks:     | 167    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 55.67% |
| Attack success rate:          | 31.56% |
| Average perturbed word %:     | 6.35%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 44.09  |
+-------------------------------+--------+


In [171]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 19 / 22 / 9 / 50:  17%|████                    | 50/300 [03:38<18:14,  4.38s/it]textattack: Saving checkpoint under "checkpoints\1722870098432.ta.chkpt" at 2024-08-05 23:01:38 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 46 / 20 / 100:  33%|███████              | 100/300 [07:21<14:43,  4.42s/it]textattack: Saving checkpoint under "checkpoints\1722870321467.ta.chkpt" at 2024-08-05 23:05:21 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 46 / 71 / 33 / 150:  50%|██████████▌          | 150/300 [11:08<11:08,  4.46s/it]textattack: Saving checkpoint under "checkpoints\1722870548328.ta.chkpt" at 2024-08-05 23:09:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 58 / 95 / 47 / 200:  67%|██████████████       | 200/300 [15:10<07:35,  4.55s/it]textattack: Saving checkpoint under "checkpoints\1722870790260.ta.chkpt" at 2024-08-05 23:13:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 73 / 123 / 54 / 250:  83%|████████████████▋   | 250/300 [19:38<03:55,  4.71s/it]textattack: Saving checkpoint under "checkpoints\1722871058116.ta.chkpt" at 2024-08-05 23:17:38 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 87 / 153 / 60 / 300: 100%|████████████████████| 300/300 [23:45<00:00,  4.75s/it]textattack: Saving checkpoint under "checkpoints\1722871304832.ta.chkpt" at 2024-08-05 23:21:44 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 87 / 153 / 60 / 300: 100%|████████████████████| 300/300 [23:45<00:00,  4.75s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 87     |
| Number of failed attacks:     | 153    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 51.0%  |
| Attack success rate:          | 36.25% |
| Average perturbed word %:     | 7.57%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 43.29  |
+-------------------------------+--------+


#### CLIP

In [172]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 45 / 4 / 50:  17%|████▏                    | 50/300 [03:01<15:05,  3.62s/it]textattack: Saving checkpoint under "checkpoints\1722871486153.ta.chkpt" at 2024-08-05 23:24:46 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 88 / 9 / 100:  33%|███████▋               | 100/300 [05:38<11:17,  3.39s/it]textattack: Saving checkpoint under "checkpoints\1722871643939.ta.chkpt" at 2024-08-05 23:27:23 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 130 / 14 / 150:  50%|██████████▌          | 150/300 [08:19<08:19,  3.33s/it]textattack: Saving checkpoint under "checkpoints\1722871804100.ta.chkpt" at 2024-08-05 23:30:04 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 169 / 20 / 200:  67%|█████████████▎      | 200/300 [10:56<05:28,  3.28s/it]textattack: Saving checkpoint under "checkpoints\1722871961574.ta.chkpt" at 2024-08-05 23:32:41 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 211 / 24 / 250:  83%|████████████████▋   | 250/300 [13:40<02:44,  3.28s/it]textattack: Saving checkpoint under "checkpoints\1722872125989.ta.chkpt" at 2024-08-05 23:35:25 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 252 / 29 / 300: 100%|████████████████████| 300/300 [16:15<00:00,  3.25s/it]textattack: Saving checkpoint under "checkpoints\1722872280685.ta.chkpt" at 2024-08-05 23:38:00 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 252 / 29 / 300: 100%|████████████████████| 300/300 [16:15<00:00,  3.25s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 252    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 84.0%  |
| Attack success rate:          | 7.01%  |
| Average perturbed word %:     | 9.72%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 47.09  |
+-------------------------------+--------+


In [173]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 45 / 4 / 50:  17%|████▏                    | 50/300 [02:31<12:38,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722872432597.ta.chkpt" at 2024-08-05 23:40:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 88 / 9 / 100:  33%|███████▋               | 100/300 [05:04<10:08,  3.04s/it]textattack: Saving checkpoint under "checkpoints\1722872585312.ta.chkpt" at 2024-08-05 23:43:05 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 130 / 14 / 150:  50%|██████████▌          | 150/300 [07:37<07:37,  3.05s/it]textattack: Saving checkpoint under "checkpoints\1722872738607.ta.chkpt" at 2024-08-05 23:45:38 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 169 / 20 / 200:  67%|█████████████▎      | 200/300 [10:09<05:04,  3.05s/it]textattack: Saving checkpoint under "checkpoints\1722872889975.ta.chkpt" at 2024-08-05 23:48:09 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 211 / 24 / 250:  83%|████████████████▋   | 250/300 [12:49<02:33,  3.08s/it]textattack: Saving checkpoint under "checkpoints\1722873050323.ta.chkpt" at 2024-08-05 23:50:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 252 / 29 / 300: 100%|████████████████████| 300/300 [15:20<00:00,  3.07s/it]textattack: Saving checkpoint under "checkpoints\1722873201109.ta.chkpt" at 2024-08-05 23:53:21 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 252 / 29 / 300: 100%|████████████████████| 300/300 [15:20<00:00,  3.07s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 252    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 84.0%  |
| Attack success rate:          | 7.01%  |
| Average perturbed word %:     | 9.72%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 47.09  |
+-------------------------------+--------+


In [174]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 45 / 4 / 50:  17%|████▏                    | 50/300 [02:32<12:40,  3.04s/it]textattack: Saving checkpoint under "checkpoints\1722873353427.ta.chkpt" at 2024-08-05 23:55:53 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 88 / 9 / 100:  33%|███████▋               | 100/300 [05:02<10:05,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722873504233.ta.chkpt" at 2024-08-05 23:58:24 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 130 / 14 / 150:  50%|██████████▌          | 150/300 [07:35<07:35,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722873656410.ta.chkpt" at 2024-08-06 00:00:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 169 / 20 / 200:  67%|█████████████▎      | 200/300 [10:06<05:03,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722873807984.ta.chkpt" at 2024-08-06 00:03:27 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 211 / 24 / 250:  83%|████████████████▋   | 250/300 [12:48<02:33,  3.07s/it]textattack: Saving checkpoint under "checkpoints\1722873969813.ta.chkpt" at 2024-08-06 00:06:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 252 / 29 / 300: 100%|████████████████████| 300/300 [15:22<00:00,  3.07s/it]textattack: Saving checkpoint under "checkpoints\1722874123411.ta.chkpt" at 2024-08-06 00:08:43 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 252 / 29 / 300: 100%|████████████████████| 300/300 [15:22<00:00,  3.07s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 252    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 84.0%  |
| Attack success rate:          | 7.01%  |
| Average perturbed word %:     | 9.72%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 47.09  |
+-------------------------------+--------+


In [175]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 43 / 5 / 50:  17%|████▏                    | 50/300 [02:23<11:55,  2.86s/it]textattack: Saving checkpoint under "checkpoints\1722874266815.ta.chkpt" at 2024-08-06 00:11:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 80 / 13 / 100:  33%|███████▎              | 100/300 [04:47<09:34,  2.87s/it]textattack: Saving checkpoint under "checkpoints\1722874410897.ta.chkpt" at 2024-08-06 00:13:30 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 119 / 19 / 150:  50%|██████████          | 150/300 [07:12<07:12,  2.88s/it]textattack: Saving checkpoint under "checkpoints\1722874556218.ta.chkpt" at 2024-08-06 00:15:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 149 / 35 / 200:  67%|█████████████▎      | 200/300 [09:00<04:30,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1722874664607.ta.chkpt" at 2024-08-06 00:17:44 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 186 / 44 / 250:  83%|████████████████▋   | 250/300 [11:23<02:16,  2.73s/it]textattack: Saving checkpoint under "checkpoints\1722874807160.ta.chkpt" at 2024-08-06 00:20:07 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 229 / 49 / 300: 100%|████████████████████| 300/300 [13:58<00:00,  2.80s/it]textattack: Saving checkpoint under "checkpoints\1722874962353.ta.chkpt" at 2024-08-06 00:22:42 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 22 / 229 / 49 / 300: 100%|████████████████████| 300/300 [13:58<00:00,  2.80s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 22     |
| Number of failed attacks:     | 229    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 76.33% |
| Attack success rate:          | 8.76%  |
| Average perturbed word %:     | 5.68%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 46.22  |
+-------------------------------+--------+


In [176]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 44 / 2 / 50:  17%|████▏                    | 50/300 [02:40<13:20,  3.20s/it]textattack: Saving checkpoint under "checkpoints\1722875122468.ta.chkpt" at 2024-08-06 00:25:22 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 85 / 7 / 100:  33%|███████▋               | 100/300 [05:15<10:31,  3.16s/it]textattack: Saving checkpoint under "checkpoints\1722875278025.ta.chkpt" at 2024-08-06 00:27:58 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 126 / 13 / 150:  50%|██████████          | 150/300 [07:45<07:45,  3.10s/it]textattack: Saving checkpoint under "checkpoints\1722875427794.ta.chkpt" at 2024-08-06 00:30:27 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 161 / 23 / 200:  67%|█████████████▎      | 200/300 [09:57<04:58,  2.99s/it]textattack: Saving checkpoint under "checkpoints\1722875560104.ta.chkpt" at 2024-08-06 00:32:40 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 202 / 29 / 250:  83%|████████████████▋   | 250/300 [12:37<02:31,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722875720295.ta.chkpt" at 2024-08-06 00:35:20 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 245 / 35 / 300: 100%|████████████████████| 300/300 [15:10<00:00,  3.03s/it]textattack: Saving checkpoint under "checkpoints\1722875872939.ta.chkpt" at 2024-08-06 00:37:52 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 20 / 245 / 35 / 300: 100%|████████████████████| 300/300 [15:10<00:00,  3.04s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 20     |
| Number of failed attacks:     | 245    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 81.67% |
| Attack success rate:          | 7.55%  |
| Average perturbed word %:     | 7.09%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 46.66  |
+-------------------------------+--------+


In [177]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 44 / 2 / 50:  17%|████▏                    | 50/300 [03:07<15:36,  3.75s/it]textattack: Saving checkpoint under "checkpoints\1722876060478.ta.chkpt" at 2024-08-06 00:41:00 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 83 / 7 / 100:  33%|███████▎              | 100/300 [05:45<11:30,  3.45s/it]textattack: Saving checkpoint under "checkpoints\1722876218235.ta.chkpt" at 2024-08-06 00:43:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 124 / 10 / 150:  50%|██████████          | 150/300 [08:21<08:21,  3.34s/it]textattack: Saving checkpoint under "checkpoints\1722876374241.ta.chkpt" at 2024-08-06 00:46:14 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 22 / 158 / 20 / 200:  67%|█████████████▎      | 200/300 [10:40<05:20,  3.20s/it]textattack: Saving checkpoint under "checkpoints\1722876513180.ta.chkpt" at 2024-08-06 00:48:33 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 197 / 25 / 250:  83%|████████████████▋   | 250/300 [13:19<02:39,  3.20s/it]textattack: Saving checkpoint under "checkpoints\1722876672145.ta.chkpt" at 2024-08-06 00:51:12 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 238 / 31 / 300: 100%|████████████████████| 300/300 [15:48<00:00,  3.16s/it]textattack: Saving checkpoint under "checkpoints\1722876821373.ta.chkpt" at 2024-08-06 00:53:41 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 31 / 238 / 31 / 300: 100%|████████████████████| 300/300 [15:48<00:00,  3.16s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 31     |
| Number of failed attacks:     | 238    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 79.33% |
| Attack success rate:          | 11.52% |
| Average perturbed word %:     | 8.44%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 46.57  |
+-------------------------------+--------+


In [178]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [33:38<2:48:10, 40.36s/it]textattack: Saving checkpoint under "checkpoints\1722878839621.ta.chkpt" at 2024-08-06 01:27:19 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 95 / 4 / 100:  33%|███████              | 100/300 [37:27<1:14:55, 22.48s/it]textattack: Saving checkpoint under "checkpoints\1722879069138.ta.chkpt" at 2024-08-06 01:31:09 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 141 / 7 / 150:  50%|███████████           | 150/300 [41:08<41:08, 16.46s/it]textattack: Saving checkpoint under "checkpoints\1722879289897.ta.chkpt" at 2024-08-06 01:34:49 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 184 / 14 / 200:  67%|██████████████       | 200/300 [44:51<22:25, 13.46s/it]textattack: Saving checkpoint under "checkpoints\1722879512634.ta.chkpt" at 2024-08-06 01:38:32 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 228 / 17 / 250:  83%|█████████████████▌   | 250/300 [48:50<09:46, 11.72s/it]textattack: Saving checkpoint under "checkpoints\1722879751729.ta.chkpt" at 2024-08-06 01:42:31 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 272 / 22 / 300: 100%|█████████████████████| 300/300 [52:19<00:00, 10.46s/it]textattack: Saving checkpoint under "checkpoints\1722879960736.ta.chkpt" at 2024-08-06 01:46:00 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 6 / 272 / 22 / 300: 100%|█████████████████████| 300/300 [52:19<00:00, 10.46s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 272    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 90.67% |
| Attack success rate:          | 2.16%  |
| Average perturbed word %:     | 2.85%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 47.4   |
+-------------------------------+--------+


In [179]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 48 / 0 / 50:  17%|████▏                    | 50/300 [03:46<18:51,  4.53s/it]textattack: Saving checkpoint under "checkpoints\1722880187420.ta.chkpt" at 2024-08-06 01:49:47 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 91 / 3 / 100:  33%|███████▋               | 100/300 [07:35<15:10,  4.55s/it]textattack: Saving checkpoint under "checkpoints\1722880416132.ta.chkpt" at 2024-08-06 01:53:36 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 137 / 6 / 150:  50%|███████████           | 150/300 [11:18<11:18,  4.53s/it]textattack: Saving checkpoint under "checkpoints\1722880640001.ta.chkpt" at 2024-08-06 01:57:20 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 176 / 13 / 200:  67%|█████████████▎      | 200/300 [14:51<07:25,  4.46s/it]textattack: Saving checkpoint under "checkpoints\1722880852243.ta.chkpt" at 2024-08-06 02:00:52 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 219 / 16 / 250:  83%|████████████████▋   | 250/300 [18:42<03:44,  4.49s/it]textattack: Saving checkpoint under "checkpoints\1722881083822.ta.chkpt" at 2024-08-06 02:04:43 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 262 / 22 / 300: 100%|████████████████████| 300/300 [22:11<00:00,  4.44s/it]textattack: Saving checkpoint under "checkpoints\1722881292183.ta.chkpt" at 2024-08-06 02:08:12 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 16 / 262 / 22 / 300: 100%|████████████████████| 300/300 [22:11<00:00,  4.44s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 262    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 87.33% |
| Attack success rate:          | 5.76%  |
| Average perturbed word %:     | 5.54%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 47.14  |
+-------------------------------+--------+


In [180]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175sim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 48 / 1 / 50:  17%|████▏                    | 50/300 [03:42<18:31,  4.45s/it]textattack: Saving checkpoint under "checkpoints\1722881514642.ta.chkpt" at 2024-08-06 02:11:54 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 89 / 5 / 100:  33%|███████▋               | 100/300 [07:21<14:43,  4.42s/it]textattack: Saving checkpoint under "checkpoints\1722881734328.ta.chkpt" at 2024-08-06 02:15:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 132 / 9 / 150:  50%|███████████           | 150/300 [10:56<10:56,  4.37s/it]textattack: Saving checkpoint under "checkpoints\1722881948603.ta.chkpt" at 2024-08-06 02:19:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 168 / 21 / 200:  67%|█████████████▎      | 200/300 [14:03<07:01,  4.22s/it]textattack: Saving checkpoint under "checkpoints\1722882135517.ta.chkpt" at 2024-08-06 02:22:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 209 / 28 / 250:  83%|████████████████▋   | 250/300 [17:37<03:31,  4.23s/it]textattack: Saving checkpoint under "checkpoints\1722882350291.ta.chkpt" at 2024-08-06 02:25:50 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 253 / 34 / 300: 100%|████████████████████| 300/300 [21:05<00:00,  4.22s/it]textattack: Saving checkpoint under "checkpoints\1722882557633.ta.chkpt" at 2024-08-06 02:29:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 13 / 253 / 34 / 300: 100%|████████████████████| 300/300 [21:05<00:00,  4.22s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 13     |
| Number of failed attacks:     | 253    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 84.33% |
| Attack success rate:          | 4.89%  |
| Average perturbed word %:     | 7.73%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 46.84  |
+-------------------------------+--------+


In [181]:
et8 = datetime.datetime.now()
et8 - et7

datetime.timedelta(seconds=24294, microseconds=102795)

### MinSim 0.9, threshold 0.5

In [182]:
suffix = 'sim09.csv'
thre = 1-0.5/np.pi
minsim = 0.9

#### BERT

In [183]:
runAttack(bert_lr1_ag, n_samples, folder+'bertagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr1sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:31<12:35,  3.02s/it]textattack: Saving checkpoint under "checkpoints\1722882709447.ta.chkpt" at 2024-08-06 02:31:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 83 / 10 / 100:  33%|███████▎              | 100/300 [05:15<10:30,  3.15s/it]textattack: Saving checkpoint under "checkpoints\1722882873630.ta.chkpt" at 2024-08-06 02:34:33 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 127 / 15 / 150:  50%|██████████▌          | 150/300 [08:06<08:06,  3.24s/it]textattack: Saving checkpoint under "checkpoints\1722883044804.ta.chkpt" at 2024-08-06 02:37:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 168 / 21 / 200:  67%|█████████████▎      | 200/300 [11:27<05:43,  3.44s/it]textattack: Saving checkpoint under "checkpoints\1722883245281.ta.chkpt" at 2024-08-06 02:40:45 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 210 / 25 / 250:  83%|████████████████▋   | 250/300 [14:43<02:56,  3.53s/it]textattack: Saving checkpoint under "checkpoints\1722883441712.ta.chkpt" at 2024-08-06 02:44:01 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 253 / 30 / 300: 100%|████████████████████| 300/300 [17:40<00:00,  3.54s/it]textattack: Saving checkpoint under "checkpoints\1722883619181.ta.chkpt" at 2024-08-06 02:46:59 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 17 / 253 / 30 / 300: 100%|████████████████████| 300/300 [17:40<00:00,  3.54s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 17     |
| Number of failed attacks:     | 253    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 84.33% |
| Attack success rate:          | 6.3%   |
| Average perturbed word %:     | 6.1%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.73  |
+-------------------------------+--------+


In [184]:
runAttack(bert_lr01_ag, n_samples, folder+'bertagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr01sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:48<14:00,  3.36s/it]textattack: Saving checkpoint under "checkpoints\1722883788047.ta.chkpt" at 2024-08-06 02:49:48 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 82 / 11 / 100:  33%|███████▎              | 100/300 [05:26<10:52,  3.26s/it]textattack: Saving checkpoint under "checkpoints\1722883946127.ta.chkpt" at 2024-08-06 02:52:26 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 126 / 16 / 150:  50%|██████████▌          | 150/300 [08:14<08:14,  3.30s/it]textattack: Saving checkpoint under "checkpoints\1722884114540.ta.chkpt" at 2024-08-06 02:55:14 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 165 / 23 / 200:  67%|█████████████▎      | 200/300 [11:07<05:33,  3.34s/it]textattack: Saving checkpoint under "checkpoints\1722884287347.ta.chkpt" at 2024-08-06 02:58:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 207 / 27 / 250:  83%|████████████████▋   | 250/300 [14:00<02:48,  3.36s/it]textattack: Saving checkpoint under "checkpoints\1722884460728.ta.chkpt" at 2024-08-06 03:01:00 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 250 / 32 / 300: 100%|████████████████████| 300/300 [16:51<00:00,  3.37s/it]textattack: Saving checkpoint under "checkpoints\1722884631829.ta.chkpt" at 2024-08-06 03:03:51 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 18 / 250 / 32 / 300: 100%|████████████████████| 300/300 [16:51<00:00,  3.37s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 18     |
| Number of failed attacks:     | 250    |
| Number of skipped attacks:    | 32     |
| Original accuracy:            | 89.33% |
| Accuracy under attack:        | 83.33% |
| Attack success rate:          | 6.72%  |
| Average perturbed word %:     | 6.61%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.78  |
+-------------------------------+--------+

In [185]:
runAttack(bert_lr10_ag, n_samples, folder+'bertagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattacklr10sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 44 / 5 / 50:  17%|████▏                    | 50/300 [02:36<13:02,  3.13s/it]textattack: Saving checkpoint under "checkpoints\1722884788882.ta.chkpt" at 2024-08-06 03:06:28 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 83 / 11 / 100:  33%|███████▎              | 100/300 [05:18<10:37,  3.19s/it]textattack: Saving checkpoint under "checkpoints\1722884951075.ta.chkpt" at 2024-08-06 03:09:11 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 127 / 16 / 150:  50%|██████████▌          | 150/300 [08:02<08:02,  3.22s/it]textattack: Saving checkpoint under "checkpoints\1722885114647.ta.chkpt" at 2024-08-06 03:11:54 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 168 / 21 / 200:  67%|█████████████▎      | 200/300 [10:57<05:28,  3.29s/it]textattack: Saving checkpoint under "checkpoints\1722885290226.ta.chkpt" at 2024-08-06 03:14:50 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 211 / 25 / 250:  83%|████████████████▋   | 250/300 [13:51<02:46,  3.32s/it]textattack: Saving checkpoint under "checkpoints\1722885463503.ta.chkpt" at 2024-08-06 03:17:43 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 254 / 30 / 300: 100%|████████████████████| 300/300 [16:43<00:00,  3.35s/it]textattack: Saving checkpoint under "checkpoints\1722885636184.ta.chkpt" at 2024-08-06 03:20:36 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 16 / 254 / 30 / 300: 100%|████████████████████| 300/300 [16:43<00:00,  3.35s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 254    |
| Number of skipped attacks:    | 30     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 84.67% |
| Attack success rate:          | 5.93%  |
| Average perturbed word %:     | 6.19%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.78  |
+-------------------------------+--------+


In [186]:
runAttack(bert_rf7_ag, n_samples, folder+'bertagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 26 / 16 / 50:  17%|████                    | 50/300 [01:52<09:24,  2.26s/it]textattack: Saving checkpoint under "checkpoints\1722885749281.ta.chkpt" at 2024-08-06 03:22:29 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 53 / 31 / 100:  33%|███████              | 100/300 [03:54<07:48,  2.34s/it]textattack: Saving checkpoint under "checkpoints\1722885870446.ta.chkpt" at 2024-08-06 03:24:30 after 100 attacks.
[Succeeded / Failed / Skipped / Total] 16 / 53 / 32 / 101:  34%|███████              | 101/300 [03:54<07:41,  2.32s/it]

[Succeeded / Failed / Skipped / Total] 24 / 86 / 40 / 150:  50%|██████████▌          | 150/300 [06:14<06:14,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1722886011433.ta.chkpt" at 2024-08-06 03:26:51 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 113 / 55 / 200:  67%|█████████████▎      | 200/300 [08:37<04:18,  2.59s/it]textattack: Saving checkpoint under "checkpoints\1722886154395.ta.chkpt" at 2024-08-06 03:29:14 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 143 / 66 / 250:  83%|████████████████▋   | 250/300 [10:54<02:10,  2.62s/it]textattack: Saving checkpoint under "checkpoints\1722886290509.ta.chkpt" at 2024-08-06 03:31:30 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 177 / 78 / 300: 100%|████████████████████| 300/300 [13:40<00:00,  2.74s/it]textattack: Saving checkpoint under "checkpoints\1722886457267.ta.chkpt" at 2024-08-06 03:34:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 45 / 177 / 78 / 300: 100%|████████████████████| 300/300 [13:40<00:00,  2.74s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 45     |
| Number of failed attacks:     | 177    |
| Number of skipped attacks:    | 78     |
| Original accuracy:            | 74.0%  |
| Accuracy under attack:        | 59.0%  |
| Attack success rate:          | 20.27% |
| Average perturbed word %:     | 5.8%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 36.61  |
+-------------------------------+--------+


In [187]:
runAttack(bert_rf14_ag, n_samples, folder+'bertagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf14sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 35 / 9 / 50:  17%|████▏                    | 50/300 [02:38<13:14,  3.18s/it]textattack: Saving checkpoint under "checkpoints\1722886616522.ta.chkpt" at 2024-08-06 03:36:56 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 64 / 20 / 100:  33%|███████              | 100/300 [04:57<09:54,  2.97s/it]textattack: Saving checkpoint under "checkpoints\1722886754807.ta.chkpt" at 2024-08-06 03:39:14 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 98 / 29 / 150:  50%|██████████▌          | 150/300 [07:31<07:31,  3.01s/it]textattack: Saving checkpoint under "checkpoints\1722886909191.ta.chkpt" at 2024-08-06 03:41:49 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 27 / 128 / 45 / 200:  67%|█████████████▎      | 200/300 [10:00<05:00,  3.00s/it]textattack: Saving checkpoint under "checkpoints\1722887057679.ta.chkpt" at 2024-08-06 03:44:17 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 164 / 51 / 250:  83%|████████████████▋   | 250/300 [12:52<02:34,  3.09s/it]textattack: Saving checkpoint under "checkpoints\1722887229970.ta.chkpt" at 2024-08-06 03:47:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 39 / 203 / 58 / 300: 100%|████████████████████| 300/300 [16:08<00:00,  3.23s/it]textattack: Saving checkpoint under "checkpoints\1722887426226.ta.chkpt" at 2024-08-06 03:50:26 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 39 / 203 / 58 / 300: 100%|████████████████████| 300/300 [16:08<00:00,  3.23s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 39     |
| Number of failed attacks:     | 203    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 67.67% |
| Attack success rate:          | 16.12% |
| Average perturbed word %:     | 5.52%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.29  |
+-------------------------------+--------+


In [188]:
runAttack(bert_rf30_ag, n_samples, folder+'bertagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackrf30sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 9 / 32 / 9 / 50:  17%|████▏                    | 50/300 [02:46<13:54,  3.34s/it]textattack: Saving checkpoint under "checkpoints\1722887594595.ta.chkpt" at 2024-08-06 03:53:14 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 63 / 18 / 100:  33%|███████              | 100/300 [05:11<10:22,  3.11s/it]textattack: Saving checkpoint under "checkpoints\1722887738899.ta.chkpt" at 2024-08-06 03:55:38 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 97 / 27 / 150:  50%|██████████▌          | 150/300 [07:48<07:48,  3.13s/it]textattack: Saving checkpoint under "checkpoints\1722887896477.ta.chkpt" at 2024-08-06 03:58:16 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 125 / 42 / 200:  67%|█████████████▎      | 200/300 [10:13<05:06,  3.07s/it]textattack: Saving checkpoint under "checkpoints\1722888041341.ta.chkpt" at 2024-08-06 04:00:41 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 162 / 48 / 250:  83%|████████████████▋   | 250/300 [13:00<02:36,  3.12s/it]textattack: Saving checkpoint under "checkpoints\1722888207650.ta.chkpt" at 2024-08-06 04:03:27 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 202 / 55 / 300: 100%|████████████████████| 300/300 [15:43<00:00,  3.14s/it]textattack: Saving checkpoint under "checkpoints\1722888370862.ta.chkpt" at 2024-08-06 04:06:10 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 43 / 202 / 55 / 300: 100%|████████████████████| 300/300 [15:43<00:00,  3.14s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 43     |
| Number of failed attacks:     | 202    |
| Number of skipped attacks:    | 55     |
| Original accuracy:            | 81.67% |
| Accuracy under attack:        | 67.33% |
| Attack success rate:          | 17.55% |
| Average perturbed word %:     | 4.86%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 36.87  |
+-------------------------------+--------+


In [189]:
runAttack(bert_knn7_ag, n_samples, folder+'bertagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 10 / 29 / 11 / 50:  17%|███▌                 | 50/300 [28:21<2:21:45, 34.02s/it]textattack: Saving checkpoint under "checkpoints\1722890072090.ta.chkpt" at 2024-08-06 04:34:32 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 55 / 20 / 100:  33%|██████▎            | 100/300 [31:34<1:03:09, 18.95s/it]textattack: Saving checkpoint under "checkpoints\1722890265987.ta.chkpt" at 2024-08-06 04:37:45 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 84 / 32 / 150:  50%|██████████▌          | 150/300 [34:34<34:34, 13.83s/it]textattack: Saving checkpoint under "checkpoints\1722890445417.ta.chkpt" at 2024-08-06 04:40:45 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 45 / 110 / 45 / 200:  67%|█████████████▎      | 200/300 [37:59<18:59, 11.40s/it]textattack: Saving checkpoint under "checkpoints\1722890650602.ta.chkpt" at 2024-08-06 04:44:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 55 / 143 / 52 / 250:  83%|████████████████▋   | 250/300 [41:48<08:21, 10.03s/it]textattack: Saving checkpoint under "checkpoints\1722890879425.ta.chkpt" at 2024-08-06 04:47:59 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 64 / 178 / 58 / 300: 100%|████████████████████| 300/300 [45:35<00:00,  9.12s/it]textattack: Saving checkpoint under "checkpoints\1722891106776.ta.chkpt" at 2024-08-06 04:51:46 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 64 / 178 / 58 / 300: 100%|████████████████████| 300/300 [45:35<00:00,  9.12s/it]






+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 64     |
| Number of failed attacks:     | 178    |
| Number of skipped attacks:    | 58     |
| Original accuracy:            | 80.67% |
| Accuracy under attack:        | 59.33% |
| Attack success rate:          | 26.45% |
| Average perturbed word %:     | 3.83%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 36.64  |
+-------------------------------+--------+


In [190]:
runAttack(bert_knn35_ag, n_samples, folder+'bertagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn35sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 33 / 9 / 50:  17%|████▏                    | 50/300 [03:22<16:54,  4.06s/it]textattack: Saving checkpoint under "checkpoints\1722891311330.ta.chkpt" at 2024-08-06 04:55:11 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 64 / 19 / 100:  33%|███████              | 100/300 [06:45<13:31,  4.06s/it]textattack: Saving checkpoint under "checkpoints\1722891514119.ta.chkpt" at 2024-08-06 04:58:34 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 97 / 28 / 150:  50%|██████████▌          | 150/300 [09:59<09:59,  3.99s/it]textattack: Saving checkpoint under "checkpoints\1722891707597.ta.chkpt" at 2024-08-06 05:01:47 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 127 / 43 / 200:  67%|█████████████▎      | 200/300 [13:21<06:40,  4.01s/it]textattack: Saving checkpoint under "checkpoints\1722891910324.ta.chkpt" at 2024-08-06 05:05:10 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 166 / 51 / 250:  83%|████████████████▋   | 250/300 [16:59<03:23,  4.08s/it]textattack: Saving checkpoint under "checkpoints\1722892127594.ta.chkpt" at 2024-08-06 05:08:47 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 203 / 56 / 300: 100%|████████████████████| 300/300 [20:42<00:00,  4.14s/it]textattack: Saving checkpoint under "checkpoints\1722892351100.ta.chkpt" at 2024-08-06 05:12:31 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 41 / 203 / 56 / 300: 100%|████████████████████| 300/300 [20:42<00:00,  4.14s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 41     |
| Number of failed attacks:     | 203    |
| Number of skipped attacks:    | 56     |
| Original accuracy:            | 81.33% |
| Accuracy under attack:        | 67.67% |
| Attack success rate:          | 16.8%  |
| Average perturbed word %:     | 4.31%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 36.97  |
+-------------------------------+--------+


In [191]:
runAttack(bert_knn175_ag, n_samples, folder+'bertagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertModel'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackknn175sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 8 / 33 / 9 / 50:  17%|████▏                    | 50/300 [03:08<15:42,  3.77s/it]textattack: Saving checkpoint under "checkpoints\1722892539803.ta.chkpt" at 2024-08-06 05:15:39 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 60 / 20 / 100:  33%|███████              | 100/300 [06:10<12:21,  3.71s/it]textattack: Saving checkpoint under "checkpoints\1722892722002.ta.chkpt" at 2024-08-06 05:18:42 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 91 / 33 / 150:  50%|██████████▌          | 150/300 [09:11<09:11,  3.68s/it]textattack: Saving checkpoint under "checkpoints\1722892902712.ta.chkpt" at 2024-08-06 05:21:42 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 34 / 119 / 47 / 200:  67%|█████████████▎      | 200/300 [12:35<06:17,  3.78s/it]textattack: Saving checkpoint under "checkpoints\1722893107299.ta.chkpt" at 2024-08-06 05:25:07 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 40 / 156 / 54 / 250:  83%|████████████████▋   | 250/300 [16:17<03:15,  3.91s/it]textattack: Saving checkpoint under "checkpoints\1722893328948.ta.chkpt" at 2024-08-06 05:28:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 47 / 193 / 60 / 300: 100%|████████████████████| 300/300 [19:53<00:00,  3.98s/it]textattack: Saving checkpoint under "checkpoints\1722893545000.ta.chkpt" at 2024-08-06 05:32:25 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 47 / 193 / 60 / 300: 100%|████████████████████| 300/300 [19:53<00:00,  3.98s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 47     |
| Number of failed attacks:     | 193    |
| Number of skipped attacks:    | 60     |
| Original accuracy:            | 80.0%  |
| Accuracy under attack:        | 64.33% |
| Attack success rate:          | 19.58% |
| Average perturbed word %:     | 5.1%   |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 36.74  |
+-------------------------------+--------+


#### CLIP

In [192]:
runAttack(clip_lr1_ag, n_samples, folder+'clipagattacklr1'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr1sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 45 / 4 / 50:  17%|████▏                    | 50/300 [02:24<12:04,  2.90s/it]textattack: Saving checkpoint under "checkpoints\1722893690023.ta.chkpt" at 2024-08-06 05:34:50 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 88 / 9 / 100:  33%|███████▋               | 100/300 [04:33<09:07,  2.74s/it]textattack: Saving checkpoint under "checkpoints\1722893819076.ta.chkpt" at 2024-08-06 05:36:59 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 132 / 14 / 150:  50%|██████████▌          | 150/300 [06:43<06:43,  2.69s/it]textattack: Saving checkpoint under "checkpoints\1722893949027.ta.chkpt" at 2024-08-06 05:39:09 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 174 / 20 / 200:  67%|██████████████       | 200/300 [08:53<04:26,  2.67s/it]textattack: Saving checkpoint under "checkpoints\1722894079035.ta.chkpt" at 2024-08-06 05:41:19 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 216 / 24 / 250:  83%|████████████████▋   | 250/300 [11:15<02:15,  2.70s/it]textattack: Saving checkpoint under "checkpoints\1722894220824.ta.chkpt" at 2024-08-06 05:43:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 260 / 29 / 300: 100%|████████████████████| 300/300 [13:25<00:00,  2.69s/it]textattack: Saving checkpoint under "checkpoints\1722894350899.ta.chkpt" at 2024-08-06 05:45:50 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 11 / 260 / 29 / 300: 100%|████████████████████| 300/300 [13:25<00:00,  2.69s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 4.06%  |
| Average perturbed word %:     | 8.79%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.89  |
+-------------------------------+--------+


In [193]:
runAttack(clip_lr01_ag, n_samples, folder+'clipagattacklr01'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr01sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 45 / 4 / 50:  17%|████▏                    | 50/300 [01:58<09:51,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1722894469261.ta.chkpt" at 2024-08-06 05:47:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 88 / 9 / 100:  33%|███████▋               | 100/300 [04:01<08:02,  2.41s/it]textattack: Saving checkpoint under "checkpoints\1722894592063.ta.chkpt" at 2024-08-06 05:49:52 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 132 / 14 / 150:  50%|██████████▌          | 150/300 [06:05<06:05,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1722894716189.ta.chkpt" at 2024-08-06 05:51:56 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 174 / 20 / 200:  67%|██████████████       | 200/300 [08:06<04:03,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1722894837604.ta.chkpt" at 2024-08-06 05:53:57 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 216 / 24 / 250:  83%|████████████████▋   | 250/300 [10:17<02:03,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1722894968718.ta.chkpt" at 2024-08-06 05:56:08 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 260 / 29 / 300: 100%|████████████████████| 300/300 [12:27<00:00,  2.49s/it]textattack: Saving checkpoint under "checkpoints\1722895098380.ta.chkpt" at 2024-08-06 05:58:18 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 11 / 260 / 29 / 300: 100%|████████████████████| 300/300 [12:27<00:00,  2.49s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 4.06%  |
| Average perturbed word %:     | 8.79%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.89  |
+-------------------------------+--------+


In [194]:
runAttack(clip_lr10_ag, n_samples, folder+'clipagattacklr10'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattacklr10sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 45 / 4 / 50:  17%|████▏                    | 50/300 [01:59<09:57,  2.39s/it]textattack: Saving checkpoint under "checkpoints\1722895218151.ta.chkpt" at 2024-08-06 06:00:18 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 88 / 9 / 100:  33%|███████▋               | 100/300 [04:02<08:05,  2.43s/it]textattack: Saving checkpoint under "checkpoints\1722895341095.ta.chkpt" at 2024-08-06 06:02:21 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 132 / 14 / 150:  50%|██████████▌          | 150/300 [06:06<06:06,  2.44s/it]textattack: Saving checkpoint under "checkpoints\1722895464614.ta.chkpt" at 2024-08-06 06:04:24 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 174 / 20 / 200:  67%|██████████████       | 200/300 [08:12<04:06,  2.46s/it]textattack: Saving checkpoint under "checkpoints\1722895590759.ta.chkpt" at 2024-08-06 06:06:30 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 216 / 24 / 250:  83%|████████████████▋   | 250/300 [10:25<02:05,  2.50s/it]textattack: Saving checkpoint under "checkpoints\1722895723579.ta.chkpt" at 2024-08-06 06:08:43 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 260 / 29 / 300: 100%|████████████████████| 300/300 [12:32<00:00,  2.51s/it]textattack: Saving checkpoint under "checkpoints\1722895850931.ta.chkpt" at 2024-08-06 06:10:50 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 11 / 260 / 29 / 300: 100%|████████████████████| 300/300 [12:32<00:00,  2.51s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 11     |
| Number of failed attacks:     | 260    |
| Number of skipped attacks:    | 29     |
| Original accuracy:            | 90.33% |
| Accuracy under attack:        | 86.67% |
| Attack success rate:          | 4.06%  |
| Average perturbed word %:     | 8.79%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.89  |
+-------------------------------+--------+


In [195]:
runAttack(clip_rf7_ag, n_samples, folder+'clipagattackrf7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 43 / 5 / 50:  17%|████▏                    | 50/300 [01:56<09:41,  2.33s/it]textattack: Saving checkpoint under "checkpoints\1722895967732.ta.chkpt" at 2024-08-06 06:12:47 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 81 / 13 / 100:  33%|███████▎              | 100/300 [03:56<07:52,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1722896087702.ta.chkpt" at 2024-08-06 06:14:47 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 122 / 19 / 150:  50%|██████████▌          | 150/300 [05:54<05:54,  2.36s/it]textattack: Saving checkpoint under "checkpoints\1722896205916.ta.chkpt" at 2024-08-06 06:16:45 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 153 / 35 / 200:  67%|█████████████▎      | 200/300 [07:20<03:40,  2.20s/it]textattack: Saving checkpoint under "checkpoints\1722896291526.ta.chkpt" at 2024-08-06 06:18:11 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 15 / 191 / 44 / 250:  83%|████████████████▋   | 250/300 [09:15<01:51,  2.22s/it]textattack: Saving checkpoint under "checkpoints\1722896406646.ta.chkpt" at 2024-08-06 06:20:06 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 235 / 49 / 300: 100%|████████████████████| 300/300 [11:24<00:00,  2.28s/it]textattack: Saving checkpoint under "checkpoints\1722896536431.ta.chkpt" at 2024-08-06 06:22:16 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 16 / 235 / 49 / 300: 100%|████████████████████| 300/300 [11:24<00:00,  2.28s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 16     |
| Number of failed attacks:     | 235    |
| Number of skipped attacks:    | 49     |
| Original accuracy:            | 83.67% |
| Accuracy under attack:        | 78.33% |
| Attack success rate:          | 6.37%  |
| Average perturbed word %:     | 4.52%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.15  |
+-------------------------------+--------+


In [196]:
runAttack(clip_rf14_ag, n_samples, folder+'clipagattackrf14'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf14sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 3 / 45 / 2 / 50:  17%|████▏                    | 50/300 [02:13<11:09,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1722896670462.ta.chkpt" at 2024-08-06 06:24:30 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 87 / 7 / 100:  33%|███████▋               | 100/300 [04:20<08:41,  2.61s/it]textattack: Saving checkpoint under "checkpoints\1722896797089.ta.chkpt" at 2024-08-06 06:26:37 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 129 / 13 / 150:  50%|██████████▌          | 150/300 [06:20<06:20,  2.54s/it]textattack: Saving checkpoint under "checkpoints\1722896917082.ta.chkpt" at 2024-08-06 06:28:37 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 12 / 165 / 23 / 200:  67%|█████████████▎      | 200/300 [08:10<04:05,  2.45s/it]textattack: Saving checkpoint under "checkpoints\1722897026962.ta.chkpt" at 2024-08-06 06:30:26 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 207 / 29 / 250:  83%|████████████████▋   | 250/300 [10:19<02:03,  2.48s/it]textattack: Saving checkpoint under "checkpoints\1722897156471.ta.chkpt" at 2024-08-06 06:32:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 251 / 35 / 300: 100%|████████████████████| 300/300 [12:22<00:00,  2.47s/it]textattack: Saving checkpoint under "checkpoints\1722897279049.ta.chkpt" at 2024-08-06 06:34:39 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 14 / 251 / 35 / 300: 100%|████████████████████| 300/300 [12:22<00:00,  2.48s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 251    |
| Number of skipped attacks:    | 35     |
| Original accuracy:            | 88.33% |
| Accuracy under attack:        | 83.67% |
| Attack success rate:          | 5.28%  |
| Average perturbed word %:     | 5.31%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.39  |
+-------------------------------+--------+


In [197]:
runAttack(clip_rf30_ag, n_samples, folder+'clipagattackrf30'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackrf30sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 46 / 2 / 50:  17%|████▏                    | 50/300 [02:20<11:43,  2.81s/it]textattack: Saving checkpoint under "checkpoints\1722897419931.ta.chkpt" at 2024-08-06 06:36:59 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 88 / 7 / 100:  33%|███████▋               | 100/300 [04:38<09:17,  2.79s/it]textattack: Saving checkpoint under "checkpoints\1722897558187.ta.chkpt" at 2024-08-06 06:39:18 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 130 / 10 / 150:  50%|██████████          | 150/300 [06:56<06:56,  2.77s/it]textattack: Saving checkpoint under "checkpoints\1722897695413.ta.chkpt" at 2024-08-06 06:41:35 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 166 / 20 / 200:  67%|█████████████▎      | 200/300 [08:55<04:27,  2.68s/it]textattack: Saving checkpoint under "checkpoints\1722897815118.ta.chkpt" at 2024-08-06 06:43:35 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 207 / 25 / 250:  83%|████████████████▋   | 250/300 [11:12<02:14,  2.69s/it]textattack: Saving checkpoint under "checkpoints\1722897951674.ta.chkpt" at 2024-08-06 06:45:51 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 19 / 250 / 31 / 300: 100%|████████████████████| 300/300 [13:17<00:00,  2.66s/it]textattack: Saving checkpoint under "checkpoints\1722898076676.ta.chkpt" at 2024-08-06 06:47:56 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 19 / 250 / 31 / 300: 100%|████████████████████| 300/300 [13:17<00:00,  2.66s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 19     |
| Number of failed attacks:     | 250    |
| Number of skipped attacks:    | 31     |
| Original accuracy:            | 89.67% |
| Accuracy under attack:        | 83.33% |
| Attack success rate:          | 7.06%  |
| Average perturbed word %:     | 6.79%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.46  |
+-------------------------------+--------+


In [198]:
runAttack(clip_knn7_ag, n_samples, folder+'clipagattackknn7'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn7sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 0 / 49 / 1 / 50:  17%|███▊                   | 50/300 [32:34<2:42:53, 39.09s/it]textattack: Saving checkpoint under "checkpoints\1722900031447.ta.chkpt" at 2024-08-06 07:20:31 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 95 / 4 / 100:  33%|███████              | 100/300 [35:34<1:11:08, 21.34s/it]textattack: Saving checkpoint under "checkpoints\1722900210876.ta.chkpt" at 2024-08-06 07:23:30 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 141 / 7 / 150:  50%|███████████           | 150/300 [38:21<38:21, 15.34s/it]textattack: Saving checkpoint under "checkpoints\1722900378415.ta.chkpt" at 2024-08-06 07:26:18 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 184 / 14 / 200:  67%|██████████████       | 200/300 [41:07<20:33, 12.34s/it]textattack: Saving checkpoint under "checkpoints\1722900544066.ta.chkpt" at 2024-08-06 07:29:04 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 230 / 17 / 250:  83%|█████████████████▌   | 250/300 [44:12<08:50, 10.61s/it]textattack: Saving checkpoint under "checkpoints\1722900729431.ta.chkpt" at 2024-08-06 07:32:09 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 274 / 22 / 300: 100%|█████████████████████| 300/300 [46:54<00:00,  9.38s/it]textattack: Saving checkpoint under "checkpoints\1722900891767.ta.chkpt" at 2024-08-06 07:34:51 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 4 / 274 / 22 / 300: 100%|█████████████████████| 300/300 [46:55<00:00,  9.38s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 274    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 91.33% |
| Attack success rate:          | 1.44%  |
| Average perturbed word %:     | 2.83%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.68  |
+-------------------------------+--------+


In [199]:
runAttack(clip_knn35_ag, n_samples, folder+'clipagattackknn35'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn35sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 49 / 0 / 50:  17%|████▏                    | 50/300 [02:50<14:11,  3.41s/it]textattack: Saving checkpoint under "checkpoints\1722901062430.ta.chkpt" at 2024-08-06 07:37:42 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 94 / 3 / 100:  33%|███████▋               | 100/300 [05:52<11:45,  3.53s/it]textattack: Saving checkpoint under "checkpoints\1722901244936.ta.chkpt" at 2024-08-06 07:40:44 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 3 / 141 / 6 / 150:  50%|███████████           | 150/300 [08:41<08:41,  3.48s/it]textattack: Saving checkpoint under "checkpoints\1722901413865.ta.chkpt" at 2024-08-06 07:43:33 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 181 / 13 / 200:  67%|██████████████       | 200/300 [11:22<05:41,  3.41s/it]textattack: Saving checkpoint under "checkpoints\1722901575031.ta.chkpt" at 2024-08-06 07:46:15 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 227 / 16 / 250:  83%|█████████████████▌   | 250/300 [14:26<02:53,  3.46s/it]textattack: Saving checkpoint under "checkpoints\1722901758148.ta.chkpt" at 2024-08-06 07:49:18 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 270 / 22 / 300: 100%|█████████████████████| 300/300 [17:06<00:00,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1722901918161.ta.chkpt" at 2024-08-06 07:51:58 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 270 / 22 / 300: 100%|█████████████████████| 300/300 [17:06<00:00,  3.42s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 270    |
| Number of skipped attacks:    | 22     |
| Original accuracy:            | 92.67% |
| Accuracy under attack:        | 90.0%  |
| Attack success rate:          | 2.88%  |
| Average perturbed word %:     | 3.96%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.77  |
+-------------------------------+--------+


In [200]:
runAttack(clip_knn175_ag, n_samples, folder+'clipagattackknn175'+suffix, target_ds_ag, minsim, thre)

textattack: Unknown if model of class <class 'transformers.models.clip.modeling_clip.CLIPTextTransformer'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/clipagattackknn175sim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 1 / 48 / 1 / 50:  17%|████▏                    | 50/300 [02:51<14:15,  3.42s/it]textattack: Saving checkpoint under "checkpoints\1722902089525.ta.chkpt" at 2024-08-06 07:54:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 4 / 91 / 5 / 100:  33%|███████▋               | 100/300 [05:47<11:35,  3.48s/it]textattack: Saving checkpoint under "checkpoints\1722902266182.ta.chkpt" at 2024-08-06 07:57:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 6 / 135 / 9 / 150:  50%|███████████           | 150/300 [08:35<08:35,  3.44s/it]textattack: Saving checkpoint under "checkpoints\1722902433841.ta.chkpt" at 2024-08-06 08:00:33 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 172 / 21 / 200:  67%|██████████████       | 200/300 [10:56<05:28,  3.28s/it]textattack: Saving checkpoint under "checkpoints\1722902575293.ta.chkpt" at 2024-08-06 08:02:55 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 214 / 28 / 250:  83%|█████████████████▌   | 250/300 [13:47<02:45,  3.31s/it]textattack: Saving checkpoint under "checkpoints\1722902745716.ta.chkpt" at 2024-08-06 08:05:45 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 258 / 34 / 300: 100%|█████████████████████| 300/300 [16:32<00:00,  3.31s/it]textattack: Saving checkpoint under "checkpoints\1722902910776.ta.chkpt" at 2024-08-06 08:08:30 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 8 / 258 / 34 / 300: 100%|█████████████████████| 300/300 [16:32<00:00,  3.31s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 8      |
| Number of failed attacks:     | 258    |
| Number of skipped attacks:    | 34     |
| Original accuracy:            | 88.67% |
| Accuracy under attack:        | 86.0%  |
| Attack success rate:          | 3.01%  |
| Average perturbed word %:     | 5.13%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.36  |
+-------------------------------+--------+


In [201]:
et9 = datetime.datetime.now()
et9 - et8

datetime.timedelta(seconds=20353, microseconds=401670)

### Fine-tuned

In [202]:
# bert-based-uncased
bert_ft_path = 'textattack/bert-base-uncased-ag-news'

model_bert_ft = transformers.AutoModelForSequenceClassification.from_pretrained(bert_ft_path)
tokenizer_bert_ft = transformers.AutoTokenizer.from_pretrained(bert_ft_path)
wrapper_bert_ft = HuggingFaceModelWrapper(model_bert_ft, tokenizer_bert_ft)

D:\anaconda3\envs\tf\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [203]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep0809.csv', target_ds_ag, 0.5, 1-0.6/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptep0809.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8090140682897256
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 10 / 38 / 2 / 50:  17%|███▎                | 50/300 [1:05:05<5:25:25, 78.10s/it]textattack: Saving checkpoint under "checkpoints\1722906829795.ta.chkpt" at 2024-08-06 09:13:49 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 76 / 6 / 100:  33%|██████            | 100/300 [2:10:43<4:21:26, 78.43s/it]textattack: Saving checkpoint under "checkpoints\1722910768087.ta.chkpt" at 2024-08-06 10:19:28 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 32 / 110 / 8 / 150:  50%|████████▌        | 150/300 [3:16:10<3:16:10, 78.47s/it]textattack: Saving checkpoint under "checkpoints\1722914695627.ta.chkpt" at 2024-08-06 11:24:55 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 43 / 146 / 11 / 200:  67%|██████████▋     | 200/300 [4:24:01<2:12:00, 79.21s/it]textattack: Saving checkpoint under "checkpoints\1722918766550.ta.chkpt" at 2024-08-06 12:32:46 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 54 / 181 / 15 / 250:  83%|█████████████▎  | 250/300 [5:34:48<1:06:57, 80.35s/it]textattack: Saving checkpoint under "checkpoints\1722923012891.ta.chkpt" at 2024-08-06 13:43:32 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 66 / 215 / 19 / 300: 100%|██████████████████| 300/300 [6:39:09<00:00, 79.83s/it]textattack: Saving checkpoint under "checkpoints\1722926873967.ta.chkpt" at 2024-08-06 14:47:53 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 66 / 215 / 19 / 300: 100%|██████████████████| 300/300 [6:39:09<00:00, 79.83s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 66     |
| Number of failed attacks:     | 215    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 71.67% |
| Attack success rate:          | 23.49% |
| Average perturbed word %:     | 17.42% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 77.05  |
+-------------------------------+--------+


In [204]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep0841.csv', target_ds_ag, 0.5, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptep0841.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 9 / 39 / 2 / 50:  17%|███▌                 | 50/300 [1:01:59<5:09:55, 74.38s/it]textattack: Saving checkpoint under "checkpoints\1722930593215.ta.chkpt" at 2024-08-06 15:49:53 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 77 / 6 / 100:  33%|██████            | 100/300 [2:06:18<4:12:37, 75.79s/it]textattack: Saving checkpoint under "checkpoints\1722934452648.ta.chkpt" at 2024-08-06 16:54:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 30 / 112 / 8 / 150:  50%|████████▌        | 150/300 [3:10:52<3:10:52, 76.35s/it]textattack: Saving checkpoint under "checkpoints\1722938326500.ta.chkpt" at 2024-08-06 17:58:46 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 148 / 11 / 200:  67%|██████████▋     | 200/300 [4:19:00<2:09:30, 77.70s/it]textattack: Saving checkpoint under "checkpoints\1722942414118.ta.chkpt" at 2024-08-06 19:06:54 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 185 / 15 / 250:  83%|█████████████▎  | 250/300 [5:31:59<1:06:23, 79.68s/it]textattack: Saving checkpoint under "checkpoints\1722946793845.ta.chkpt" at 2024-08-06 20:19:53 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 62 / 219 / 19 / 300: 100%|██████████████████| 300/300 [6:36:21<00:00, 79.27s/it]textattack: Saving checkpoint under "checkpoints\1722950655831.ta.chkpt" at 2024-08-06 21:24:15 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 62 / 219 / 19 / 300: 100%|██████████████████| 300/300 [6:36:21<00:00, 79.27s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 62     |
| Number of failed attacks:     | 219    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 73.0%  |
| Attack success rate:          | 22.06% |
| Average perturbed word %:     | 16.69% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 76.23  |
+-------------------------------+--------+


In [205]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep0873.csv', target_ds_ag, 0.5, 1-0.4/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptep0873.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8726760455264837
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 7 / 41 / 2 / 50:  17%|███▌                 | 50/300 [1:01:54<5:09:34, 74.30s/it]textattack: Saving checkpoint under "checkpoints\1722954370897.ta.chkpt" at 2024-08-06 22:26:10 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 13 / 81 / 6 / 100:  33%|██████            | 100/300 [2:03:00<4:06:00, 73.80s/it]textattack: Saving checkpoint under "checkpoints\1722958036215.ta.chkpt" at 2024-08-06 23:27:16 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 117 / 8 / 150:  50%|████████▌        | 150/300 [3:04:59<3:04:59, 74.00s/it]textattack: Saving checkpoint under "checkpoints\1722961755745.ta.chkpt" at 2024-08-07 00:29:15 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 35 / 154 / 11 / 200:  67%|██████████▋     | 200/300 [4:09:45<2:04:52, 74.93s/it]textattack: Saving checkpoint under "checkpoints\1722965641382.ta.chkpt" at 2024-08-07 01:34:01 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 44 / 191 / 15 / 250:  83%|█████████████▎  | 250/300 [5:16:01<1:03:12, 75.85s/it]textattack: Saving checkpoint under "checkpoints\1722969617937.ta.chkpt" at 2024-08-07 02:40:17 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 53 / 228 / 19 / 300: 100%|██████████████████| 300/300 [6:16:30<00:00, 75.30s/it]textattack: Saving checkpoint under "checkpoints\1722973245966.ta.chkpt" at 2024-08-07 03:40:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 53 / 228 / 19 / 300: 100%|██████████████████| 300/300 [6:16:30<00:00, 75.30s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 53     |
| Number of failed attacks:     | 228    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 76.0%  |
| Attack success rate:          | 18.86% |
| Average perturbed word %:     | 15.81% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 72.01  |
+-------------------------------+--------+


In [206]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep09.csv', target_ds_ag, 0.5, 0.9)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptep09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (column

[Succeeded / Failed / Skipped / Total] 5 / 43 / 2 / 50:  17%|███▊                   | 50/300 [50:47<4:13:57, 60.95s/it]textattack: Saving checkpoint under "checkpoints\1722976293580.ta.chkpt" at 2024-08-07 04:31:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 8 / 86 / 6 / 100:  33%|██████▎            | 100/300 [1:43:00<3:26:00, 61.80s/it]textattack: Saving checkpoint under "checkpoints\1722979426107.ta.chkpt" at 2024-08-07 05:23:46 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 16 / 126 / 8 / 150:  50%|████████▌        | 150/300 [2:36:44<2:36:44, 62.69s/it]textattack: Saving checkpoint under "checkpoints\1722982650181.ta.chkpt" at 2024-08-07 06:17:30 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 23 / 166 / 11 / 200:  67%|██████████▋     | 200/300 [3:33:09<1:46:34, 63.95s/it]textattack: Saving checkpoint under "checkpoints\1722986035164.ta.chkpt" at 2024-08-07 07:13:55 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 28 / 207 / 15 / 250:  83%|███████████████   | 250/300 [4:30:06<54:01, 64.83s/it]textattack: Saving checkpoint under "checkpoints\1722989452841.ta.chkpt" at 2024-08-07 08:10:52 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 36 / 245 / 19 / 300: 100%|██████████████████| 300/300 [5:21:23<00:00, 64.28s/it]textattack: Saving checkpoint under "checkpoints\1722992529175.ta.chkpt" at 2024-08-07 09:02:09 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 36 / 245 / 19 / 300: 100%|██████████████████| 300/300 [5:21:23<00:00, 64.28s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 36     |
| Number of failed attacks:     | 245    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 81.67% |
| Attack success rate:          | 12.81% |
| Average perturbed word %:     | 12.7%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 61.77  |
+-------------------------------+--------+


In [207]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptep095.csv', target_ds_ag, 0.5, 0.95)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptep095.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.95
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
        (colum

[Succeeded / Failed / Skipped / Total] 0 / 48 / 2 / 50:  17%|███▊                   | 50/300 [26:56<2:14:43, 32.34s/it]textattack: Saving checkpoint under "checkpoints\1722994146147.ta.chkpt" at 2024-08-07 09:29:06 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 94 / 6 / 100:  33%|███████              | 100/300 [54:31<1:49:03, 32.72s/it]textattack: Saving checkpoint under "checkpoints\1722995801197.ta.chkpt" at 2024-08-07 09:56:41 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 142 / 8 / 150:  50%|█████████         | 150/300 [1:23:02<1:23:02, 33.22s/it]textattack: Saving checkpoint under "checkpoints\1722997511933.ta.chkpt" at 2024-08-07 10:25:11 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 0 / 189 / 11 / 200:  67%|████████████▋      | 200/300 [1:51:08<55:34, 33.34s/it]textattack: Saving checkpoint under "checkpoints\1722999198059.ta.chkpt" at 2024-08-07 10:53:18 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 234 / 15 / 250:  83%|███████████████▊   | 250/300 [2:19:26<27:53, 33.47s/it]textattack: Saving checkpoint under "checkpoints\1723000896082.ta.chkpt" at 2024-08-07 11:21:36 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 1 / 280 / 19 / 300: 100%|███████████████████| 300/300 [2:47:36<00:00, 33.52s/it]textattack: Saving checkpoint under "checkpoints\1723002585982.ta.chkpt" at 2024-08-07 11:49:45 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 1 / 280 / 19 / 300: 100%|███████████████████| 300/300 [2:47:36<00:00, 33.52s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 280    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 93.33% |
| Attack success rate:          | 0.36%  |
| Average perturbed word %:     | 2.44%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 30.98  |
+-------------------------------+--------+


In [208]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim06.csv', target_ds_ag, 0.6, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptepsim06.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.6
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 6 / 42 / 2 / 50:  17%|███▊                   | 50/300 [53:27<4:27:18, 64.16s/it]textattack: Saving checkpoint under "checkpoints\1723005793918.ta.chkpt" at 2024-08-07 12:43:13 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 80 / 6 / 100:  33%|██████            | 100/300 [1:48:57<3:37:54, 65.37s/it]textattack: Saving checkpoint under "checkpoints\1723009123477.ta.chkpt" at 2024-08-07 13:38:43 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 24 / 118 / 8 / 150:  50%|████████▌        | 150/300 [2:44:20<2:44:20, 65.74s/it]textattack: Saving checkpoint under "checkpoints\1723012446645.ta.chkpt" at 2024-08-07 14:34:06 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 33 / 156 / 11 / 200:  67%|██████████▋     | 200/300 [3:42:50<1:51:25, 66.85s/it]textattack: Saving checkpoint under "checkpoints\1723015956567.ta.chkpt" at 2024-08-07 15:32:36 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 41 / 194 / 15 / 250:  83%|███████████████   | 250/300 [4:42:02<56:24, 67.69s/it]textattack: Saving checkpoint under "checkpoints\1723019508313.ta.chkpt" at 2024-08-07 16:31:48 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 50 / 231 / 19 / 300: 100%|██████████████████| 300/300 [5:37:19<00:00, 67.47s/it]textattack: Saving checkpoint under "checkpoints\1723022825869.ta.chkpt" at 2024-08-07 17:27:05 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 50 / 231 / 19 / 300: 100%|██████████████████| 300/300 [5:37:19<00:00, 67.47s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 50     |
| Number of failed attacks:     | 231    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 77.0%  |
| Attack success rate:          | 17.79% |
| Average perturbed word %:     | 16.01% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 65.94  |
+-------------------------------+--------+


In [209]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim07.csv', target_ds_ag, 0.7, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptepsim07.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 4 / 44 / 2 / 50:  17%|███▊                   | 50/300 [47:13<3:56:09, 56.68s/it]textattack: Saving checkpoint under "checkpoints\1723025659841.ta.chkpt" at 2024-08-07 18:14:19 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 83 / 6 / 100:  33%|██████            | 100/300 [1:36:01<3:12:03, 57.62s/it]textattack: Saving checkpoint under "checkpoints\1723028587649.ta.chkpt" at 2024-08-07 19:03:07 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 18 / 124 / 8 / 150:  50%|████████▌        | 150/300 [2:24:41<2:24:41, 57.87s/it]textattack: Saving checkpoint under "checkpoints\1723031507160.ta.chkpt" at 2024-08-07 19:51:47 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 25 / 164 / 11 / 200:  67%|██████████▋     | 200/300 [3:15:47<1:37:53, 58.74s/it]textattack: Saving checkpoint under "checkpoints\1723034573373.ta.chkpt" at 2024-08-07 20:42:53 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 31 / 204 / 15 / 250:  83%|███████████████   | 250/300 [4:06:34<49:18, 59.18s/it]textattack: Saving checkpoint under "checkpoints\1723037620371.ta.chkpt" at 2024-08-07 21:33:40 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 38 / 243 / 19 / 300: 100%|██████████████████| 300/300 [4:54:41<00:00, 58.94s/it]textattack: Saving checkpoint under "checkpoints\1723040507082.ta.chkpt" at 2024-08-07 22:21:47 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 38 / 243 / 19 / 300: 100%|██████████████████| 300/300 [4:54:41<00:00, 58.94s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 38     |
| Number of failed attacks:     | 243    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 81.0%  |
| Attack success rate:          | 13.52% |
| Average perturbed word %:     | 13.11% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 57.15  |
+-------------------------------+--------+


In [210]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim08.csv', target_ds_ag, 0.8, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptepsim08.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.8
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 46 / 2 / 50:  17%|███▊                   | 50/300 [40:46<3:23:53, 48.93s/it]textattack: Saving checkpoint under "checkpoints\1723042953876.ta.chkpt" at 2024-08-07 23:02:33 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 5 / 89 / 6 / 100:  33%|██████▎            | 100/300 [1:21:25<2:42:50, 48.85s/it]textattack: Saving checkpoint under "checkpoints\1723045392280.ta.chkpt" at 2024-08-07 23:43:12 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 11 / 131 / 8 / 150:  50%|████████▌        | 150/300 [2:02:21<2:02:21, 48.94s/it]textattack: Saving checkpoint under "checkpoints\1723047848762.ta.chkpt" at 2024-08-08 00:24:08 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 17 / 172 / 11 / 200:  67%|██████████▋     | 200/300 [2:45:21<1:22:40, 49.61s/it]textattack: Saving checkpoint under "checkpoints\1723050428819.ta.chkpt" at 2024-08-08 01:07:08 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 20 / 215 / 15 / 250:  83%|███████████████   | 250/300 [3:28:10<41:38, 49.96s/it]textattack: Saving checkpoint under "checkpoints\1723052997860.ta.chkpt" at 2024-08-08 01:49:57 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 26 / 255 / 19 / 300: 100%|██████████████████| 300/300 [4:08:29<00:00, 49.70s/it]textattack: Saving checkpoint under "checkpoints\1723055417025.ta.chkpt" at 2024-08-08 02:30:17 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 26 / 255 / 19 / 300: 100%|██████████████████| 300/300 [4:08:29<00:00, 49.70s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 26     |
| Number of failed attacks:     | 255    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 85.0%  |
| Attack success rate:          | 9.25%  |
| Average perturbed word %:     | 10.12% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 47.28  |
+-------------------------------+--------+


In [211]:
runAttack(wrapper_bert_ft, n_samples, folder+'bertagattackptepsim09.csv', target_ds_ag, 0.9, 1-0.5/np.pi)

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path ../attack_results_new/agclip/bertagattackptepsim09.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.9
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.8408450569081046
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']


[Succeeded / Failed / Skipped / Total] 2 / 46 / 2 / 50:  17%|███▊                   | 50/300 [32:04<2:40:23, 38.50s/it]textattack: Saving checkpoint under "checkpoints\1723057341913.ta.chkpt" at 2024-08-08 03:02:21 after 50 attacks.


[Succeeded / Failed / Skipped / Total] 2 / 92 / 6 / 100:  33%|██████▎            | 100/300 [1:06:30<2:13:00, 39.90s/it]textattack: Saving checkpoint under "checkpoints\1723059407472.ta.chkpt" at 2024-08-08 03:36:47 after 100 attacks.


[Succeeded / Failed / Skipped / Total] 7 / 135 / 8 / 150:  50%|█████████         | 150/300 [1:39:35<1:39:35, 39.84s/it]textattack: Saving checkpoint under "checkpoints\1723061392417.ta.chkpt" at 2024-08-08 04:09:52 after 150 attacks.


[Succeeded / Failed / Skipped / Total] 9 / 180 / 11 / 200:  67%|███████████▎     | 200/300 [2:14:08<1:07:04, 40.24s/it]textattack: Saving checkpoint under "checkpoints\1723063465980.ta.chkpt" at 2024-08-08 04:44:25 after 200 attacks.


[Succeeded / Failed / Skipped / Total] 10 / 225 / 15 / 250:  83%|███████████████   | 250/300 [2:48:00<33:36, 40.32s/it]textattack: Saving checkpoint under "checkpoints\1723065497593.ta.chkpt" at 2024-08-08 05:18:17 after 250 attacks.


[Succeeded / Failed / Skipped / Total] 14 / 267 / 19 / 300: 100%|██████████████████| 300/300 [3:21:16<00:00, 40.25s/it]textattack: Saving checkpoint under "checkpoints\1723067493445.ta.chkpt" at 2024-08-08 05:51:33 after 300 attacks.
[Succeeded / Failed / Skipped / Total] 14 / 267 / 19 / 300: 100%|██████████████████| 300/300 [3:21:16<00:00, 40.25s/it]





+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 14     |
| Number of failed attacks:     | 267    |
| Number of skipped attacks:    | 19     |
| Original accuracy:            | 93.67% |
| Accuracy under attack:        | 89.0%  |
| Attack success rate:          | 4.98%  |
| Average perturbed word %:     | 6.04%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 37.93  |
+-------------------------------+--------+


In [212]:
et10 = datetime.datetime.now()
et10 - et9

datetime.timedelta(days=1, seconds=78182, microseconds=353898)

In [48]:
len(nn_cf[0])

50